In [1]:
import pandas as pd
# Excelファイルを読み込む
df = pd.read_excel(r"C:\Users\tears\OneDrive\2025\2511\01_MV\ver1_20251111-1.xlsx")
print(df.columns)

Index(['INDEX', '施設名', '人工呼吸_連続日数', 'Base_Date', 'Cox_Event', 'All_Day',
       'IntubationDate', 'Department', 'Age', 'Male', 'BMI', 'Dialysis',
       'CirculatoryDevice', 'Hypothermia', 'CPR', 'Day0_ALT', 'Day0_AST',
       'Day0_Alb', 'Day0_APTT', 'Day0_BUN', 'Day0_CK', 'Day0_CRP', 'Day0_Cl',
       'Day0_Cre', 'Day0_D-dimer', 'Day0_Hb', 'Day0_K', 'Day0_Na', 'Day0_PLT',
       'Day0_PT-INR', 'Day0_T-Bil', 'Day0_TP', 'Day0_WBC', 'Day3_ALT',
       'Day3_AST', 'Day3_Alb', 'Day3_APTT', 'Day3_BUN', 'Day3_CK', 'Day3_CRP',
       'Day3_Cl', 'Day3_Cre', 'Day3_D-dimer', 'Day3_Hb', 'Day3_K', 'Day3_Na',
       'Day3_PLT', 'Day3_PT-INR', 'Day3_T-Bil', 'Day3_TP', 'Day3_WBC',
       'Day0_CoreSed', 'Day0_Opioid', 'Day0_NAD', 'Day0_Adrenaline',
       'Day0_DOA', 'Day0_DOB', 'Day0_Insulin', 'Day0_Steroid',
       'Day0_Vasopressin', 'Day0_Diuretic', 'Day0_AntiMRSA', 'Day0_Antibiotic',
       'Day0_Carbapenem', 'Day0_Antifungal', 'Day3_CoreSed', 'Day3_Opioid',
       'Day3_NAD', 'Day3_Adrenaline'

In [7]:
# -*- coding: utf-8 -*-
# Histogram (0–21 days, integer bins, KDE overlay, no grid + legend with median [IQR] days)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# ===== パス設定 =====
EXCEL_PATH = r"C:\Users\tears\OneDrive\2025\2511\01_MV\ver1_20251111-1.xlsx"
OUT_DIR = r"C:\Users\tears\OneDrive\2025\2511\01_MV"
OUT_FILE = "Event_Timing_0to21_NoGrid_KDE_Legend_MedianIQRdays.jpg"

# ===== データ読込 =====
df = pd.read_excel(EXCEL_PATH)
trach = df.loc[df["Cox_Event"] == 1, "All_Day"].dropna().astype(float)
death = df.loc[df["Cox_Event"] == 2, "All_Day"].dropna().astype(float)

# 範囲制限
trach = trach[(trach >= 0) & (trach <= 21)]
death = death[(death >= 0) & (death <= 21)]

# ===== 中央値 + IQR =====
def median_iqr_str(values):
    med = np.median(values)
    q25, q75 = np.percentile(values, [25, 75])
    return f"median {med:.0f} [IQR {q25:.0f}–{q75:.0f}] days"

trach_label = f"Tracheostomy ({median_iqr_str(trach)})"
death_label = f"Death ({median_iqr_str(death)})"

# ===== 図設定 =====
fig, ax1 = plt.subplots(figsize=(8, 6))
ax2 = ax1.twinx()
ax1.grid(False)
ax2.grid(False)

# ===== ヒストグラム =====
bins = np.arange(1, 21 + 1, 1.0)
n_t, _, _ = ax1.hist(trach, bins=bins, alpha=0.55, edgecolor="black", label=trach_label, color="steelblue")
n_d, _, _ = ax1.hist(death, bins=bins, alpha=0.55, edgecolor="black", label=death_label, color="sandybrown")

# ===== KDE =====
def plot_kde(values, color, label):
    if len(values) > 1:
        kde = gaussian_kde(values)
        x_grid = np.linspace(0, 21, 400)
        y_kde = kde(x_grid) * len(values)
        ax1.plot(x_grid, y_kde, color=color, lw=2.0, label=f"{label}")

plot_kde(trach, "blue", "Tracheostomy")
plot_kde(death, "red", "Death")

# ===== 軸設定 =====
ax1.set_xlim(1, 21)
ax1.set_xticks(np.arange(1, 22, 1))
ax1.set_xlabel("Days after intubation", fontsize=12)
ax1.set_ylabel("Number of events (n)", fontsize=12)
ymax = max(max(n_t), max(n_d))
ax1.set_ylim(0, ymax * 1.2)
ax2.set_ylim(0, ax1.get_ylim()[1] / len(df) * 100)
ax2.set_ylabel("Relative frequency (%)", fontsize=12)

# ===== 凡例 =====
ax1.legend(
    loc="upper right",
    fontsize=10,
    frameon=False,
    handlelength=1.8,
    handletextpad=0.6,
)

# ===== タイトル・保存 =====
ax1.set_title("Timing Distribution of Tracheostomy and Death", fontsize=14, pad=10)
plt.tight_layout()
os.makedirs(OUT_DIR, exist_ok=True)
outfile = os.path.join(OUT_DIR, OUT_FILE)
plt.savefig(outfile, dpi=600, format="jpeg", bbox_inches="tight")
plt.close()
print(f"✅ 図を保存しました: {outfile}")


✅ 図を保存しました: C:\Users\tears\OneDrive\2025\2511\01_MV\Event_Timing_0to21_NoGrid_KDE_Legend_MedianIQRdays.jpg


20251111

In [2]:
# -*- coding: utf-8 -*-
# =========================================================
# Temporal本線（cutoff=2024-01-01）＋ 施設別nLOGO感度
# Final（3倍計算許容）版：HPO試行/樹本数/ES/CIのみ増量、MIは据え置き
# - LRのみ：VIF（緩和）＋ log/winsor、MI平均、Isotonic(OVR)
# - 木系（XGB/LGBM）：logは条件付き、winsor停止、Isotonic(OVR)、ES付きHPO拡張
# - LM3：Day3-Day0 の Δ特徴を自動生成
# - 診療科：頻度集約してOneHotに追加（Department_en は存続）
# - 出力：OOF、指標、CI、Reliability、DCA、前処理meta、HPO最良パラ、VIFレポート、config
# - SHAP用 analysis_features.* をエクスポート（重複列対策込み）
# - 安定化：単スレッド、逐次del/gc、float32徹底、CSR、HPO都度解放
# =========================================================

from __future__ import annotations
import os, warnings, gc, traceback, json
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)
pd.set_option("display.unicode.east_asian_width", True)

from sklearn.experimental import enable_iterative_imputer  # noqa: F401
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.isotonic import IsotonicRegression
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from sklearn.linear_model import LogisticRegression as LRClf
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score

from scipy.sparse import hstack, csr_matrix
from statsmodels.stats.outliers_influence import variance_inflation_factor

import xgboost as xgb
try:
    import lightgbm as lgb
    HAS_LGB = True
except Exception:
    HAS_LGB = False

try:
    import optuna
    HAS_OPTUNA = True
    from optuna.samplers import TPESampler
    from optuna.pruners import MedianPruner
except Exception:
    HAS_OPTUNA = False

# ---- VSCode/Pylance 向け 安定化パッチ（波線抑止） ----------------------------
from packaging.version import Version
from sklearn import __version__ as _skl_ver
def _make_onehot_encoder(dtype=np.float32):
    """sklearn 1.2+ は sparse_output、それ未満は sparse を使う"""
    if Version(_skl_ver) >= Version("1.2"):
        return OneHotEncoder(handle_unknown="ignore", sparse_output=True, dtype=dtype)
    else:
        return OneHotEncoder(handle_unknown="ignore", sparse=True, dtype=dtype)

if not HAS_LGB:
    class _LGBDummy:
        class Dataset: ...
        def train(*args, **kwargs):
            raise RuntimeError("LightGBM is not available in this environment.")
    lgb = _LGBDummy()  # type: ignore

if not HAS_OPTUNA:
    class _OptunaDummy:
        class Trial: ...
        def create_study(*args, **kwargs):
            raise RuntimeError("Optuna is not available in this environment.")
    optuna = _OptunaDummy()  # type: ignore
# ---------------------------------------------------------------------------

# ================= 実行設定（Light/Final 切替） =================
RUN_MODE = "Final"  # "Light" or "Final"

# 安定化（単スレッド固定）
os.environ["PYTHONHASHSEED"] = "0"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
RNG_SEED = 42
warnings.filterwarnings("ignore")

# ユーザ設定（パスは適宜変更）
EXCEL_PATH = Path(r"C:\Users\tears\OneDrive\2025\2511\01_MV\ver1_20251111-1.xlsx")
SHEET_NAME = 0
RESULTS_DIR = Path("./results_temporal_main"); RESULTS_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR = RESULTS_DIR / "models"; MODELS_DIR.mkdir(parents=True, exist_ok=True)

TEMPORAL_CUTOFF = pd.Timestamp("2024-01-01")
CLASS_NAME_MAP = {0: "Other", 1: "Tracheostomy", 2: "Death"}  # ラベル表示用

# ---- Light / Final の差分パラメータ定義 ----
PARAMS_BY_MODE = {
    "Light": {
        "MIN_TEST_SAMPLES": 50,
        "CAL_FRAC": 0.30,
        "B_BOOT": 50,
        "MI_COUNT": 5,
        "MI_MAXITER": 10,
        "MI_NEAREST": 32,
        "N_TRIALS_LR": 8,
        "N_TRIALS_XGB": 20,
        "N_TRIALS_LGB": 20,
        "LR_max_iter": 3000,
        "LR_C_default": 1.0,
        "XGB_n_estimators": 300,
        "LGB_n_estimators": 300,
        "ES_rounds": 50
    },
    "Final": {
        "MIN_TEST_SAMPLES": 50,
        "CAL_FRAC": 0.30,
        "B_BOOT": 1500,
        "MI_COUNT": 20,
        "MI_MAXITER": 20,
        "MI_NEAREST": 64,
        "N_TRIALS_LR": 90,
        "N_TRIALS_XGB": 200,
        "N_TRIALS_LGB": 200,
        "LR_max_iter": 7000,
        "LR_C_default": 1.0,
        "XGB_n_estimators": 1200,
        "LGB_n_estimators": 1200,
        "ES_rounds": 200
    }
}
if RUN_MODE not in PARAMS_BY_MODE:
    raise ValueError("RUN_MODE は 'Light' または 'Final' を指定してください。")

m = PARAMS_BY_MODE[RUN_MODE]
MIN_TEST_SAMPLES = m["MIN_TEST_SAMPLES"]
CAL_FRAC = m["CAL_FRAC"]
B_BOOT = m["B_BOOT"]
MI_COUNT = m["MI_COUNT"]
MI_MAXITER = m["MI_MAXITER"]
MI_NEAREST = m["MI_NEAREST"]
N_TRIALS_LR = m["N_TRIALS_LR"]
N_TRIALS_XGB = m["N_TRIALS_XGB"]
N_TRIALS_LGB = m["N_TRIALS_LGB"]
ES_ROUNDS = m["ES_rounds"]

LR_PARAMS = dict(
    multi_class="multinomial", solver="saga", penalty="l2",
    C=m["LR_C_default"], max_iter=m["LR_max_iter"], n_jobs=1, random_state=RNG_SEED, tol=1e-4
)
XGB_PARAMS = dict(
    objective="multi:softprob", tree_method="hist", eval_metric="mlogloss",
    learning_rate=0.05, max_depth=6, min_child_weight=2.0,
    subsample=0.85, colsample_bytree=0.85, reg_lambda=1.0,
    n_jobs=1, num_class=None, n_estimators=m["XGB_n_estimators"], verbosity=0, random_state=RNG_SEED
)
LGB_PARAMS = dict(
    objective="multiclass", num_leaves=63, min_child_samples=50,
    feature_fraction=0.85, bagging_fraction=0.85, lambda_l2=1.0,
    learning_rate=0.05, n_estimators=m["LGB_n_estimators"], num_threads=1,
    deterministic=True, force_col_wise=True, verbosity=-1, num_class=None, seed=RNG_SEED
)

# ---- 非正規・外れ値処理の閾値 ----
LOG_SKEW_THRESHOLD = 1.0
LOG_P95P50_RATIO = 2.0
WINSOR_LO_P = 0.01
WINSOR_HI_P = 0.99

# ---- VIF設定（緩和）----
VIF_THRESHOLD = 25.0
VIF_MAX_DROP = 10
USE_VIF_LR = True  # LRのみ適用

# ---- 木系は winsor 停止（logは条件付きで可）----
APPLY_WINSOR_FOR_TREES = False

def log(msg: str):
    print(msg, flush=True)

# 設定保存
(CONFIG_PATH := RESULTS_DIR / "config.json").write_text(
    json.dumps({
        "RUN_MODE": RUN_MODE, "RNG_SEED": RNG_SEED,
        "TEMPORAL_CUTOFF": TEMPORAL_CUTOFF.strftime("%Y-%m-%d"),
        "MIN_TEST_SAMPLES": MIN_TEST_SAMPLES, "CAL_FRAC": CAL_FRAC,
        "MI_COUNT": MI_COUNT, "MI_MAXITER": MI_MAXITER, "MI_NEAREST": MI_NEAREST,
        "N_TRIALS_LR": N_TRIALS_LR, "N_TRIALS_XGB": N_TRIALS_XGB, "N_TRIALS_LGB": N_TRIALS_LGB,
        "Calibration": "Isotonic OVR (unified)",
        "LOG_SKEW_THRESHOLD": LOG_SKEW_THRESHOLD, "LOG_P95P50_RATIO": LOG_P95P50_RATIO,
        "WINSOR_LO_P": WINSOR_LO_P, "WINSOR_HI_P": WINSOR_HI_P,
        "VIF_THRESHOLD": VIF_THRESHOLD, "VIF_MAX_DROP": VIF_MAX_DROP,
        "USE_VIF_LR": USE_VIF_LR, "APPLY_WINSOR_FOR_TREES": APPLY_WINSOR_FOR_TREES,
        "ES_ROUNDS": ES_ROUNDS,
        "LR_params": LR_PARAMS, "XGB_params": XGB_PARAMS, "LGB_params": LGB_PARAMS if HAS_LGB else "N/A",
        "B_BOOT": B_BOOT
    }, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

# ================= メトリクス・補助 =================
def zero_safe_logit(p: np.ndarray, eps: float = 1e-8) -> np.ndarray:
    p = np.clip(p, eps, 1 - eps)
    return np.log(p / (1 - p))

def class_weights_inverse_freq(y: np.ndarray, labels: np.ndarray) -> Dict[int, float]:
    counts = np.array([(y == k).sum() for k in labels], dtype=float)
    inv = 1.0 / np.clip(counts, 1.0, None)
    w = inv / inv.sum() * len(labels)
    return {int(k): float(wi) for k, wi in zip(labels, w)}

def brier_score_mc(y_true: np.ndarray, proba: np.ndarray, labels: np.ndarray) -> float:
    K = len(labels)
    y_oh = np.zeros((len(y_true), K), dtype=np.float32)
    label_to_idx = {int(k): i for i, k in enumerate(labels)}
    for i, y in enumerate(y_true):
        y_oh[i, label_to_idx[int(y)]] = 1.0
    return float(np.mean((y_oh - proba) ** 2))

def ece_confidence(y_true: np.ndarray, proba: np.ndarray, n_bins: int = 10) -> float:
    conf = proba.max(axis=1)
    y_pred = proba.argmax(axis=1)
    correct = (y_pred == y_true).astype(np.float32)
    bins = np.linspace(0.0, 1.0, n_bins + 1, dtype=np.float32)
    ece = 0.0
    for i in range(n_bins):
        lo, hi = float(bins[i]), float(bins[i + 1])
        m = (conf >= lo) & (conf < hi if i < n_bins - 1 else conf <= hi)
        if m.any():
            acc_bin = float(correct[m].mean())
            conf_bin = float(conf[m].mean())
            ece += (m.mean()) * abs(acc_bin - conf_bin)
    return float(ece)

def macro_ovr_auc(y_true: np.ndarray, proba: np.ndarray, labels: np.ndarray):
    aurocs, auprcs = [], []
    for j, k in enumerate(labels):
        y_bin = (y_true == k).astype(int)
        p = proba[:, j]
        if len(np.unique(y_bin)) < 2:
            auroc, auprc = np.nan, np.nan
        else:
            try: auroc = roc_auc_score(y_bin, p)
            except Exception: auroc = np.nan
            try: auprc = average_precision_score(y_bin, p)
            except Exception: auprc = np.nan
        if not np.isnan(auroc): aurocs.append(auroc)
        if not np.isnan(auprc): auprcs.append(auprc)
    return (float(np.mean(aurocs)) if aurocs else np.nan,
            float(np.mean(auprcs)) if auprcs else np.nan)

def compute_calibration_slope_intercept(y_true: np.ndarray, proba: np.ndarray, labels: np.ndarray):
    xs, ys = [], []
    for j, cls in enumerate(labels):
        p = np.clip(proba[:, j], 1e-10, 1-1e-10).reshape(-1, 1)
        x = zero_safe_logit(p).reshape(-1, 1)
        y = (y_true == cls).astype(int)
        xs.append(x); ys.append(y.reshape(-1, 1))
    X = np.vstack(xs).astype(np.float32); Y = np.vstack(ys).ravel().astype(np.float32)
    try:
        lr = LRClf(C=1e6, penalty="l2", solver="lbfgs", max_iter=5000, n_jobs=1, random_state=RNG_SEED, tol=1e-6)
        lr.fit(X, Y)
        slope = float(lr.coef_[0, 0]); intercept = float(lr.intercept_[0])
    except Exception:
        slope, intercept = np.nan, np.nan
    del X, Y, xs, ys; gc.collect()
    return slope, intercept

def bootstrap_ci_stat(arr: np.ndarray, alpha=0.05):
    a = np.sort(arr[~np.isnan(arr)])
    if len(a) == 0: return (np.nan, np.nan, np.nan)
    lo = a[int(np.floor((alpha/2) * (len(a)-1)))]
    hi = a[int(np.ceil((1-alpha/2) * (len(a)-1)))]
    return float(np.mean(a)), float(lo), float(hi)

def bootstrap_metrics(y_true: np.ndarray, proba: np.ndarray, labels: np.ndarray, B: int, seed: int):
    rng = np.random.RandomState(seed)
    mets = {"AUROC_macro": [], "AUPRC_macro": [], "Brier": [], "Accuracy": [], "ECE": []}
    n = len(y_true)
    for _ in range(B):
        idx = rng.randint(0, n, size=n)
        yt = y_true[idx]; pt = proba[idx]
        auroc, auprc = macro_ovr_auc(yt, pt, labels)
        brier = brier_score_mc(yt, pt, labels)
        acc = accuracy_score(yt, pt.argmax(axis=1))
        ece = ece_confidence(yt, pt, 10)
        mets["AUROC_macro"].append(auroc)
        mets["AUPRC_macro"].append(auprc)
        mets["Brier"].append(brier)
        mets["Accuracy"].append(acc)
        mets["ECE"].append(ece)
    out = {k: np.array(v, dtype=np.float32) for k, v in mets.items()}
    del mets; gc.collect()
    return out

# ================= Isotonic OVR =================
def fit_isotonic_ovr(P_cal: np.ndarray, y_cal: np.ndarray, labels: np.ndarray):
    models = []
    for j, cls in enumerate(labels):
        y_bin = (y_cal == cls).astype(int)
        iso = IsotonicRegression(out_of_bounds="clip")
        pj = np.clip(P_cal[:, j], 1e-12, 1 - 1e-12)
        try:
            iso.fit(pj, y_bin)
        except Exception:
            class _IdentityISO:
                def transform(self, x): return x
            iso = _IdentityISO()
        models.append(iso)
    return models

def apply_isotonic_ovr(P_in: np.ndarray, models) -> np.ndarray:
    P = np.clip(P_in, 1e-12, 1.0).astype(np.float32, copy=True)
    K = P.shape[1]
    P_new = np.zeros_like(P, dtype=np.float32)
    for j in range(K):
        P_new[:, j] = np.asarray(models[j].transform(P[:, j]), dtype=np.float32)
    row_sum = P_new.sum(axis=1, keepdims=True)
    zero_mask = (row_sum <= 0)
    if np.any(zero_mask):
        P_new[zero_mask, :] = 1.0 / K
        row_sum = P_new.sum(axis=1, keepdims=True)
    P_new = P_new / row_sum
    P_new = np.clip(P_new, 1e-12, 1.0)
    P_new = P_new / P_new.sum(axis=1, keepdims=True)
    return P_new.astype(np.float32)

# ================= 前処理・特徴構成 =================
def prepare_department_en(df: pd.DataFrame) -> pd.DataFrame:
    """
    Department/診療科名 を英語4カテゴリに正規化:
      - Cardio: 心臓, 循環, 循環器, 心臓血管, cardiovascular, cardio, cardiac, cv
      - Neuro : 脳, 神経, 脳神経, neuro
      - Emergency: 救, 救急, ER, emerg, emergency
      - Other : 上記以外
    また、診療科の詳細は頻度で集約して Dept_detail に格納。
    """
    def _contains_any(text: str, keywords: list[str]) -> bool:
        if not isinstance(text, str):
            return False
        t_low = text.lower()
        for kw in keywords:
            if any(ord(ch) > 127 for ch in kw):  # 日本語キーワード
                if kw in text:
                    return True
            else:
                if kw in t_low:
                    return True
        return False

    if "Department" in df.columns:
        src = df["Department"].astype(str).fillna("Other")
    elif "診療科名" in df.columns:
        src = df["診療科名"].astype(str).fillna("Other")
    else:
        src = pd.Series(["Other"] * len(df), index=df.index, dtype="object")

    cardio_kw = ["心臓", "循環", "循環器", "心臓血管", "cardio", "cardiac", "cardiovascular", "cv"]
    neuro_kw  = ["脳", "神経", "脳神経", "neuro"]
    emerg_kw  = ["救", "救急", "er", "emerg", "emergency"]

    def map_dep(x: str) -> str:
        if _contains_any(x, cardio_kw):
            return "Cardio"
        if _contains_any(x, neuro_kw):
            return "Neuro"
        if _contains_any(x, emerg_kw):
            return "Emergency"
        return "Other"

    df = df.copy()
    df["Department_en"] = src.map(map_dep)

    if "診療科名" in df.columns:
        s = df["診療科名"].astype(str).fillna("Unknown")
        freq = s.value_counts()
        keep = set(freq[freq >= 20].index)
        df["Dept_detail"] = s.where(s.isin(keep), other="OtherMinor")
    elif "Department" in df.columns:
        s = df["Department"].astype(str).fillna("Unknown")
        freq = s.value_counts()
        keep = set(freq[freq >= 20].index)
        df["Dept_detail"] = s.where(s.isin(keep), other="OtherMinor")
    else:
        df["Dept_detail"] = "Unknown"

    return df

def select_columns_by_day(df: pd.DataFrame, day: str):
    # 提示列に合わせて更新（ハイフン表記に統一）
    lab_names = [
        "ALT","AST","Alb","APTT","BUN","CK","CRP","Cl","Cre","D-dimer",
        "Hb","K","Na","PLT","PT-INR","T-Bil","TP","WBC"
    ]
    med_names = [
        "CoreSed","Opioid","NAD","Adrenaline","DOA","DOB","FFP","Plt","RBC","Insulin",
        "Steroid","Vasopressin","Diuretic","AntiMRSA","Antibiotic","Carbapenem","Antifungal"
    ]
    labs = [f"{day}_{n}" for n in lab_names if f"{day}_{n}" in df.columns]
    meds = [f"{day}_{n}" for n in med_names if f"{day}_{n}" in df.columns]
    return labs, meds

def add_delta_features(df: pd.DataFrame) -> Tuple[pd.DataFrame, List[str]]:
    d0_labs, d0_meds = select_columns_by_day(df, "Day0")
    d3_labs, d3_meds = select_columns_by_day(df, "Day3")
    common = sorted(set(d3_labs + d3_meds).intersection(set(d0_labs + d0_meds)))
    new_cols = []
    out = df.copy()
    for c3 in common:
        base = c3.replace("Day3_", "")
        c0 = f"Day0_{base}"
        if c3 in out.columns and c0 in out.columns:
            new_c = f"Delta_{base}"
            out[new_c] = pd.to_numeric(out[c3], errors="coerce") - pd.to_numeric(out[c0], errors="coerce")
            new_cols.append(new_c)
    return out, new_cols

def _find_date_column(df: pd.DataFrame) -> str:
    for c in ["Base_Date", "BaseDate", "Date"]:
        if c in df.columns:
            return c
    raise KeyError("Temporal splitに用いる日付列（Base_Date/BaseDate/Date）が見つかりません。")

def build_feature_matrix(df: pd.DataFrame, landmark: str):
    # ==== 予測に入れるベース変数（Emergencyは削除） ====
    base_cols = [c for c in ["Age","Male","BMI","Dialysis","CirculatoryDevice","Hypothermia","CPR"] if c in df.columns]
    df = prepare_department_en(df.copy())
    base_cols += ["Department_en", "Dept_detail"]

    day0_labs, day0_meds = select_columns_by_day(df, "Day0")
    used = base_cols + day0_labs + day0_meds
    if landmark == "LM3":
        day3_labs, day3_meds = select_columns_by_day(df, "Day3")
        used += day3_labs + day3_meds
        df, delta_cols = add_delta_features(df)
        used += delta_cols

    # ==== 予測に入れてはいけない列を明示除外（ログ表示のみ。Xには含めない） ====
    for c in [c for c in ["INDEX","施設名","人工呼吸_連続日数","Base_Date","Cox_Event","All_Day","IntubationDate"] if c in df.columns]:
        log(f"[preprocess] exclude-from-features: {c}")

    # 学習・評価で必要な付帯列は保持（Xには入れない）
    needed = ["INDEX","施設名","All_Day","Cox_Event"] + [c for c in used if c in df.columns]

    date_col = ""
    if any(c in df.columns for c in ["Base_Date","BaseDate","Date"]):
        try:
            date_col = _find_date_column(df); needed.append(date_col)
        except KeyError:
            date_col = ""

    work = df[needed].copy()
    # LM0: All_Day>=0, LM3: All_Day>=4（All_Dayは特徴には入れない）
    work = work[work["All_Day"] >= (0 if landmark=="LM0" else 4)]

    dates = None
    if date_col:
        work[date_col] = pd.to_datetime(work[date_col], errors="coerce")
        dates = work[date_col].values

    X = work[[c for c in used if c in work.columns]].copy()

    for c in X.columns:
        if c in ["Department_en","Dept_detail"]:
            X[c] = X[c].astype(str).fillna("Unknown")
        else:
            X[c] = pd.to_numeric(X[c], errors="coerce").astype("float32")

    y = work["Cox_Event"].astype(int).values
    groups = work["施設名"].astype(str).values
    used_cols = list(X.columns)
    index_ids = work["INDEX"].values
    return X, y, groups, used_cols, index_ids, dates, (date_col or "")

# ===== 数値変換パラメータ学習 =====
def learn_numeric_transform_params(df_train: pd.DataFrame, num_cols: List[str]) -> Dict[str, dict]:
    meta = {}
    for c in num_cols:
        s = pd.to_numeric(df_train[c], errors="coerce")
        s_non = pd.Series(s).dropna()
        if s_non.size == 0:
            meta[c] = dict(use_log=False, offset=0.0, winsor_lo=np.nan, winsor_hi=np.nan, skew=0.0, p95_over_p50=np.nan)
            continue
        skew = float(s_non.skew()) if s_non.size > 2 else 0.0
        q50 = float(np.nanpercentile(s_non, 50))
        q95 = float(np.nanpercentile(s_non, 95))
        ratio = (q95 / max(q50, 1e-8)) if q50 != 0 else np.inf

        use_log = (skew > LOG_SKEW_THRESHOLD) or (ratio > LOG_P95P50_RATIO)
        min_val = float(np.nanmin(s_non))
        offset = 0.0
        if use_log and (min_val <= 0):
            offset = abs(min_val) + 1e-6

        s_work = s_non.copy()
        if use_log:
            s_work = np.log1p(s_work + offset)

        lo = float(np.nanpercentile(s_work, WINSOR_LO_P*100)) if s_work.size else np.nan
        hi = float(np.nanpercentile(s_work, WINSOR_HI_P*100)) if s_work.size else np.nan

        meta[c] = dict(use_log=bool(use_log), offset=float(offset), winsor_lo=lo, winsor_hi=hi,
                       skew=skew, p95_over_p50=ratio)
    return meta

def apply_numeric_transform(df: pd.DataFrame, meta: Dict[str, dict], num_cols: List[str], model_name: str) -> pd.DataFrame:
    X = df.copy()
    for c in num_cols:
        if c not in X.columns: continue
        s = pd.to_numeric(X[c], errors="coerce")
        m = meta.get(c, None)
        if m is None:
            X[c] = s.astype("float32")
            continue
        if m["use_log"]:
            s = np.log1p(s + m["offset"])
        if (model_name == "Logistic") or (model_name == "LR"):
            lo, hi = m["winsor_lo"], m["winsor_hi"]
            if not (np.isnan(lo) or np.isnan(hi)):
                s = s.clip(lower=lo, upper=hi)
        X[c] = s.astype("float32")
    return X

def encode_fit_transform_sparse(train_df: pd.DataFrame, test_df: pd.DataFrame, cat_cols: List[str]):
    enc = _make_onehot_encoder(dtype=np.float32)
    if len(cat_cols):
        enc.fit(train_df[cat_cols])
        Xtr_cat = enc.transform(train_df[cat_cols])
        Xte_cat = enc.transform(test_df[cat_cols])
    else:
        Xtr_cat = csr_matrix((len(train_df), 0), dtype=np.float32)
        Xte_cat = csr_matrix((len(test_df), 0), dtype=np.float32)
        class _Empty:
            def get_feature_names_out(self, *args, **kwargs): return np.array([], dtype=str)
            def transform(self, X): return Xtr_cat
        enc = _Empty()
    num_cols = [c for c in train_df.columns if c not in cat_cols]
    Xtr_num = train_df[num_cols].to_numpy(dtype=np.float32)
    Xte_num = test_df[num_cols].to_numpy(dtype=np.float32)
    cat_names = list(enc.get_feature_names_out(cat_cols)) if hasattr(enc, "get_feature_names_out") else []
    feature_names = num_cols + cat_names
    return Xtr_num, Xte_num, Xtr_cat, Xte_cat, enc, num_cols, feature_names

def _csr_from_df_with_onehot(df: pd.DataFrame, enc: OneHotEncoder, cat_cols: List[str]) -> csr_matrix:
    if len(cat_cols):
        X_cat = enc.transform(df[cat_cols])
    else:
        X_cat = csr_matrix((len(df), 0), dtype=np.float32)
    X_num = df[[c for c in df.columns if c not in cat_cols]].to_numpy(dtype=np.float32, copy=False)
    return hstack([csr_matrix(X_num), X_cat], format="csr")

# ================= VIF（LRのみ） =================
def compute_vif_table_numeric(X_num_df: pd.DataFrame) -> pd.DataFrame:
    simp = SimpleImputer(strategy="median")
    Xn = simp.fit_transform(X_num_df)
    var = Xn.var(axis=0)
    keep = var > 0
    Xn = Xn[:, keep]
    cols = [c for c, k in zip(X_num_df.columns, keep) if k]
    if Xn.shape[1] == 0:
        return pd.DataFrame(columns=["feature", "VIF"])
    vif_vals = []
    for i in range(Xn.shape[1]):
        try:
            vif = variance_inflation_factor(Xn, i)
        except Exception:
            vif = np.nan
        vif_vals.append(vif)
    return pd.DataFrame({"feature": cols, "VIF": vif_vals}).sort_values("VIF", ascending=False)

def iterative_vif_selection(X_num_df: pd.DataFrame, threshold=25.0, max_drop=10) -> Tuple[List[str], pd.DataFrame, List[str]]:
    keep_cols = list(X_num_df.columns)
    dropped = []
    for _ in range(max_drop):
        if len(keep_cols) < 2:
            break
        vif_df = compute_vif_table_numeric(X_num_df[keep_cols])
        if vif_df.empty or not np.isfinite(vif_df["VIF"]).any(): break
        top = vif_df.iloc[0]
        if (not np.isnan(top["VIF"])) and (top["VIF"] > threshold):
            drop_col = str(top["feature"])
            keep_cols.remove(drop_col)
            dropped.append(drop_col)
        else:
            break
    final_vif = compute_vif_table_numeric(X_num_df[keep_cols]) if keep_cols else pd.DataFrame(columns=["feature","VIF"])
    return keep_cols, final_vif, dropped

# ================= 学習器 =================
def logistic_with_sparse_pipeline_mi_isotonic(
    Xtr_df, ytr, Xcal_df, ycal, Xte_df, labels, base_tag: str
):
    cat_cols = [c for c in ["Department_en","Dept_detail"] if c in Xtr_df.columns]
    num_cols_all = [c for c in Xtr_df.columns if c not in cat_cols]
    if USE_VIF_LR and len(num_cols_all) > 0:
        keep_num_cols, vif_table, dropped_cols = iterative_vif_selection(
            Xtr_df[num_cols_all], threshold=VIF_THRESHOLD, max_drop=VIF_MAX_DROP
        )
    else:
        keep_num_cols = num_cols_all
        vif_table = pd.DataFrame({"feature": num_cols_all, "VIF": np.nan})
        dropped_cols = []

    vif_table.to_csv(RESULTS_DIR / f"vif_{base_tag}_final.csv", index=False, encoding="utf-8-sig")
    pd.DataFrame({"dropped": dropped_cols}).to_csv(RESULTS_DIR / f"vif_{base_tag}_dropped.csv", index=False, encoding="utf-8-sig")

    def apply_keep(df):
        cols = [c for c in keep_num_cols + cat_cols if c in df.columns]
        return df[cols].copy()
    Xtr_df_v = apply_keep(Xtr_df)
    Xcal_df_v = apply_keep(Xcal_df)
    Xte_df_v  = apply_keep(Xte_df)

    all_num_cols = [c for c in Xtr_df_v.columns if c not in cat_cols]
    trans_meta = learn_numeric_transform_params(pd.concat([Xtr_df_v[all_num_cols], Xcal_df_v[all_num_cols]], axis=0), all_num_cols)
    (RESULTS_DIR / f"transforms_{base_tag}_LR.json").write_text(json.dumps(trans_meta, ensure_ascii=False, indent=2), encoding="utf-8")

    Xtr_df_t = apply_numeric_transform(Xtr_df_v, trans_meta, all_num_cols, model_name="Logistic")
    Xcal_df_t = apply_numeric_transform(Xcal_df_v, trans_meta, all_num_cols, model_name="Logistic")
    Xte_df_t  = apply_numeric_transform(Xte_df_v , trans_meta, all_num_cols, model_name="Logistic")

    Xtr_num, Xte_num, Xtr_cat, Xte_cat, enc, num_cols, feature_names = encode_fit_transform_sparse(Xtr_df_t, Xte_df_t, cat_cols)
    Xcal_cat = enc.transform(Xcal_df_t[cat_cols]) if len(cat_cols) else csr_matrix((len(Xcal_df_t), 0), dtype=np.float32)
    Xcal_num = Xcal_df_t[num_cols].to_numpy(dtype=np.float32)

    nfeat = Xtr_num.shape[1] if Xtr_num.shape[1] > 0 else 1
    imp = IterativeImputer(
        random_state=RNG_SEED, max_iter=MI_MAXITER, sample_posterior=True,
        n_nearest_features=min(MI_NEAREST, nfeat),
        initial_strategy="median", skip_complete=True
    )
    X_imp_fit = np.vstack([Xtr_num, Xcal_num]) if (Xtr_num.size and Xcal_num.size) else Xtr_num
    imp.fit(X_imp_fit); del X_imp_fit; gc.collect()

    scaler = MaxAbsScaler()
    Xtr_num_once = imp.transform(Xtr_num)
    Xcal_num_once = imp.transform(Xcal_num)
    scaler.fit(np.vstack([Xtr_num_once, Xcal_num_once]) if Xtr_num_once.size else Xtr_num_once)
    del Xtr_num_once, Xcal_num_once; gc.collect()

    cw = class_weights_inverse_freq(ytr, labels)
    sample_weight_tr = np.array([cw[int(y)] for y in ytr], dtype=np.float32)

    P_cal_mean = None
    P_te_mean = None
    for t in range(MI_COUNT):
        Xtr_num_imp = imp.transform(Xtr_num)
        Xcal_num_imp = imp.transform(Xcal_num)
        Xte_num_imp  = imp.transform(Xte_num)

        Xtr_num_s = scaler.transform(Xtr_num_imp)
        Xcal_num_s = scaler.transform(Xcal_num_imp)
        Xte_num_s  = scaler.transform(Xte_num_imp)

        Xtr_num_csr = csr_matrix(Xtr_num_s, dtype=np.float32)
        Xcal_num_csr = csr_matrix(Xcal_num_s, dtype=np.float32)
        Xte_num_csr  = csr_matrix(Xte_num_s, dtype=np.float32)

        Xtr = hstack([Xtr_num_csr, Xtr_cat], format="csr")
        Xcal = hstack([Xcal_num_csr, Xcal_cat], format="csr")
        Xte  = hstack([Xte_num_csr, Xte_cat], format="csr")

        lr = LRClf(**LR_PARAMS)
        lr.fit(Xtr, ytr, sample_weight=sample_weight_tr)

        P_cal = lr.predict_proba(Xcal).astype(np.float32)
        P_te  = lr.predict_proba(Xte ).astype(np.float32)

        if P_cal_mean is None:
            P_cal_mean = P_cal.astype(np.float64, copy=True)
            P_te_mean  = P_te.astype(np.float64, copy=True)
        else:
            P_cal_mean += (P_cal - P_cal_mean) / (t + 1)
            P_te_mean  += (P_te  - P_te_mean)  / (t + 1)

        del Xtr_num_imp, Xcal_num_imp, Xte_num_imp, Xtr_num_s, Xcal_num_s, Xte_num_s
        del Xtr_num_csr, Xcal_num_csr, Xte_num_csr, Xtr, Xcal, Xte, lr, P_cal, P_te
        gc.collect()

    iso = fit_isotonic_ovr(P_cal_mean.astype(np.float32), ycal, labels)
    P_te_calibrated = apply_isotonic_ovr(P_te_mean.astype(np.float32), iso)

    meta = dict(
        encoder="OneHotEncoder", num_cols=list(num_cols),
        cat_cols=cat_cols, feature_names=feature_names,
        params=LR_PARAMS.copy(), cal_mode="isotonic",
        vif_threshold=VIF_THRESHOLD, vif_dropped=dropped_cols
    )
    del Xtr_num, Xte_num, Xtr_cat, Xte_cat, Xcal_cat, Xcal_num, enc, scaler, imp
    gc.collect()
    return P_te_calibrated, P_cal_mean, P_te_mean, meta

def train_xgb_with_isotonic(Xtr_df, ytr, Xcal_df, ycal, Xte_df, labels, params, fold_tag: str):
    cat_cols = [c for c in ["Department_en","Dept_detail"] if c in Xtr_df.columns]
    num_cols_all = [c for c in Xtr_df.columns if c not in cat_cols]
    trans_meta = learn_numeric_transform_params(pd.concat([Xtr_df[num_cols_all], Xcal_df[num_cols_all]], axis=0), num_cols_all)
    (RESULTS_DIR / f"transforms_{fold_tag}_XGB.json").write_text(json.dumps(trans_meta, ensure_ascii=False, indent=2), encoding="utf-8")
    Xtr_df_t = apply_numeric_transform(Xtr_df, trans_meta, num_cols_all, model_name="XGB")
    Xcal_df_t = apply_numeric_transform(Xcal_df, trans_meta, num_cols_all, model_name="XGB")
    Xte_df_t  = apply_numeric_transform(Xte_df , trans_meta, num_cols_all, model_name="XGB")

    enc = _make_onehot_encoder(dtype=np.float32).fit(Xtr_df_t[cat_cols])
    Xtr_csr = _csr_from_df_with_onehot(Xtr_df_t, enc, cat_cols)
    Xcal_csr = _csr_from_df_with_onehot(Xcal_df_t, enc, cat_cols)
    Xte_csr  = _csr_from_df_with_onehot(Xte_df_t, enc, cat_cols)

    params = params.copy(); params["num_class"] = len(labels)
    model = xgb.XGBClassifier(**params)

    cw = class_weights_inverse_freq(ytr, labels)
    sample_weight_tr = np.array([cw[int(y)] for y in ytr], dtype=np.float32)

    model.set_params(early_stopping_rounds=ES_ROUNDS)
    model.fit(Xtr_csr, ytr, sample_weight=sample_weight_tr,
              eval_set=[(Xcal_csr, ycal)], verbose=False)

    P_cal_raw = model.predict_proba(Xcal_csr).astype(np.float32)
    P_te_raw  = model.predict_proba(Xte_csr ).astype(np.float32)

    iso = fit_isotonic_ovr(P_cal_raw, ycal, labels)
    P_te = apply_isotonic_ovr(P_te_raw, iso)

    del Xtr_csr, Xcal_csr, Xte_csr, Xtr_df_t, Xcal_df_t, Xte_df_t, sample_weight_tr
    gc.collect()
    return P_te, P_cal_raw, P_te_raw, model, enc, cat_cols

def train_lgb_with_isotonic(Xtr_df, ytr, Xcal_df, ycal, Xte_df, labels, params, fold_tag: str):
    if not HAS_LGB:
        raise RuntimeError("LightGBM is not available in this environment.")
    cat_cols = [c for c in ["Department_en","Dept_detail"] if c in Xtr_df.columns]
    num_cols_all = [c for c in Xtr_df.columns if c not in cat_cols]
    trans_meta = learn_numeric_transform_params(pd.concat([Xtr_df[num_cols_all], Xcal_df[num_cols_all]], axis=0), num_cols_all)
    (RESULTS_DIR / f"transforms_{fold_tag}_LGB.json").write_text(json.dumps(trans_meta, ensure_ascii=False, indent=2), encoding="utf-8")
    Xtr_df_t = apply_numeric_transform(Xtr_df, trans_meta, num_cols_all, model_name="LGB")
    Xcal_df_t = apply_numeric_transform(Xcal_df, trans_meta, num_cols_all, model_name="LGB")
    Xte_df_t  = apply_numeric_transform(Xte_df , trans_meta, num_cols_all, model_name="LGB")

    enc = _make_onehot_encoder(dtype=np.float32).fit(Xtr_df_t[cat_cols])
    Xtr_csr = _csr_from_df_with_onehot(Xtr_df_t, enc, cat_cols)
    Xcal_csr = _csr_from_df_with_onehot(Xcal_df_t, enc, cat_cols)
    Xte_csr  = _csr_from_df_with_onehot(Xte_df_t, enc, cat_cols)

    params = params.copy(); params["num_class"] = len(labels)

    cw = class_weights_inverse_freq(ytr, labels)
    sw = np.array([cw[int(y)] for y in ytr], dtype=np.float32)

    train_set = lgb.Dataset(Xtr_csr, label=ytr, weight=sw, free_raw_data=True)
    valid_set = lgb.Dataset(Xcal_csr, label=ycal, free_raw_data=True)
    model = lgb.train(params, train_set, num_boost_round=params.get("n_estimators", 300),
                      valid_sets=[train_set, valid_set],
                      callbacks=[lgb.early_stopping(stopping_rounds=ES_ROUNDS, verbose=False),
                                 lgb.log_evaluation(period=0)])

    P_cal_raw = model.predict(Xcal_csr, num_iteration=model.best_iteration).astype(np.float32)
    P_te_raw  = model.predict(Xte_csr , num_iteration=model.best_iteration).astype(np.float32)

    iso = fit_isotonic_ovr(P_cal_raw, ycal, labels)
    P_te = apply_isotonic_ovr(P_te_raw, iso)

    del Xtr_csr, Xcal_csr, Xte_csr, Xtr_df_t, Xcal_df_t, Xte_df_t, sw, train_set, valid_set
    gc.collect()
    return P_te, P_cal_raw, P_te_raw, model, enc, cat_cols

# ================= 評価・保存 =================
def evaluate_fold(y_te, P_te, labels, B: int):
    auroc, auprc = macro_ovr_auc(y_te, P_te, labels)
    brier = brier_score_mc(y_te, P_te, labels)
    acc = accuracy_score(y_te, P_te.argmax(axis=1))
    ece = ece_confidence(y_te, P_te, 10)
    slope, intercept = compute_calibration_slope_intercept(y_te, P_te, labels)
    boot = bootstrap_metrics(y_te, P_te, labels, B=B_BOOT, seed=RNG_SEED+123)
    out = {
        "AUROC_macro": auroc, "AUPRC_macro": auprc, "Brier": brier, "Accuracy": acc, "ECE": ece,
        "CalSlope": slope, "CalIntercept": intercept,
        "CI_AUROC_macro": bootstrap_ci_stat(boot["AUROC_macro"]),
        "CI_AUPRC_macro": bootstrap_ci_stat(boot["AUPRC_macro"]),
        "CI_Brier": bootstrap_ci_stat(boot["Brier"]),
        "CI_Accuracy": bootstrap_ci_stat(boot["Accuracy"]),
        "CI_ECE": bootstrap_ci_stat(boot["ECE"])
    }
    del boot; gc.collect()
    return out

def save_reliability_tables(base_tag: str, y_true: np.ndarray, proba: np.ndarray, labels: np.ndarray, n_bins=10):
    bins = np.linspace(0.0, 1.0, n_bins+1)
    for j, cls in enumerate(labels):
        y_bin = (y_true == cls).astype(int)
        p = proba[:, j]
        rows = []
        for i in range(n_bins):
            lo, hi = bins[i], bins[i+1]
            m = (p >= lo) & (p < hi if i < n_bins - 1 else p <= hi)
            if m.any():
                rows.append(dict(bin_lo=lo, bin_hi=hi, n=int(m.sum()),
                                 mean_prob=float(p[m].mean()),
                                 frac_pos=float(y_bin[m].mean())))
        out = pd.DataFrame(rows)
        out.to_csv(RESULTS_DIR / f"reliability_{base_tag}_class{int(cls)}.csv", index=False, encoding="utf-8-sig")

def save_dca_tables(base_tag: str, y_true: np.ndarray, proba: np.ndarray, labels: np.ndarray):
    thr = np.linspace(0.01, 0.99, 99)
    for j, cls in enumerate(labels):
        y_bin = (y_true == cls).astype(int)
        p = proba[:, j]
        rows = []
        for t in thr:
            pred = (p >= t).astype(int)
            TP = int(((pred == 1) & (y_bin == 1)).sum())
            FP = int(((pred == 1) & (y_bin == 0)).sum())
            FN = int(((pred == 0) & (y_bin == 1)).sum())
            TN = int(((pred == 0) & (y_bin == 0)).sum())
            N = len(y_bin)
            nb = (TP/N) - (FP/N) * (t/(1.0 - t))
            preval = y_bin.mean()
            nb_all = preval - (1 - preval) * (t/(1.0 - t))
            nb_none = 0.0
            rows.append(dict(threshold=float(t), nb=nb, nb_all=nb_all, nb_none=nb_none,
                             TP=TP, FP=FP, FN=FN, TN=TN, N=N))
        pd.DataFrame(rows).to_csv(
            RESULTS_DIR / f"dca_{base_tag}_class{int(cls)}.csv", index=False, encoding="utf-8-sig"
        )

# ================= HPO（ES付：AUPRC after Isotonic 最大化） =================
def _finalize_study(study):
    try:
        study.trials_dataframe()
    except Exception:
        pass
    del study
    gc.collect()

def _make_study(study_name: str):
    return optuna.create_study(
        direction="maximize",
        study_name=study_name,
        sampler=TPESampler(seed=RNG_SEED),
        pruner=MedianPruner(n_warmup_steps=20)
    )

def hpo_objective_xgb(trial, X_tr_df, y_tr, X_cal, y_cal, labels, fold_tag: str):
    params = XGB_PARAMS.copy()
    params.update(dict(
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.2),
        max_depth=trial.suggest_int("max_depth", 3, 10),
        min_child_weight=trial.suggest_float("min_child_weight", 1.0, 8.0),
        subsample=trial.suggest_float("subsample", 0.6, 1.0),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.6, 1.0),
        reg_lambda=trial.suggest_float("reg_lambda", 0.0, 5.0),
        reg_alpha=trial.suggest_float("reg_alpha", 0.0, 5.0),
        gamma=trial.suggest_float("gamma", 0.0, 5.0),
        n_estimators=trial.suggest_int("n_estimators", 400, 1400),
    ))
    cat_cols = [c for c in ["Department_en","Dept_detail"] if c in X_tr_df.columns]
    num_cols_all = [c for c in X_tr_df.columns if c not in cat_cols]
    trans_meta = learn_numeric_transform_params(pd.concat([X_tr_df[num_cols_all], X_cal[num_cols_all]], axis=0), num_cols_all)
    Xtr_df_t = apply_numeric_transform(X_tr_df, trans_meta, num_cols_all, model_name="XGB")
    Xcal_df_t = apply_numeric_transform(X_cal , trans_meta, num_cols_all, model_name="XGB")
    enc = _make_onehot_encoder(dtype=np.float32).fit(Xtr_df_t[cat_cols])
    Xtr_csr = _csr_from_df_with_onehot(Xtr_df_t, enc, cat_cols)
    Xcal_csr = _csr_from_df_with_onehot(Xcal_df_t, enc, cat_cols)

    mdl = xgb.XGBClassifier(**{**params, "num_class": len(labels), "early_stopping_rounds": ES_ROUNDS})
    cw = class_weights_inverse_freq(y_tr, labels)
    sw = np.array([cw[int(y)] for y in y_tr], dtype=np.float32)
    mdl.fit(Xtr_csr, y_tr, sample_weight=sw, eval_set=[(Xcal_csr, y_cal)], verbose=False)
    P_cal_raw = mdl.predict_proba(Xcal_csr).astype(np.float32)
    iso = fit_isotonic_ovr(P_cal_raw, y_cal, labels)
    P_cal = apply_isotonic_ovr(P_cal_raw, iso)
    _, auprc = macro_ovr_auc(y_cal, P_cal, labels)
    del Xtr_csr, Xcal_csr, Xtr_df_t, Xcal_df_t, sw, mdl, P_cal_raw, P_cal, enc
    gc.collect()
    return auprc if not np.isnan(auprc) else -1e9

def hpo_objective_lgb(trial, X_tr_df, y_tr, X_cal, y_cal, labels, fold_tag: str):
    params = LGB_PARAMS.copy()
    params.update(dict(
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.2),
        num_leaves=trial.suggest_int("num_leaves", 31, 255),
        min_child_samples=trial.suggest_int("min_child_samples", 20, 300),
        feature_fraction=trial.suggest_float("feature_fraction", 0.6, 1.0),
        bagging_fraction=trial.suggest_float("bagging_fraction", 0.6, 1.0),
        lambda_l2=trial.suggest_float("lambda_l2", 0.0, 5.0),
        min_gain_to_split=trial.suggest_float("min_gain_to_split", 0.0, 1.0),
        n_estimators=trial.suggest_int("n_estimators", 400, 1400),
    ))
    cat_cols = [c for c in ["Department_en","Dept_detail"] if c in X_tr_df.columns]
    num_cols_all = [c for c in X_tr_df.columns if c not in cat_cols]
    trans_meta = learn_numeric_transform_params(pd.concat([X_tr_df[num_cols_all], X_cal[num_cols_all]], axis=0), num_cols_all)
    Xtr_df_t = apply_numeric_transform(X_tr_df, trans_meta, num_cols_all, model_name="LGB")
    Xcal_df_t = apply_numeric_transform(X_cal , trans_meta, num_cols_all, model_name="LGB")
    enc = _make_onehot_encoder(dtype=np.float32).fit(Xtr_df_t[cat_cols])
    Xtr_csr = _csr_from_df_with_onehot(Xtr_df_t, enc, cat_cols)
    Xcal_csr = _csr_from_df_with_onehot(Xcal_df_t, enc, cat_cols)
    cw = class_weights_inverse_freq(y_tr, labels)
    sw = np.array([cw[int(y)] for y in y_tr], dtype=np.float32)
    train_set = lgb.Dataset(Xtr_csr, label=y_tr, weight=sw, free_raw_data=True)
    valid_set = lgb.Dataset(Xcal_csr, label=y_cal, free_raw_data=True)
    mdl = lgb.train(params, train_set, num_boost_round=params.get("n_estimators", 300),
                    valid_sets=[train_set, valid_set],
                    callbacks=[lgb.early_stopping(stopping_rounds=ES_ROUNDS, verbose=False),
                               lgb.log_evaluation(period=0)])
    P_cal_raw = mdl.predict(Xcal_csr, num_iteration=mdl.best_iteration).astype(np.float32)
    iso = fit_isotonic_ovr(P_cal_raw, y_cal, labels)
    P_cal = apply_isotonic_ovr(P_cal_raw, iso)
    _, auprc = macro_ovr_auc(y_cal, P_cal, labels)
    del Xtr_csr, Xcal_csr, Xtr_df_t, Xcal_df_t, sw, train_set, valid_set, mdl, P_cal_raw, P_cal, enc
    gc.collect()
    return auprc if not np.isnan(auprc) else -1e9

def hpo_objective_lr(trial, X_tr_df, y_tr, X_cal, y_cal, labels, fold_tag: str):
    cat_cols = [c for c in ["Department_en","Dept_detail"] if c in X_tr_df.columns]
    num_cols_all = [c for c in X_tr_df.columns if c not in cat_cols]
    trans_meta = learn_numeric_transform_params(pd.concat([X_tr_df[num_cols_all], X_cal[num_cols_all]], axis=0), num_cols_all)
    Xtr_df_t = apply_numeric_transform(X_tr_df, trans_meta, num_cols_all, model_name="Logistic")
    Xcal_df_t = apply_numeric_transform(X_cal , trans_meta, num_cols_all, model_name="Logistic")

    enc = _make_onehot_encoder(dtype=np.float32).fit(Xtr_df_t[cat_cols])

    def to_csr(df_is_train=True):
        df = Xtr_df_t if df_is_train else Xcal_df_t
        X_cat = enc.transform(df[cat_cols]) if len(cat_cols) else csr_matrix((len(df), 0), dtype=np.float32)
        num_cols = [c for c in df.columns if c not in cat_cols]
        simp = SimpleImputer(strategy="median", copy=True).fit(pd.concat([Xtr_df_t[num_cols], Xcal_df_t[num_cols]], axis=0))
        Xn_tr = simp.transform(Xtr_df_t[num_cols]); Xn_cal = simp.transform(Xcal_df_t[num_cols])
        scaler = MaxAbsScaler().fit(np.vstack([Xn_tr, Xn_cal]))
        Xn = scaler.transform(Xn_tr if df_is_train else Xn_cal)
        return hstack([csr_matrix(Xn), X_cat], format="csr")

    Xtr_csr = to_csr(True); Xcal_csr = to_csr(False)
    C = trial.suggest_float("C", 1e-3, 1e2, log=True)
    lr = LRClf(multi_class="multinomial", solver="saga", penalty="l2",
               C=C, max_iter=max(LR_PARAMS["max_iter"], 3000), n_jobs=1, tol=1e-4, random_state=RNG_SEED)
    cw = class_weights_inverse_freq(y_tr, labels)
    sw = np.array([cw[int(y)] for y in y_tr], dtype=np.float32)
    lr.fit(Xtr_csr, y_tr, sample_weight=sw)
    P_cal_raw = lr.predict_proba(Xcal_csr).astype(np.float32)
    iso = fit_isotonic_ovr(P_cal_raw, y_cal, labels)
    P_cal = apply_isotonic_ovr(P_cal_raw, iso)
    _, auprc = macro_ovr_auc(y_cal, P_cal, labels)
    del Xtr_csr, Xcal_csr, Xtr_df_t, Xcal_df_t, sw, lr, P_cal_raw, P_cal, enc
    gc.collect()
    return auprc if not np.isnan(auprc) else -1e9

def run_hpo_per_fold(X_tr2, y_tr2, X_cal, y_cal, labels, fold_tag: str) -> Dict[str, dict]:
    final_params = {"Logistic": LR_PARAMS.copy(), "XGBoost": XGB_PARAMS.copy()}
    if HAS_LGB: final_params["LightGBM"] = LGB_PARAMS.copy()
    if not HAS_OPTUNA:
        log(f"[HPO][{fold_tag}] Optuna未導入 → 既定値を使用")
        return final_params
    try:
        study = _make_study(f"LR_AUPRC_iso_{fold_tag}")
        study.optimize(lambda t: hpo_objective_lr(t, X_tr2, y_tr2, X_cal, y_cal, labels, fold_tag),
                       n_trials=N_TRIALS_LR, show_progress_bar=False)
        final_params["Logistic"]["C"] = float(study.best_params["C"])
        log(f"[HPO][{fold_tag}][LR] best: {study.best_params}")
        _finalize_study(study)
    except Exception as e:
        log(f"[HPO][{fold_tag}][LR] skipped: {repr(e)}")
    try:
        study = _make_study(f"XGB_AUPRC_iso_{fold_tag}")
        study.optimize(lambda t: hpo_objective_xgb(t, X_tr2, y_tr2, X_cal, y_cal, labels, fold_tag),
                       n_trials=N_TRIALS_XGB, show_progress_bar=False)
        for k, v in study.best_params.items(): final_params["XGBoost"][k] = v
        log(f"[HPO][{fold_tag}][XGB] best: {study.best_params}")
        _finalize_study(study)
    except Exception as e:
        log(f"[HPO][{fold_tag}][XGB] skipped: {repr(e)}")
    if HAS_LGB:
        try:
            study = _make_study(f"LGB_AUPRC_iso_{fold_tag}")
            study.optimize(lambda t: hpo_objective_lgb(t, X_tr2, y_tr2, X_cal, y_cal, labels, fold_tag),
                           n_trials=N_TRIALS_LGB, show_progress_bar=False)
            for k, v in study.best_params.items(): final_params["LightGBM"][k] = v
            log(f"[HPO][{fold_tag}][LGB] best: {study.best_params}")
            _finalize_study(study)
        except Exception as e:
            log(f"[HPO][{fold_tag}][LGB] skipped: {repr(e)}")
    gc.collect()
    return final_params

# ================== 重複列対策＋SHAP用エクスポート ==================
def _dedupe_columns(df: pd.DataFrame, make_copy: bool = True) -> pd.DataFrame:
    if make_copy:
        df = df.copy()
    seen = {}
    new_cols: List[str] = []
    for c in df.columns.map(str):
        if c not in seen:
            seen[c] = 0
            new_cols.append(c)
        else:
            seen[c] += 1
            new_cols.append(f"{c}_dup{seen[c]}")
    df.columns = new_cols
    return df

def export_analysis_features_for_shap(df: pd.DataFrame, results_dir: Path, cutoff: pd.Timestamp | None = None, overwrite: bool = True) -> Path:
    results_dir.mkdir(parents=True, exist_ok=True)
    out_parquet = results_dir / "analysis_features.parquet"
    out_csv     = results_dir / "analysis_features.csv"

    if (not overwrite) and out_parquet.exists():
        print(f"[export] skip (exists): {out_parquet}")
        return out_parquet

    frames = []
    for lm in ["LM0", "LM3"]:
        X, y, groups, used_cols, index_ids, dates, date_col = build_feature_matrix(df, lm)
        if date_col == "":
            raise KeyError("Base_Date/BaseDate/Date が見つからず Temporal split に使用できません")

        work = pd.DataFrame({
            "INDEX": index_ids,
            "Facility": groups.astype(str),
            "Landmark": lm,
            "y_true": y
        })
        work["Base_Date"] = pd.to_datetime(dates, errors="coerce")

        if "Department_en" in X.columns:
            work["Department_en"] = X["Department_en"].astype(str).values
        if "Dept_detail" in X.columns:
            work["Dept_detail"] = X["Dept_detail"].astype(str).values

        feat_cols = list(X.columns)
        overlap = set(work.columns) & set(feat_cols)
        feat_cols_no_overlap = [c for c in feat_cols if c not in overlap]

        X_sel = X[feat_cols_no_overlap].copy()
        for c in X_sel.columns:
            if X_sel[c].dtype.kind in "ifu":
                X_sel[c] = pd.to_numeric(X_sel[c], errors="coerce").astype("float32")

        temp = pd.concat([work, X_sel], axis=1)
        if temp.columns.duplicated().any():
            temp = _dedupe_columns(temp, make_copy=False)
        frames.append(temp)

    out = pd.concat(frames, axis=0, ignore_index=True)

    for c in out.columns:
        if c in ["INDEX", "y_true"]:
            out[c] = pd.to_numeric(out[c], errors="coerce").astype("Int64")
        elif c in ["Base_Date"]:
            out[c] = pd.to_datetime(out[c], errors="coerce")
        elif out[c].dtype.kind in "ifu":
            out[c] = pd.to_numeric(out[c], errors="coerce").astype("float32")

    if out.columns.duplicated().any():
        out = _dedupe_columns(out, make_copy=False)

    out.to_parquet(out_parquet, index=False)
    out.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"[export] analysis features saved:\n - {out_parquet}\n - {out_csv}")
    return out_parquet

# ================= 分割（Temporal／nLOGO） =================
def split_temporal_with_transforms(df: pd.DataFrame, landmark: str, cutoff: pd.Timestamp):
    X_df, y, groups, used_cols, index_ids, dates, date_col = build_feature_matrix(df, landmark)
    if dates is None:
        raise KeyError("Temporal split用の日付列が見つかりません（Base_Date/BaseDate/Date）")
    labels = np.unique(y)
    dates_all = pd.to_datetime(dates)
    tr_mask = (dates_all < cutoff)
    te_mask = (dates_all >= cutoff)
    if te_mask.sum() < MIN_TEST_SAMPLES:
        log(f"[temporal] warning: test n={te_mask.sum()} < {MIN_TEST_SAMPLES}")

    cat_cols = [c for c in ["Department_en","Dept_detail"] if c in X_df.columns]
    num_cols = [c for c in X_df.columns if c not in cat_cols]

    X_tr_df_raw = X_df.loc[tr_mask].copy()
    X_te_df_raw = X_df.loc[te_mask].copy()

    trans_meta = learn_numeric_transform_params(X_tr_df_raw[num_cols], num_cols)
    (RESULTS_DIR / f"transforms_{landmark}_temporal_BASE.json").write_text(
        json.dumps(trans_meta, ensure_ascii=False, indent=2), encoding="utf-8"
    )

    return (X_tr_df_raw, y[tr_mask], X_te_df_raw, y[te_mask],
            index_ids[te_mask], groups[te_mask], dates_all[te_mask], labels, trans_meta)

def split_logo_with_transforms(df: pd.DataFrame, landmark: str, facility: str):
    X_df, y, groups, used_cols, index_ids, dates, date_col = build_feature_matrix(df, landmark)
    labels = np.unique(y)
    te_mask = (groups == facility)
    tr_mask = ~te_mask
    if te_mask.sum() < MIN_TEST_SAMPLES:
        return None
    cat_cols = [c for c in ["Department_en","Dept_detail"] if c in X_df.columns]
    num_cols = [c for c in X_df.columns if c not in cat_cols]
    X_tr_df_raw = X_df.loc[tr_mask].copy()
    X_te_df_raw = X_df.loc[te_mask].copy()

    trans_meta = learn_numeric_transform_params(X_tr_df_raw[num_cols], num_cols)
    (RESULTS_DIR / f"transforms_{landmark}_logo_{facility}_BASE.json").write_text(
        json.dumps(trans_meta, ensure_ascii=False, indent=2), encoding="utf-8"
    )

    return (X_tr_df_raw, y[tr_mask], X_te_df_raw, y[te_mask],
            index_ids[te_mask], groups[te_mask], labels, trans_meta)

# ================= 実行（Temporal／nLOGO） =================
def run_temporal_main(df: pd.DataFrame, landmark: str, cutoff: pd.Timestamp):
    log(f"[Temporal MAIN] landmark={landmark} cutoff={cutoff.date()} (Iso + LR=VIF&winsor / Tree=logのみ)")
    (X_tr_df, y_tr, X_te_df, y_te, idx_te, fac_te, base_dates_te, labels, trans_meta_base) = \
        split_temporal_with_transforms(df, landmark, cutoff)

    dept_te = X_te_df["Department_en"].astype(str).fillna("Other").reset_index(drop=True)
    fac_te_series = pd.Series(fac_te).astype(str).reset_index(drop=True)
    idx_te_series = pd.Series(idx_te).reset_index(drop=True)
    base_dates_str = pd.Series(base_dates_te).reset_index(drop=True)

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=CAL_FRAC, random_state=RNG_SEED)
    tr_idx, cal_idx = next(splitter.split(X_tr_df, y_tr))
    X_tr2 = X_tr_df.iloc[tr_idx].copy(); y_tr2 = y_tr[tr_idx]
    X_cal = X_tr_df.iloc[cal_idx].copy(); y_cal = y_tr[cal_idx]

    final_params = run_hpo_per_fold(X_tr2, y_tr2, X_cal, y_cal, labels, fold_tag=f"{landmark}_Temporal")
    (RESULTS_DIR / f"best_params_temporal_{landmark}.json").write_text(
        json.dumps(final_params, ensure_ascii=False, indent=2), encoding="utf-8"
    )

    results_rows = []

    # Logistic
    try:
        P_lr, Pcal_lr_raw, _, meta_lr = logistic_with_sparse_pipeline_mi_isotonic(
            X_tr2, y_tr2, X_cal, y_cal, X_te_df, labels, base_tag=f"temporal_{landmark}_Logistic"
        )
        row = evaluate_fold(y_te, P_lr, labels, B=B_BOOT); row.update(dict(Model="Logistic", Facility="Temporal", Landmark=landmark, CalMode="isotonic", Note=f"VIF={'on' if USE_VIF_LR else 'off'}"))
        results_rows.append(row)
        save_reliability_tables(f"temporal_{landmark}_Logistic", y_te, P_lr, labels, n_bins=10)
        save_dca_tables(f"temporal_{landmark}_Logistic", y_te, P_lr, labels)
        yhat = P_lr.argmax(axis=1)
        recs = []
        for i in range(len(idx_te_series)):
            rec = {"INDEX": int(idx_te_series.iloc[i]), "Facility": fac_te_series.iloc[i],
                   "Department": dept_te.iloc[i], "Landmark": landmark, "Model": "Logistic",
                   "y_true": int(y_te[i]), "y_pred": int(yhat[i]),
                   "Base_Date": (pd.to_datetime(base_dates_str.iloc[i]).strftime("%Y-%m-%d")
                                 if pd.notnull(base_dates_str.iloc[i]) else "")}
            for j, cls in enumerate(labels): rec[f"p_{int(cls)}"] = float(P_lr[i, j])
            recs.append(rec)
        pd.DataFrame(recs).to_csv(RESULTS_DIR / f"oof_temporal_{landmark}_Logistic_iso.csv", index=False, encoding="utf-8-sig")
        (MODELS_DIR / landmark).mkdir(parents=True, exist_ok=True)
        (MODELS_DIR / landmark / f"Temporal_Logistic_feature_meta_iso.json").write_text(
            json.dumps(meta_lr, ensure_ascii=False, indent=2), encoding="utf-8"
        )
        del Pcal_lr_raw, yhat, recs, meta_lr
        gc.collect()
    except Exception as e:
        log(f"[temporal][{landmark}] Logistic failed: {repr(e)}"); traceback.print_exc()

    # XGB
    try:
        P_xgb, Pcal_xgb_raw, _, mdl_xgb, enc_xgb, cat_xgb = train_xgb_with_isotonic(
            X_tr2, y_tr2, X_cal, y_cal, X_te_df, labels, params=final_params.get("XGBoost", XGB_PARAMS),
            fold_tag=f"temporal_{landmark}"
        )
        row = evaluate_fold(y_te, P_xgb, labels, B=B_BOOT); row.update(dict(Model="XGBoost", Facility="Temporal", Landmark=landmark, CalMode="isotonic"))
        results_rows.append(row)
        save_reliability_tables(f"temporal_{landmark}_XGB", y_te, P_xgb, labels, n_bins=10)
        save_dca_tables(f"temporal_{landmark}_XGB", y_te, P_xgb, labels)
        yhat = P_xgb.argmax(axis=1); recs=[]
        for i in range(len(idx_te_series)):
            rec = {"INDEX": int(idx_te_series.iloc[i]), "Facility": fac_te_series.iloc[i],
                   "Department": dept_te.iloc[i], "Landmark": landmark, "Model": "XGBoost",
                   "y_true": int(y_te[i]), "y_pred": int(yhat[i]),
                   "Base_Date": (pd.to_datetime(base_dates_str.iloc[i]).strftime("%Y-%m-%d")
                                 if pd.notnull(base_dates_str.iloc[i]) else "")}
            for j, cls in enumerate(labels): rec[f"p_{int(cls)}"] = float(P_xgb[i, j])
            recs.append(rec)
        pd.DataFrame(recs).to_csv(RESULTS_DIR / f"oof_temporal_{landmark}_XGBoost_iso.csv", index=False, encoding="utf-8-sig")
        del Pcal_xgb_raw, mdl_xgb, enc_xgb, cat_xgb, yhat, recs
        gc.collect()
    except Exception as e:
        log(f"[temporal][{landmark}] XGBoost failed: {repr(e)}"); traceback.print_exc()

    # LGB
    if HAS_LGB:
        try:
            P_lgb, Pcal_lgb_raw, _, mdl_lgb, enc_lgb, cat_lgb = train_lgb_with_isotonic(
                X_tr2, y_tr2, X_cal, y_cal, X_te_df, labels, params=final_params.get("LightGBM", LGB_PARAMS),
                fold_tag=f"temporal_{landmark}"
            )
            row = evaluate_fold(y_te, P_lgb, labels, B=B_BOOT); row.update(dict(Model="LightGBM", Facility="Temporal", Landmark=landmark, CalMode="isotonic"))
            results_rows.append(row)
            save_reliability_tables(f"temporal_{landmark}_LGB", y_te, P_lgb, labels, n_bins=10)
            save_dca_tables(f"temporal_{landmark}_LGB", y_te, P_lgb, labels)
            yhat = P_lgb.argmax(axis=1); recs=[]
            for i in range(len(idx_te_series)):
                rec = {"INDEX": int(idx_te_series.iloc[i]), "Facility": fac_te_series.iloc[i],
                       "Department": dept_te.iloc[i], "Landmark": landmark, "Model": "LightGBM",
                       "y_true": int(y_te[i]), "y_pred": int(yhat[i]),
                       "Base_Date": (pd.to_datetime(base_dates_str.iloc[i]).strftime("%Y-%m-%d")
                                     if pd.notnull(base_dates_str.iloc[i]) else "")}
                for j, cls in enumerate(labels): rec[f"p_{int(cls)}"] = float(P_lgb[i, j])
                recs.append(rec)
            pd.DataFrame(recs).to_csv(RESULTS_DIR / f"oof_temporal_{landmark}_LightGBM_iso.csv", index=False, encoding="utf-8-sig")
            del Pcal_lgb_raw, mdl_lgb, enc_lgb, cat_lgb, yhat, recs
            gc.collect()
        except Exception as e:
            log(f"[temporal][{landmark}] LightGBM failed: {repr(e)}"); traceback.print_exc()

    res_df = pd.DataFrame(results_rows)
    out_path = RESULTS_DIR / f"results_temporal_metrics_{landmark}.csv"
    res_df.to_csv(out_path, index=False, encoding="utf-8-sig"); log(f"[save] {out_path}")
    del X_tr_df, y_tr, X_te_df, y_te, idx_te, fac_te, base_dates_te, X_tr2, y_tr2, X_cal, y_cal
    gc.collect()
    return res_df

def run_facilitywise_logo(df: pd.DataFrame, landmark: str):
    log(f"[nLOGO] landmark={landmark} (Iso + LR=VIF&winsor / Tree=logのみ)")
    X_base, y_base, groups, _, _, _, _ = build_feature_matrix(df, landmark)
    facilities = np.unique(groups)
    all_rows = []
    for fac in facilities:
        pack = split_logo_with_transforms(df, landmark, facility=fac)
        if pack is None:
            log(f"[nLOGO] skip {fac} (test n < {MIN_TEST_SAMPLES})"); continue
        (X_tr_df, y_tr, X_te_df, y_te, idx_te, fac_te, labels, trans_meta) = pack

        dept_te = X_te_df["Department_en"].astype(str).fillna("Other").reset_index(drop=True)
        idx_te_series = pd.Series(idx_te).reset_index(drop=True)

        splitter = StratifiedShuffleSplit(n_splits=1, test_size=CAL_FRAC, random_state=RNG_SEED)
        tr_idx, cal_idx = next(splitter.split(X_tr_df, y_tr))
        X_tr2 = X_tr_df.iloc[tr_idx].copy(); y_tr2 = y_tr[tr_idx]
        X_cal = X_tr_df.iloc[cal_idx].copy(); y_cal = y_tr[cal_idx]

        final_params = run_hpo_per_fold(X_tr2, y_tr2, X_cal, y_cal, labels, fold_tag=f"{landmark}_LOGO_{fac}")
        (RESULTS_DIR / f"best_params_logo_{landmark}_{fac}.json").write_text(
            json.dumps(final_params, ensure_ascii=False, indent=2), encoding="utf-8"
        )

        for model_name in ["Logistic", "XGBoost"] + (["LightGBM"] if HAS_LGB else []):
            try:
                if model_name == "Logistic":
                    P_te, _, _, meta_lr = logistic_with_sparse_pipeline_mi_isotonic(
                        X_tr2, y_tr2, X_cal, y_cal, X_te_df, labels, base_tag=f"logo_{landmark}_{fac}_Logistic"
                    )
                elif model_name == "XGBoost":
                    P_te, _, _, mdl, enc, cat = train_xgb_with_isotonic(
                        X_tr2, y_tr2, X_cal, y_cal, X_te_df, labels, final_params.get("XGBoost", XGB_PARAMS),
                        fold_tag=f"logo_{landmark}_{fac}"
                    )
                else:
                    P_te, _, _, mdl, enc, cat = train_lgb_with_isotonic(
                        X_tr2, y_tr2, X_cal, y_cal, X_te_df, labels, final_params.get("LightGBM", LGB_PARAMS),
                        fold_tag=f"logo_{landmark}_{fac}"
                    )

                row = evaluate_fold(y_te, P_te, labels, B=B_BOOT)
                row.update(dict(Model=model_name, Facility=str(fac), Landmark=landmark, CalMode="isotonic",
                                Note=("VIF on" if (model_name=="Logistic" and USE_VIF_LR) else "")))
                all_rows.append(row)

                yhat = P_te.argmax(axis=1); recs=[]
                for i in range(len(idx_te_series)):
                    rec = {"INDEX": int(idx_te_series.iloc[i]), "Facility": str(fac), "Department": dept_te.iloc[i],
                           "Landmark": landmark, "Model": model_name, "y_true": int(y_te[i]), "y_pred": int(yhat[i])}
                    for j, cls in enumerate(labels): rec[f"p_{int(cls)}"] = float(P_te[i, j])
                    recs.append(rec)
                pd.DataFrame(recs).to_csv(RESULTS_DIR / f"oof_logo_{landmark}_{model_name}_{fac}_iso.csv", index=False, encoding="utf-8-sig")

                del yhat, recs
                gc.collect()
            except Exception as e:
                log(f"[nLOGO][{fac}] {model_name} failed: {repr(e)}"); traceback.print_exc()

        del X_tr_df, y_tr, X_te_df, y_te, idx_te, fac_te, trans_meta, X_tr2, y_tr2, X_cal, y_cal
        gc.collect()

    res_df = pd.DataFrame(all_rows)
    out_path = RESULTS_DIR / f"results_facilitywise_metrics_{landmark}.csv"
    res_df.to_csv(out_path, index=False, encoding="utf-8-sig"); log(f"[save] {out_path}")
    return res_df

# ================= main =================
def main():
    log(f"[config] RUN_MODE={RUN_MODE} | MI_COUNT={MI_COUNT} | Trials(LR/XGB/LGB)={N_TRIALS_LR}/{N_TRIALS_XGB}/{N_TRIALS_LGB} | ES={ES_ROUNDS} | B_BOOT={B_BOOT}")
    log("[run_all] data loading...")
    df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME)
    log("[run_all] data loaded.")

    # 列名の重複に対する安全化
    df = df.loc[:, ~df.columns.duplicated()].copy()
    df = _dedupe_columns(df, make_copy=False)

    log("[Temporal MAIN] LM0 start")
    t_lm0 = run_temporal_main(df, landmark="LM0", cutoff=TEMPORAL_CUTOFF)

    log("[Temporal MAIN] LM3 start")
    t_lm3 = run_temporal_main(df, landmark="LM3", cutoff=TEMPORAL_CUTOFF)

    log("[nLOGO] LM0 start")
    logo_lm0 = run_facilitywise_logo(df, landmark="LM0")

    log("[nLOGO] LM3 start")
    logo_lm3 = run_facilitywise_logo(df, landmark="LM3")

    export_analysis_features_for_shap(df, RESULTS_DIR, overwrite=True)

    show_cols = ["Model","Landmark","Facility","CalMode","AUROC_macro","AUPRC_macro","Brier","Accuracy","ECE","CalSlope","CalIntercept","Note"]
    log("===== TEMPORAL (LM0) ====="); 
    try: print(t_lm0[show_cols].to_string(index=False))
    except Exception: print(t_lm0.head())
    log("===== TEMPORAL (LM3) =====");
    try: print(t_lm3[show_cols].to_string(index=False))
    except Exception: print(t_lm3.head())
    log("===== nLOGO (LM0) =====");
    try: print(logo_lm0[show_cols].to_string(index=False))
    except Exception: print(logo_lm0.head())
    log("===== nLOGO (LM3) =====");
    try: print(logo_lm3[show_cols].to_string(index=False))
    except Exception: print(logo_lm3.head())

    log(f"[done] Final(×3) : HPO拡張({N_TRIALS_LR}/{N_TRIALS_XGB}/{N_TRIALS_LGB})・樹本数↑・ES={ES_ROUNDS}・B_BOOT={B_BOOT}（MI据え置き）")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        traceback.print_exc()
        log(f"[FATAL] {repr(e)}")


[config] RUN_MODE=Final | MI_COUNT=20 | Trials(LR/XGB/LGB)=90/200/200 | ES=200 | B_BOOT=1500
[run_all] data loading...


c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[run_all] data loaded.
[Temporal MAIN] LM0 start
[Temporal MAIN] landmark=LM0 cutoff=2024-01-01 (Iso + LR=VIF&winsor / Tree=logのみ)
[preprocess] exclude-from-features: INDEX
[preprocess] exclude-from-features: 施設名
[preprocess] exclude-from-features: 人工呼吸_連続日数
[preprocess] exclude-from-features: Base_Date
[preprocess] exclude-from-features: Cox_Event
[preprocess] exclude-from-features: All_Day
[preprocess] exclude-from-features: IntubationDate


[I 2025-11-28 13:19:42,066] A new study created in memory with name: LR_AUPRC_iso_LM0_Temporal
[I 2025-11-28 13:19:42,241] Trial 0 finished with value: 0.4873598984478316 and parameters: {'C': 0.0745934328572655}. Best is trial 0 with value: 0.4873598984478316.
[I 2025-11-28 13:19:42,741] Trial 1 finished with value: 0.4999559472302466 and parameters: {'C': 56.69849511478853}. Best is trial 1 with value: 0.4999559472302466.
[I 2025-11-28 13:19:43,068] Trial 2 finished with value: 0.5036909925518279 and parameters: {'C': 4.5705630998014515}. Best is trial 2 with value: 0.5036909925518279.
[I 2025-11-28 13:19:43,268] Trial 3 finished with value: 0.49760110510037975 and parameters: {'C': 0.9846738873614566}. Best is trial 2 with value: 0.5036909925518279.
[I 2025-11-28 13:19:43,491] Trial 4 finished with value: 0.4738041058074525 and parameters: {'C': 0.006026889128682512}. Best is trial 2 with value: 0.5036909925518279.
[I 2025-11-28 13:19:43,707] Trial 5 finished with value: 0.473804105

[HPO][LM0_Temporal][LR] best: {'C': 4.5705630998014515}


...

[HPO][LM3_LOGO_永山][LR] best: {'C': 56.69849511478853}


[I 2025-11-28 16:12:46,974] A new study created in memory with name: XGB_AUPRC_iso_LM3_LOGO_永山
[I 2025-11-28 16:12:51,304] Trial 0 finished with value: 0.6045722091762874 and parameters: {'learning_rate': 0.08116262258099886, 'max_depth': 10, 'min_child_weight': 6.123957592679836, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'reg_lambda': 0.7799726016810132, 'reg_alpha': 0.2904180608409973, 'gamma': 4.330880728874676, 'n_estimators': 1001}. Best is trial 0 with value: 0.6045722091762874.
[I 2025-11-28 16:12:55,360] Trial 1 finished with value: 0.6093721649339954 and parameters: {'learning_rate': 0.14453378978124864, 'max_depth': 3, 'min_child_weight': 7.78936896513396, 'subsample': 0.9329770563201687, 'colsample_bytree': 0.6849356442713105, 'reg_lambda': 0.9091248360355031, 'reg_alpha': 0.9170225492671691, 'gamma': 1.5212112147976886, 'n_estimators': 925}. Best is trial 1 with value: 0.6093721649339954.
[I 2025-11-28 16:12:58,067] Trial 2 finished with value

[HPO][LM3_LOGO_永山][XGB] best: {'learning_rate': 0.05684768207384378, 'max_depth': 8, 'min_child_weight': 7.210759298952831, 'subsample': 0.6495961542382096, 'colsample_bytree': 0.6637513304002004, 'reg_lambda': 2.5309345206667793, 'reg_alpha': 0.09974607218853278, 'gamma': 1.4461017409309234, 'n_estimators': 1112}


[I 2025-11-28 16:27:25,909] A new study created in memory with name: LGB_AUPRC_iso_LM3_LOGO_永山
[W 2025-11-28 16:27:25,990] Trial 0 failed with parameters: {'learning_rate': 0.08116262258099886, 'num_leaves': 244, 'min_child_samples': 225, 'feature_fraction': 0.8394633936788146, 'bagging_fraction': 0.6624074561769746, 'lambda_l2': 0.7799726016810132, 'min_gain_to_split': 0.05808361216819946, 'n_estimators': 1267} because of the following error: LightGBMError('Number of classes should be specified and greater than 1 for multiclass training').
Traceback (most recent call last):
  File "c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\tears\AppData\Local\Temp\ipykernel_29416\1189802858.py", line 975, in <lambda>
    study.optimize(lambda t: hpo_objective_lgb(t, X_tr2, y_tr2, X_cal, y_cal, labels, fold_tag),
                             ^^^^^^^^^^^^^^

[HPO][LM3_LOGO_永山][LGB] skipped: LightGBMError('Number of classes should be specified and greater than 1 for multiclass training')
[save] results_temporal_main\results_facilitywise_metrics_LM3.csv
[preprocess] exclude-from-features: INDEX
[preprocess] exclude-from-features: 施設名
[preprocess] exclude-from-features: 人工呼吸_連続日数
[preprocess] exclude-from-features: Base_Date
[preprocess] exclude-from-features: Cox_Event
[preprocess] exclude-from-features: All_Day
[preprocess] exclude-from-features: IntubationDate
[preprocess] exclude-from-features: INDEX
[preprocess] exclude-from-features: 施設名
[preprocess] exclude-from-features: 人工呼吸_連続日数
[preprocess] exclude-from-features: Base_Date
[preprocess] exclude-from-features: Cox_Event
[preprocess] exclude-from-features: All_Day
[preprocess] exclude-from-features: IntubationDate
[export] analysis features saved:
 - results_temporal_main\analysis_features.parquet
 - results_temporal_main\analysis_features.csv
===== TEMPORAL (LM0) =====
   Model Landm

ハイパーパラメーター

In [1]:
# -*- coding: utf-8 -*-
# ============================================================
# Supplementary Table S1: Hyperparameter Summary
# - 目的: 探索範囲と最良値を Excel にまとめる
# - 入力: best_params_*.json
# - 出力: Supplementary_Table_S1_Hyperparams.xlsx
# ============================================================

from __future__ import annotations
import json, re
from pathlib import Path
from typing import Dict, List
import pandas as pd
from openpyxl import Workbook

# ==== パス設定 =========================================================
RESULTS_ROOT = Path("./results_temporal_main")
OUT_XLSX = RESULTS_ROOT / "Supplementary_Table_S1_Hyperparams.xlsx"

# ==== 探索範囲定義 =====================================================
SEARCH_SPACES: Dict[str, Dict[str, Dict[str, object]]] = {
    "Logistic": {
        "C": {"type": "logfloat", "range": [1e-3, 1e2], "note": "multinomial, saga, L2"},
    },
    "XGBoost": {
        "learning_rate":   {"type": "float",   "range": [0.01, 0.20]},
        "max_depth":       {"type": "int",     "range": [3, 10]},
        "min_child_weight":{"type": "float",   "range": [1.0, 8.0]},
        "subsample":       {"type": "float",   "range": [0.60, 1.00]},
        "colsample_bytree":{"type": "float",   "range": [0.60, 1.00]},
        "reg_lambda":      {"type": "float",   "range": [0.00, 5.00]},
        "reg_alpha":       {"type": "float",   "range": [0.00, 5.00]},
        "gamma":           {"type": "float",   "range": [0.00, 5.00]},
        "n_estimators":    {"type": "int",     "range": [400, 1400]},
    },
    "LightGBM": {
        "learning_rate":     {"type": "float", "range": [0.01, 0.20]},
        "num_leaves":        {"type": "int",   "range": [31, 255]},
        "min_child_samples": {"type": "int",   "range": [20, 300]},
        "feature_fraction":  {"type": "float", "range": [0.60, 1.00]},
        "bagging_fraction":  {"type": "float", "range": [0.60, 1.00]},
        "lambda_l2":         {"type": "float", "range": [0.00, 5.00]},
        "min_gain_to_split": {"type": "float", "range": [0.00, 1.00]},
        "n_estimators":      {"type": "int",   "range": [400, 1400]},
    },
}

# ==== 補助関数 =========================================================
def fmt_space(space: Dict[str, object]) -> str:
    """型と範囲を文字列化"""
    t = str(space.get("type", ""))
    lo, hi = space.get("range", [None, None])
    if lo is None or hi is None:
        return t
    return f"{t} [{lo}, {hi}]"

def infer_study_name(path: Path) -> str:
    """ファイル名からスタディ名を推定"""
    name = path.stem
    if "temporal" in name:
        m = re.search(r"temporal_(LM\d)", name)
        return f"Temporal-{m.group(1)}" if m else "Temporal"
    if "logo" in name:
        m = re.search(r"logo_(LM\d)_(.+)", name)
        if m:
            return f"LOGO-{m.group(1)}({m.group(2)})"
        return "LOGO"
    return name

def collect_best_params(files: List[Path]) -> pd.DataFrame:
    rows = []
    for f in files:
        try:
            d = json.loads(f.read_text(encoding="utf-8"))
        except Exception:
            continue
        study = infer_study_name(f)
        for model, params in d.items():
            if not isinstance(params, dict):
                continue
            space = SEARCH_SPACES.get(model, {})
            for p_name, p_space in space.items():
                selected = params.get(p_name, None)
                rows.append({
                    "Study": study,
                    "Model": model,
                    "Parameter": p_name,
                    "Search Space": fmt_space(p_space),
                    "Selected Value": selected,
                })
    return pd.DataFrame(rows)

# ==== メイン処理 =======================================================
def main():
    files = sorted(RESULTS_ROOT.glob("best_params_*.json"))
    if not files:
        raise FileNotFoundError(f"best_params_*.json が見つかりません: {RESULTS_ROOT}")
    df = collect_best_params(files)

    # 並べ替え
    order = pd.CategoricalDtype(["Logistic", "XGBoost", "LightGBM"], ordered=True)
    df["Model"] = df["Model"].astype(order)
    df = df.sort_values(["Study", "Model", "Parameter"]).reset_index(drop=True)

    # ---- Excel書き出し ----
    with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as writer:
        for study, g in df.groupby("Study"):
            g.to_excel(writer, index=False, sheet_name=study[:31])
        # Summaryシート
        summary = pd.DataFrame({
            "Total Files": [len(files)],
            "Total Records": [len(df)],
        })
        summary.to_excel(writer, index=False, sheet_name="Summary")

    print(f"[OK] Exported Excel: {OUT_XLSX}")

if __name__ == "__main__":
    main()


[OK] Exported Excel: results_temporal_main\Supplementary_Table_S1_Hyperparams.xlsx


図表は下記想定です。

🩺 主本文（Main Figures & Tables）

Figure 1. 研究デザインとデータソースの概要

Figure 2. 各モデルの予測性能比較（ROC, Calibration）

(A) LM0 (Day 0) AUPRC curves (macro-average)

(B) LM3 (Day 3) AUPRC curves (macro-average)

(C) LM0 calibration plot (10-bin reliability curve, calibration slope/intercept, and ECE)

(D) LM3 calibration plot (10-bin reliability curve, calibration slope/intercept, and ECE)

Figure 3. 重要特徴量とSHAP値によるモデルの説明可能性（XGB）

(A) LM0_Tracheostomy

(B) LM0_Death

(C) LM3_Tracheostomy

(D) LM3_Death

Table 1. 対象患者の背景特性（施設別）

Table 2. 各モデルの性能指標（AUROC, PRAUC, Brier, Calibration）

Table 3. 施設間各モデルの性能指標（AUROC, PRAUC, Brier, Calibration）


📘 補足資料（Supplementary Figures & Tables）

Figure S1. AUROC曲線（全モデル）

Figure S2. SHAP値によるモデルの説明可能性（LR）

(A) LM0_Tracheostomy

(B) LM0_Death

(C) LM3_Tracheostomy

(D) LM3_Death

Figure S3. SHAP値によるモデルの説明可能性（LGBM）

(A) LM0_Tracheostomy

(B) LM0_Death

(C) LM3_Tracheostomy

(D) LM3_Death


Table S1. 欠損パターンおよび多重代入の診断結果

Table S2. ハイパーパラメータの探索範囲と最終値（補足詳細）

Table S3. 時間的検証（感度分析）

Table S4. 診療科別性能指標（感度分析）


図表コード

In [ ]:
# -*- coding: utf-8 -*-
# figures_and_tables_nonshap_full.py
# （安定化版・指標再編：Main=AUROC/AUPRC/Brier、Supp=Accuracy/ECE/LogLoss/F1）
# VSCode-Jupyter想定 / 3分類（OVR確率を前提）/ 色覚配慮パレット / 凡例は各パネル内

from __future__ import annotations

# ---- スレッド抑制：カーネル安定化 ----
import os as _os
_os.environ.setdefault("OMP_NUM_THREADS", "1")
_os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
_os.environ.setdefault("MKL_NUM_THREADS", "1")
_os.environ.setdefault("VECLIB_MAXIMUM_THREADS", "1")
_os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

# ---- 非GUIバックエンド（サーバ/CI安定化）----
import matplotlib
matplotlib.use("Agg")

import os
from pathlib import Path
from typing import Dict, Tuple, List, Callable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    precision_recall_curve, average_precision_score,
    roc_curve, auc, accuracy_score,
    log_loss, f1_score
)

# ====== 体裁（色・線幅・フォント） =========================================
plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 600,
    "font.size": 10,
    "axes.labelsize": 10,
    "axes.titlesize": 11,
    "legend.fontsize": 9,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "axes.linewidth": 0.8,
})

# 論文用カラーパレット（色覚配慮）
COLOR_MAP = {
    "Logistic":  "#1b9e77",  # green-teal
    "XGBoost":   "#7570b3",  # purple
    "LightGBM":  "#d95f02",  # orange
}
LINESTYLE_MAP = {"Logistic": "-", "XGBoost": "-", "LightGBM": "-"}
LINEWIDTH = 2.0
MARKER_STYLE = dict(marker="o", markersize=4, linewidth=1.5)

# ==== 出力形式（ジャーナル仕様） ============================================
SAVE_FMT = "tiff"               # {"tiff","jpeg","png"}（EJHFはtiff, JMIRはpng/jpegが多い）
SAVE_KW = dict(bbox_inches="tight")

def _savefig(fig: plt.Figure, path_wo_ext: Path):
    out = path_wo_ext.with_suffix(f".{SAVE_FMT}")
    fig.savefig(out, **SAVE_KW)
    plt.close(fig)
    _log(f"[save] {out}")

# ==== パス・定数 ============================================================
def _log(s: str):
    print(s, flush=True)

def _auto_results_dir() -> Path:
    cand = [
        Path("./results_temporal_main"),
        Path("./results_nested_logo"),
        Path("./results_light"),
        Path("./results"),
    ]
    for p in cand:
        if p.exists():
            _log(f"[info] RESULTS_DIR -> {p.resolve()}")
            return p
    p = Path("./results_temporal_main")
    p.mkdir(parents=True, exist_ok=True)
    _log(f"[info] RESULTS_DIR created -> {p.resolve()}")
    return p

RESULTS_DIR = _auto_results_dir()
FIG_DIR = RESULTS_DIR / "figures"; FIG_DIR.mkdir(parents=True, exist_ok=True)
TABLE_DIR = RESULTS_DIR / "tables";  TABLE_DIR.mkdir(parents=True, exist_ok=True)

# ハイパラ出力（必要に応じて別スクリプトで作成済みのものを同階層に）
# OUT_XLSX_HYPER = RESULTS_DIR / "Supplementary_Table_S1_Hyperparams.xlsx"  # 任意

LANDMARKS = ["LM0", "LM3"]
MODELS = ["Logistic", "XGBoost", "LightGBM"]
CLASS_LABELS = [0, 1, 2]
K = len(CLASS_LABELS)
EXPECTED_LABELS = np.array(CLASS_LABELS)

# DCAで陽性集合とみなすクラス
CLINICAL_POSITIVE_CLASSES = [1, 2]
THRESH_GRID = np.linspace(0.01, 0.99, 99)

# ==== ブート回数（計算負荷に応じて調整） ====================================
BOOT_OVERALL   = 1000  # Table2 pooled（Main）
BOOT_TEMPORAL  = 800   # TableS3 temporal（Supp）
BOOT_FACILITY  = 400   # TableS2 facility（Supp）
BOOT_DEPT      = 200   # TableS4 department（Supp）

# Temporalのカットオフ
TEMPORAL_CUTOFF = pd.Timestamp("2024-01-01")

RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

# ==== OOFローディング =======================================================
def _validate_labels(df: pd.DataFrame):
    u = np.sort(df["y_true"].dropna().unique())
    if not np.all(np.isin(u, EXPECTED_LABELS)):
        raise ValueError(f"Unexpected labels in y_true: {u}. Expected subset of {EXPECTED_LABELS}.")

def _read_oof_file(path: Path) -> pd.DataFrame | None:
    try:
        df = pd.read_csv(path)
    except Exception as e:
        _log(f"[warn] read fail: {path.name} ({e})"); return None
    if "y_true" not in df.columns:
        _log(f"[warn] y_true missing: {path.name}"); return None
    if ("Department" not in df.columns) and ("Department_en" in df.columns):
        df = df.copy()
        df["Department"] = df["Department_en"]
    keep = [c for c in ["INDEX","Facility","Landmark","Model","y_true","y_pred","Base_Date","Department","Department_en"] if c in df.columns]
    prob_cols = [c for c in df.columns if c.startswith("p_")]
    if not prob_cols:
        _log(f"[warn] probability columns missing: {path.name}"); return None
    keep += prob_cols
    df = df[keep].dropna(subset=["y_true"] + prob_cols)

    if "Landmark" not in df.columns:
        name = path.name
        lm = "LM3" if "LM3" in name else ("LM0" if "LM0" in name else None)
        if lm: df["Landmark"] = lm
    if "Model" not in df.columns:
        for m in MODELS:
            if m in path.name:
                df["Model"] = m
                break
    return df

def load_oof_merged(landmark: str, model: str) -> pd.DataFrame | None:
    patterns = [
        f"oof_temporal_{landmark}_{model}_iso.csv",
        f"oof_logo_{landmark}_{model}_*_iso.csv",
        f"oof_{landmark}_{model}_nestedLOGO.csv",
        f"oof_{landmark}_{model}.csv",
    ]
    files: List[Path] = []
    for pat in patterns:
        files.extend(RESULTS_DIR.rglob(pat))  # 再帰的に探索
    files = sorted(set(files))
    if not files:
        _log(f"[warn] OOF not found: {landmark}-{model}")
        return None

    dfs = []
    for f in files:
        d = _read_oof_file(f)
        if d is not None and len(d):
            dfs.append(d)
    if not dfs:
        return None
    df_all = pd.concat(dfs, axis=0, ignore_index=True)
    _validate_labels(df_all)
    return df_all

# ==== y, P 変換と基本ユーティリティ ========================================
def to_yP(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    y = df["y_true"].astype(np.int32).values
    P = np.zeros((len(df), K), dtype=np.float32)
    for j, k in enumerate(CLASS_LABELS):
        c = f"p_{k}"
        if c in df.columns:
            P[:, j] = df[c].astype(np.float32).values
    # 正規化・クリップ
    np.clip(P, 1e-7, 1.0, out=P)
    denom = P.sum(axis=1, keepdims=True)
    denom[denom == 0.0] = 1.0
    P /= denom
    return y, P

def _has_pos_neg(y_bin: np.ndarray) -> bool:
    if y_bin.size == 0:
        return False
    s = int(y_bin.sum())
    return (s > 0) and (s < y_bin.size)

# ==== メトリクス ============================================================
def macro_auc_from_yP(y: np.ndarray, P: np.ndarray, kind: str = "roc") -> float:
    vals = []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin):
            continue
        s = P[:, j]
        if kind == "roc":
            fpr, tpr, _ = roc_curve(y_bin, s)
            if fpr.size > 1:
                vals.append(auc(fpr, tpr))
        else:
            try:
                vals.append(average_precision_score(y_bin, s))
            except Exception:
                pass
    return float(np.mean(vals)) if vals else np.nan

def multiclass_brier_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    Y = np.zeros_like(P, dtype=np.float32)
    for i, yi in enumerate(y):
        idx = int(yi)
        if 0 <= idx < K:
            Y[i, idx] = 1.0
    diff = Y - P
    return float(np.mean(np.sum(diff * diff, axis=1)))

def ece_10bin_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    conf = P.max(axis=1)
    y_pred = P.argmax(axis=1)
    acc = (y == y_pred).astype(np.float32)
    bins = np.linspace(0.0, 1.0, 11, dtype=np.float32)
    ece = 0.0
    for i in range(10):
        lo, hi = bins[i], bins[i+1]
        m = (conf >= lo) & (conf < hi if i < 9 else conf <= hi)
        if m.any():
            ece += float(m.mean() * abs(acc[m].mean() - conf[m].mean()))
    return ece

def accuracy_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    return float((y == P.argmax(axis=1)).mean())

def logloss_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    P_clip = np.clip(P, 1e-15, 1 - 1e-15)
    row_sum = P_clip.sum(axis=1, keepdims=True)
    row_sum[row_sum == 0.0] = 1.0
    P_clip = P_clip / row_sum
    return float(log_loss(y, P_clip, labels=np.array(CLASS_LABELS)))

def f1_macro_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    y_pred = P.argmax(axis=1)
    return float(f1_score(y, y_pred, average="macro"))

# ---- 軽量ブートストラップ ---------------------------------------------------
def boot_stat_yP(y: np.ndarray, P: np.ndarray, func: Callable[[np.ndarray, np.ndarray], float],
                 n_boot: int) -> tuple[float, float, float]:
    n = y.size
    if n < 2:
        return np.nan, np.nan, np.nan
    vals = np.empty(n_boot, dtype=np.float32)
    for b in range(n_boot):
        idx = rng.integers(0, n, n, dtype=np.int32)
        vals[b] = func(y[idx], P[idx])
    mean = float(np.nanmean(vals))
    lo, hi = np.nanpercentile(vals, [2.5, 97.5]).astype(float)
    return mean, lo, hi

def compute_block_with_ci_df(df: pd.DataFrame, n_boot: int) -> Dict[str, float]:
    y, P = to_yP(df)
    # ---- Main（本文） ----
    auroc_m, auprc_m, brier_m = (
        macro_auc_from_yP(y, P, "roc"),
        macro_auc_from_yP(y, P, "pr"),
        multiclass_brier_from_yP(y, P),
    )
    # CIはブート
    auroc_m, auroc_lo, auroc_hi = boot_stat_yP(y, P, lambda yy, PP: macro_auc_from_yP(yy, PP, "roc"), n_boot)
    auprc_m, auprc_lo, auprc_hi = boot_stat_yP(y, P, lambda yy, PP: macro_auc_from_yP(yy, PP, "pr"),  n_boot)
    brier_m, brier_lo, brier_hi = boot_stat_yP(y, P, multiclass_brier_from_yP, n_boot)
    # ---- Supplement（参考） ----
    acc_m, acc_lo, acc_hi         = boot_stat_yP(y, P, accuracy_from_yP,      n_boot)
    ece_m, ece_lo, ece_hi         = boot_stat_yP(y, P, ece_10bin_from_yP,     n_boot)
    logloss_m, logloss_lo, logloss_hi = boot_stat_yP(y, P, logloss_from_yP,   n_boot)
    f1m_m, f1m_lo, f1m_hi         = boot_stat_yP(y, P, f1_macro_from_yP,      n_boot)

    return dict(
        # Main
        AUROC_macro=auroc_m, AUROC_low=auroc_lo, AUROC_high=auroc_hi,
        AUPRC_macro=auprc_m, AUPRC_low=auprc_lo, AUPRC_high=auprc_hi,
        Brier=brier_m, Brier_low=brier_lo, Brier_high=brier_hi,
        # Supplement
        Accuracy=acc_m, Accuracy_low=acc_lo, Accuracy_high=acc_hi,
        ECE=ece_m, ECE_low=ece_lo, ECE_high=ece_hi,
        LogLoss=logloss_m, LogLoss_low=logloss_lo, LogLoss_high=logloss_hi,
        F1_macro=f1m_m, F1_macro_low=f1m_lo, F1_macro_high=f1m_hi,
    )

# ==== 曲線補助 ==============================================================
def pr_curve_macro(df: pd.DataFrame):
    y, P = to_yP(df)
    rec_grid = np.linspace(0, 1, 501, dtype=np.float32)
    precs, aps = [], []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): 
            continue
        prec, rec, _ = precision_recall_curve(y_bin, P[:, j])
        order = np.argsort(rec)
        rec, prec = rec[order], prec[order]
        prec_interp = np.interp(rec_grid, rec, prec, left=prec[0], right=prec[-1])
        precs.append(prec_interp.astype(np.float32))
        try:
            aps.append(average_precision_score(y_bin, P[:, j]))
        except Exception:
            pass
    if not precs:
        return rec_grid, np.full_like(rec_grid, np.nan), np.nan
    return rec_grid, np.nanmean(np.vstack(precs), axis=0), float(np.nanmean(aps)) if aps else np.nan

def roc_curve_macro(df: pd.DataFrame):
    y, P = to_yP(df)
    fpr_grid = np.linspace(0, 1, 501, dtype=np.float32)
    tprs, aucs = [], []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin):
            continue
        fpr, tpr, _ = roc_curve(y_bin, P[:, j])
        order = np.argsort(fpr)
        fpr, tpr = fpr[order], tpr[order]
        tpr_interp = np.interp(fpr_grid, fpr, tpr, left=tpr[0], right=tpr[-1])
        tprs.append(tpr_interp.astype(np.float32))
        if fpr.size > 1:
            aucs.append(auc(fpr, tpr))
    if not tprs:
        return fpr_grid, np.full_like(fpr_grid, np.nan), np.nan
    return fpr_grid, np.nanmean(np.vstack(tprs), axis=0), float(np.nanmean(aucs)) if aucs else np.nan

def _legend_inside(ax, loc="best"):
    lgd = ax.legend(loc=loc, frameon=False, borderaxespad=0.0)
    return lgd

def _safe_plot_exists(d: Dict[str, pd.DataFrame]) -> bool:
    return bool(d) and all(isinstance(v, pd.DataFrame) and len(v) for v in d.values())

# ==== 図作成（Figure 2, 3, S1）=============================================
def plot_figure2(df_lm0: Dict[str, pd.DataFrame], df_lm3: Dict[str, pd.DataFrame]):
    """Figure 2（A:LM0 PR, B:LM3 PR, C:LM0 Reliability, D:LM3 Reliability）"""
    if not (_safe_plot_exists(df_lm0) and _safe_plot_exists(df_lm3)):
        _log("[warn] Figure2 skipped (OOF不足)"); return
    fig, axes = plt.subplots(2, 2, figsize=(10, 8), dpi=120)

    # A: LM0 PR
    ax = axes[0, 0]
    for mdl, df in df_lm0.items():
        x, y, ap = pr_curve_macro(df)
        ax.plot(x, y, label=f"{mdl} (AUPRC={ap:.3f})",
                color=COLOR_MAP.get(mdl, None), linestyle=LINESTYLE_MAP.get(mdl,"-"), linewidth=LINEWIDTH)
    ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
    ax.grid(True, alpha=0.3)
    ax.text(0.01, 0.99, "A", transform=ax.transAxes, ha="left", va="top", fontsize=12, fontweight="bold")
    _legend_inside(ax, loc="lower left")

    # B: LM3 PR
    ax = axes[0, 1]
    for mdl, df in df_lm3.items():
        x, y, ap = pr_curve_macro(df)
        ax.plot(x, y, label=f"{mdl} (AUPRC={ap:.3f})",
                color=COLOR_MAP.get(mdl, None), linestyle=LINESTYLE_MAP.get(mdl,"-"), linewidth=LINEWIDTH)
    ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
    ax.grid(True, alpha=0.3)
    ax.text(0.01, 0.99, "B", transform=ax.transAxes, ha="left", va="top", fontsize=12, fontweight="bold")
    _legend_inside(ax, loc="lower left")

    # C/D: Confidence-based reliability（Macro-ECEを凡例表示、CIなし）
    def _plot_reli(ax, df_dict, tag):
        for mdl, df in df_dict.items():
            y, P = to_yP(df)
            conf = P.max(axis=1)
            y_pred = P.argmax(axis=1)
            acc = (y == y_pred).astype(np.float32)
            bins = np.linspace(0, 1, 11, dtype=np.float32)
            xs, ys = [], []
            for i in range(10):
                m = (conf >= bins[i]) & (conf < bins[i+1] if i < 9 else conf <= bins[i+1])
                if m.any():
                    xs.append(float(conf[m].mean())); ys.append(float(acc[m].mean()))
            # 空ビン対策：全く点がない場合は恒等線に倒す
            if not xs:
                xs, ys = [0.0, 1.0], [0.0, 1.0]
            ece_val = ece_10bin_from_yP(y, P)
            ax.plot(xs, ys, **MARKER_STYLE,
                    label=f"{mdl} (Macro-ECE={ece_val:.3f})",
                    color=COLOR_MAP.get(mdl, None))
        ax.plot([0, 1], [0, 1], "--", color="gray", linewidth=1.2)
        ax.set_xlabel("Predicted confidence"); ax.set_ylabel("Observed accuracy")
        ax.grid(True, alpha=0.3)
        ax.text(0.01, 0.99, tag, transform=ax.transAxes, ha="left", va="top", fontsize=12, fontweight="bold")
        _legend_inside(ax, loc="lower right")

    _plot_reli(axes[1, 0], df_lm0, "C")
    _plot_reli(axes[1, 1], df_lm3, "D")

    plt.tight_layout()
    _savefig(fig, FIG_DIR / "Figure2")

def decision_curve_nb(df: pd.DataFrame, pos_classes: List[int], thresholds: np.ndarray) -> np.ndarray:
    y, P = to_yP(df)
    pos_mask = np.isin(y, np.array(pos_classes, dtype=np.int32)).astype(np.uint8)
    score = P[:, [CLASS_LABELS.index(c) for c in pos_classes]].max(axis=1)
    nb = np.empty_like(thresholds, dtype=np.float32)
    N = float(y.size)
    for i, t in enumerate(thresholds):
        pred_pos = (score >= t).astype(np.uint8)
        TP = int(((pred_pos == 1) & (pos_mask == 1)).sum())
        FP = int(((pred_pos == 1) & (pos_mask == 0)).sum())
        w = t / (1 - t)
        nb[i] = (TP / N) - (FP / N) * w
    return nb

def plot_figure3(df_lm0: Dict[str, pd.DataFrame], df_lm3: Dict[str, pd.DataFrame]):
    """Figure 3（A:LM0 DCA, B:LM3 DCA）"""
    if not (_safe_plot_exists(df_lm0) and _safe_plot_exists(df_lm3)):
        _log("[warn] Figure3 skipped (OOF不足)"); return
    fig, axes = plt.subplots(1, 2, figsize=(10, 4.5), dpi=120)

    # A: LM0
    ax = axes[0]
    for mdl, df in df_lm0.items():
        nb = decision_curve_nb(df, CLINICAL_POSITIVE_CLASSES, THRESH_GRID)
        ax.plot(THRESH_GRID, nb, label=mdl,
                color=COLOR_MAP.get(mdl, None), linestyle=LINESTYLE_MAP.get(mdl,"-"), linewidth=LINEWIDTH)
    ax.plot(THRESH_GRID, np.zeros_like(THRESH_GRID), "--", color="0.5", linewidth=1.2, label="Treat-none")
    any_df = next(iter(df_lm0.values()))
    y_any, _P_any = to_yP(any_df)
    prev = np.isin(y_any, np.array(CLINICAL_POSITIVE_CLASSES, dtype=np.int32)).mean()
    nb_all = prev - (1 - prev) * (THRESH_GRID / (1 - THRESH_GRID))
    ax.plot(THRESH_GRID, nb_all, ":", color="0.0", linewidth=1.4, label="Treat-all")
    ax.set_xlabel("Threshold probability"); ax.set_ylabel("Net benefit")
    ax.set_ylim(-0.75, 0.75)
    ax.grid(True, alpha=0.3)
    ax.text(0.01, 0.99, "A", transform=ax.transAxes, ha="left", va="top", fontsize=12, fontweight="bold")
    _legend_inside(ax, loc="lower right")

    # B: LM3
    ax = axes[1]
    for mdl, df in df_lm3.items():
        nb = decision_curve_nb(df, CLINICAL_POSITIVE_CLASSES, THRESH_GRID)
        ax.plot(THRESH_GRID, nb, label=mdl,
                color=COLOR_MAP.get(mdl, None), linestyle=LINESTYLE_MAP.get(mdl,"-"), linewidth=LINEWIDTH)
    ax.plot(THRESH_GRID, np.zeros_like(THRESH_GRID), "--", color="0.5", linewidth=1.2, label="Treat-none")
    any_df = next(iter(df_lm3.values()))
    y_any, _P_any = to_yP(any_df)
    prev = np.isin(y_any, np.array(CLINICAL_POSITIVE_CLASSES, dtype=np.int32)).mean()
    nb_all = prev - (1 - prev) * (THRESH_GRID / (1 - THRESH_GRID))
    ax.plot(THRESH_GRID, nb_all, ":", color="0.0", linewidth=1.4, label="Treat-all")
    ax.set_xlabel("Threshold probability"); ax.set_ylabel("Net benefit")
    ax.set_ylim(-0.75, 0.75)
    ax.grid(True, alpha=0.3)
    ax.text(0.01, 0.99, "B", transform=ax.transAxes, ha="left", va="top", fontsize=12, fontweight="bold")
    _legend_inside(ax, loc="lower right")

    plt.tight_layout()
    _savefig(fig, FIG_DIR / "Figure3")

def plot_figureS1(df_lm0: Dict[str, pd.DataFrame], df_lm3: Dict[str, pd.DataFrame]):
    """Figure S1（A:LM0 ROC, B:LM3 ROC）"""
    if not (_safe_plot_exists(df_lm0) and _safe_plot_exists(df_lm3)):
        _log("[warn] FigureS1 skipped (OOF不足)"); return
    fig, axes = plt.subplots(1, 2, figsize=(10, 4.5), dpi=120)

    ax = axes[0]
    for mdl, df in df_lm0.items():
        fpr, tpr, au = roc_curve_macro(df)
        ax.plot(fpr, tpr, label=f"{mdl} (AUROC={au:.3f})",
                color=COLOR_MAP.get(mdl, None), linestyle=LINESTYLE_MAP.get(mdl,"-"), linewidth=LINEWIDTH)
    ax.plot([0,1],[0,1],"--", color="0.5", linewidth=1.2)
    ax.set_xlabel("FPR"); ax.set_ylabel("TPR"); ax.grid(True, alpha=0.3)
    ax.text(0.01, 0.99, "A", transform=ax.transAxes, ha="left", va="top", fontsize=12, fontweight="bold")
    _legend_inside(ax, loc="lower right")

    ax = axes[1]
    for mdl, df in df_lm3.items():
        fpr, tpr, au = roc_curve_macro(df)
        ax.plot(fpr, tpr, label=f"{mdl} (AUROC={au:.3f})",
                color=COLOR_MAP.get(mdl, None), linestyle=LINESTYLE_MAP.get(mdl,"-"), linewidth=LINEWIDTH)
    ax.plot([0,1],[0,1],"--", color="0.5", linewidth=1.2)
    ax.set_xlabel("FPR"); ax.set_ylabel("TPR"); ax.grid(True, alpha=0.3)
    ax.text(0.01, 0.99, "B", transform=ax.transAxes, ha="left", va="top", fontsize=12, fontweight="bold")
    _legend_inside(ax, loc="lower right")

    plt.tight_layout()
    _savefig(fig, FIG_DIR / "FigureS1")

# ==== 表作成（Table 2 / S2 / S3 / S4） =====================================
def build_tables_excel():
    """
    Table 2: Overall pooled（LM0/LM3 × Model）→ Main（AUROC/AUPRC/Brier）
    Table S2: Facility-level（施設別）
    Table S3: Temporal（Base_Date >= TEMPORAL_CUTOFF）
    Table S4: Department-level（診療科別）
    """
    oof: Dict[Tuple[str, str], pd.DataFrame] = {}
    for lm in LANDMARKS:
        for mdl in MODELS:
            df = load_oof_merged(lm, mdl)
            if df is not None and len(df):
                oof[(lm, mdl)] = df

    pooled_rows = []
    for (lm, mdl), df in oof.items():
        met = compute_block_with_ci_df(df, BOOT_OVERALL)
        pooled_rows.append(dict(Landmark=lm, Model=mdl, **met))
    df_pooled = pd.DataFrame(pooled_rows)

    fac_rows = []
    for (lm, mdl), df in oof.items():
        if "Facility" not in df.columns:
            continue
        for fac, g in df.groupby("Facility", dropna=False):
            if len(g) < 2:
                continue
            met = compute_block_with_ci_df(g, BOOT_FACILITY)
            fac_rows.append(dict(Landmark=lm, Model=mdl, Facility=fac, **met))
    df_fac = pd.DataFrame(fac_rows)

    temp_rows = []
    for (lm, mdl), df in oof.items():
        if "Base_Date" not in df.columns:
            _log(f"[warn] Temporal skipped: {lm}-{mdl} (Base_Date missing)"); continue
        tmp = df.copy()
        tmp["Base_Date"] = pd.to_datetime(tmp["Base_Date"], errors="coerce")
        tmp = tmp[tmp["Base_Date"] >= TEMPORAL_CUTOFF]
        if len(tmp) < 2:
            _log(f"[warn] Temporal empty/small: {lm}-{mdl} (>= {TEMPORAL_CUTOFF.date()} rows < 2)"); continue
        met = compute_block_with_ci_df(tmp, BOOT_TEMPORAL)
        temp_rows.append(dict(Landmark=f"{lm}_Temporal", Model=mdl, **met))
    df_temp = pd.DataFrame(temp_rows)

    dept_rows = []
    for (lm, mdl), df in oof.items():
        dep_col = "Department" if "Department" in df.columns else ("Department_en" if "Department_en" in df.columns else None)
        if not dep_col:
            _log(f"[warn] Department-level skipped: {lm}-{mdl} (Department column missing)"); continue
        tmp = df.copy()
        tmp[dep_col] = tmp[dep_col].astype(str).replace({"": "Other"}).fillna("Other")
        for dep, g in tmp.groupby(dep_col, dropna=False):
            if len(g) < 5:
                continue
            met = compute_block_with_ci_df(g, BOOT_DEPT)
            dept_rows.append(dict(Landmark=lm, Model=mdl, Department=str(dep), **met))
    df_dept = pd.DataFrame(dept_rows)

    xlsx_path = TABLE_DIR / "metrics_summary.xlsx"
    # openpyxl に統一（環境依存を減らす）
    with pd.ExcelWriter(xlsx_path, engine="openpyxl") as xw:
        cols_main = [
            "Landmark","Model",
            "AUROC_macro","AUROC_low","AUROC_high",
            "AUPRC_macro","AUPRC_low","AUPRC_high",
            "Brier","Brier_low","Brier_high",
        ]
        if len(df_pooled):
            df_pooled[[c for c in cols_main if c in df_pooled.columns]].to_excel(
                xw, sheet_name="Table2_overall_pooled", index=False
            )
        sup_cols_tail = [
            "Accuracy","Accuracy_low","Accuracy_high",
            "ECE","ECE_low","ECE_high",
            "LogLoss","LogLoss_low","LogLoss_high",
            "F1_macro","F1_macro_low","F1_macro_high",
        ]
        cols_sup_base = [
            "Landmark","Model",
            "AUROC_macro","AUROC_low","AUROC_high",
            "AUPRC_macro","AUPRC_low","AUPRC_high",
            "Brier","Brier_low","Brier_high",
        ] + sup_cols_tail

        if len(df_fac):
            cols_fac = ["Landmark","Model","Facility"] + [c for c in cols_sup_base if c not in ("Landmark","Model")]
            df_fac[[c for c in cols_fac if c in df_fac.columns]].to_excel(
                xw, sheet_name="TableS2_facility_level", index=False
            )
        if len(df_temp):
            df_temp[[c for c in cols_sup_base if c in df_temp.columns]].to_excel(
                xw, sheet_name="TableS3_temporal", index=False
            )
        if len(df_dept):
            cols_dept = ["Landmark","Model","Department"] + [c for c in cols_sup_base if c not in ("Landmark","Model")]
            df_dept[[c for c in cols_dept if c in df_dept.columns]].to_excel(
                xw, sheet_name="TableS4_department_level", index=False
            )
    _log(f"[save] Excel tables -> {xlsx_path}")

# ==== メイン実行 ============================================================
def main(run_figures: bool = True, run_tables: bool = True):
    _log("[info] Loading OOFs (merged across temporal/logo)...")
    df_lm0 = {m: load_oof_merged("LM0", m) for m in MODELS}
    df_lm0 = {k: v for k, v in df_lm0.items() if v is not None and len(v)}
    df_lm3 = {m: load_oof_merged("LM3", m) for m in MODELS}
    df_lm3 = {k: v for k, v in df_lm3.items() if v is not None and len(v)}

    if not df_lm0 or not df_lm3:
        _log("[FATAL] OOF files not found/empty for LM0 or LM3. （出力先/ファイル名/列を確認）")
        return

    if run_figures:
        plot_figure2(df_lm0, df_lm3)
        plot_figure3(df_lm0, df_lm3)
        plot_figureS1(df_lm0, df_lm3)

    if run_tables:
        build_tables_excel()

    _log(f"[done] Figures -> {FIG_DIR}")
    _log(f"[done] Tables  -> {TABLE_DIR}")

if __name__ == "__main__":
    main()


[info] RESULTS_DIR -> C:\Users\tears\OneDrive\2025\2511\01_MV\results_temporal_main
[info] Loading OOFs (merged across temporal/logo)...
[save] results_temporal_main\figures\Figure2.tiff
[save] results_temporal_main\figures\Figure3.tiff
[save] results_temporal_main\figures\FigureS1.tiff
[save] Excel tables -> results_temporal_main\tables\metrics_summary.xlsx
[done] Figures -> results_temporal_main\figures
[done] Tables  -> results_temporal_main\tables


: 

###Table####

In [3]:
# -*- coding: utf-8 -*-
# tables_only_with_citl_slope.py
# 目的:
#   - Overall pooled は出力しない
#   - Temporal / Facility / Department のみをExcelに出力
#   - 指標: AUROC_macro, AUPRC_macro, Brier に加えて CITL_macro / Slope_macro（いずれも95% CI）
# 前提:
#   - results_* 配下に OOF CSV（y_true, p_0,p_1,p_2, Landmark, Model, Base_Date など）が存在

from __future__ import annotations

# ---- スレッド抑制：カーネル安定化 ----
import os as _os
_os.environ.setdefault("OMP_NUM_THREADS", "1")
_os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
_os.environ.setdefault("MKL_NUM_THREADS", "1")
_os.environ.setdefault("VECLIB_MAXIMUM_THREADS", "1")
_os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

from pathlib import Path
from typing import Dict, Tuple, List, Callable
import numpy as np
import pandas as pd

from sklearn.metrics import (
    precision_recall_curve, average_precision_score,
    roc_curve, auc, accuracy_score, log_loss, f1_score
)
from sklearn.linear_model import LogisticRegression

# ==== パス・定数 ============================================================
def _log(s: str): print(s, flush=True)

def _auto_results_dir() -> Path:
    cand = [
        Path("./results_temporal_main"),
        Path("./results_nested_logo"),
        Path("./results_light"),
        Path("./results"),
    ]
    for p in cand:
        if p.exists():
            _log(f"[info] RESULTS_DIR -> {p.resolve()}")
            return p
    p = Path("./results_temporal_main"); p.mkdir(parents=True, exist_ok=True)
    _log(f"[info] RESULTS_DIR created -> {p.resolve()}")
    return p

RESULTS_DIR = _auto_results_dir()
TABLE_DIR   = RESULTS_DIR / "tables";  TABLE_DIR.mkdir(parents=True, exist_ok=True)

LANDMARKS = ["LM0", "LM3"]
MODELS    = ["Logistic", "XGBoost", "LightGBM"]
CLASS_LABELS = [0, 1, 2]
K = len(CLASS_LABELS)
EXPECTED_LABELS = np.array(CLASS_LABELS)

# ==== ブート回数（増量） =====================================================
BOOT_TEMPORAL  = 1000   # Temporal（本文用）
BOOT_FACILITY  = 1000   # 施設別
BOOT_DEPT      = 1000   # 診療科別

# ==== Temporalのカットオフ ===================================================
TEMPORAL_CUTOFF = pd.Timestamp("2024-01-01")

RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

# ==== OOFローディング =======================================================
def _validate_labels(df: pd.DataFrame):
    u = np.sort(df["y_true"].dropna().unique())
    if not np.all(np.isin(u, EXPECTED_LABELS)):
        raise ValueError(f"Unexpected labels in y_true: {u}. Expected subset of {EXPECTED_LABELS}.")

def _read_oof_file(path: Path) -> pd.DataFrame | None:
    try:
        df = pd.read_csv(path)
    except Exception as e:
        _log(f"[warn] read fail: {path.name} ({e})"); return None
    if "y_true" not in df.columns:
        _log(f"[warn] y_true missing: {path.name}"); return None

    # Department列の補完（Department_enがあれば優先利用）
    if ("Department" not in df.columns) and ("Department_en" in df.columns):
        df = df.copy()
        df["Department"] = df["Department_en"]

    keep = [c for c in ["INDEX","Facility","Landmark","Model","y_true","y_pred","Base_Date","Department","Department_en"] if c in df.columns]
    prob_cols = [c for c in df.columns if c.startswith("p_")]
    if not prob_cols:
        _log(f"[warn] probability columns missing: {path.name}"); return None
    keep += prob_cols
    df = df[keep].dropna(subset=["y_true"] + prob_cols)

    if "Landmark" not in df.columns:
        name = path.name
        lm = "LM3" if "LM3" in name else ("LM0" if "LM0" in name else None)
        if lm: df["Landmark"] = lm
    if "Model" not in df.columns:
        for m in MODELS:
            if m in path.name:
                df["Model"] = m; break
    return df

def load_oof_merged(landmark: str, model: str) -> pd.DataFrame | None:
    patterns = [
        f"oof_temporal_{landmark}_{model}_iso.csv",
        f"oof_logo_{landmark}_{model}_*_iso.csv",
        f"oof_{landmark}_{model}_nestedLOGO.csv",
        f"oof_{landmark}_{model}.csv",
    ]
    files: List[Path] = []
    for pat in patterns:
        files.extend(RESULTS_DIR.rglob(pat))
    files = sorted(set(files))
    if not files:
        _log(f"[warn] OOF not found: {landmark}-{model}")
        return None

    dfs = []
    for f in files:
        d = _read_oof_file(f)
        if d is not None and len(d):
            dfs.append(d)
    if not dfs: return None
    df_all = pd.concat(dfs, axis=0, ignore_index=True)
    _validate_labels(df_all)
    return df_all

# ==== y, P 変換と基本ユーティリティ ========================================
def to_yP(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    y = df["y_true"].astype(np.int32).values
    P = np.zeros((len(df), K), dtype=np.float32)
    for j, k in enumerate(CLASS_LABELS):
        c = f"p_{k}"
        if c in df.columns:
            P[:, j] = df[c].astype(np.float32).values
    # 正規化・クリップ
    np.clip(P, 1e-7, 1.0, out=P)
    denom = P.sum(axis=1, keepdims=True)
    denom[denom == 0.0] = 1.0
    P /= denom
    return y, P

def _has_pos_neg(y_bin: np.ndarray) -> bool:
    if y_bin.size == 0: return False
    s = int(y_bin.sum())
    return (s > 0) and (s < y_bin.size)

# ==== メトリクス ============================================================
def macro_auc_from_yP(y: np.ndarray, P: np.ndarray, kind: str = "roc") -> float:
    vals = []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): continue
        s = P[:, j]
        if kind == "roc":
            fpr, tpr, _ = roc_curve(y_bin, s)
            if fpr.size > 1: vals.append(auc(fpr, tpr))
        else:
            try:
                vals.append(average_precision_score(y_bin, s))
            except Exception:
                pass
    return float(np.mean(vals)) if vals else np.nan

def multiclass_brier_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    Y = np.zeros_like(P, dtype=np.float32)
    for i, yi in enumerate(y):
        idx = int(yi)
        if 0 <= idx < K: Y[i, idx] = 1.0
    diff = Y - P
    return float(np.mean(np.sum(diff * diff, axis=1)))

def ece_10bin_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    conf = P.max(axis=1)
    y_pred = P.argmax(axis=1)
    acc = (y == y_pred).astype(np.float32)
    bins = np.linspace(0.0, 1.0, 11, dtype=np.float32)
    ece = 0.0
    for i in range(10):
        lo, hi = bins[i], bins[i+1]
        m = (conf >= lo) & (conf < hi if i < 9 else conf <= hi)
        if m.any():
            ece += float(m.mean() * abs(acc[m].mean() - conf[m].mean()))
    return ece

def accuracy_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    return float((y == P.argmax(axis=1)).mean())

def logloss_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    P_clip = np.clip(P, 1e-15, 1 - 1e-15)
    row_sum = P_clip.sum(axis=1, keepdims=True)
    row_sum[row_sum == 0.0] = 1.0
    P_clip = P_clip / row_sum
    return float(log_loss(y, P_clip, labels=np.array(CLASS_LABELS)))

def f1_macro_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    y_pred = P.argmax(axis=1)
    return float(f1_score(y, y_pred, average="macro"))

# ---- CITL / Calibration slope（OVR→macro） --------------------------------
def _logit(p: np.ndarray) -> np.ndarray:
    p = np.clip(p, 1e-12, 1 - 1e-12)
    return np.log(p / (1.0 - p))

def _fit_cali_no_penalty(x: np.ndarray, y_bin: np.ndarray) -> tuple[float, float]:
    """
    scikit-learn 1.6.1 互換: 正則化なし
      - primary: solver='lbfgs', penalty=None
      - fallback: penalty='l2', C=1e9
    戻り値: (intercept, slope)
    """
    # primary（正則化なし）
    try:
        lr = LogisticRegression(
            solver="lbfgs", penalty=None, fit_intercept=True, max_iter=2000
        )
        lr.fit(x, y_bin)
        return float(lr.intercept_[0]), float(lr.coef_[0, 0])
    except Exception:
        # fallback（L2極小正則化）
        lr = LogisticRegression(
            solver="lbfgs", penalty="l2", C=1e9, fit_intercept=True, max_iter=2000
        )
        lr.fit(x, y_bin)
        return float(lr.intercept_[0]), float(lr.coef_[0, 0])

def citl_slope_from_yP(y: np.ndarray, P: np.ndarray) -> tuple[float, float]:
    """
    2パラメータ再較正:  y_bin ~ a + b * logit(p_j)
    クラス j=0,1,2 で a,b を推定し macro 平均を返す。
    """
    citls, slopes = [], []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): 
            continue
        x = _logit(P[:, j]).reshape(-1, 1)
        a, b = _fit_cali_no_penalty(x, y_bin)
        slopes.append(b); citls.append(a)
    if not slopes:
        return np.nan, np.nan
    return float(np.mean(citls)), float(np.mean(slopes))

# ---- ブートストラップ ------------------------------------------------------
def boot_stat_yP(y: np.ndarray, P: np.ndarray, func: Callable[[np.ndarray, np.ndarray], float],
                 n_boot: int) -> tuple[float, float, float]:
    n = y.size
    if n < 2: return np.nan, np.nan, np.nan
    vals = np.empty(n_boot, dtype=np.float32)
    for b in range(n_boot):
        idx = rng.integers(0, n, n, dtype=np.int32)
        vals[b] = func(y[idx], P[idx])
    mean = float(np.nanmean(vals))
    lo, hi = np.nanpercentile(vals, [2.5, 97.5]).astype(float)
    return mean, lo, hi

def boot_citl_slope_yP(y: np.ndarray, P: np.ndarray, n_boot: int) -> tuple[float,float,float,float,float,float]:
    n = y.size
    if n < 2: return (np.nan,)*6
    vals = np.empty((n_boot, 2), dtype=np.float32)
    for b in range(n_boot):
        idx = rng.integers(0, n, n, dtype=np.int32)
        vals[b, :] = np.array(citl_slope_from_yP(y[idx], P[idx]), dtype=np.float32)
    mean_c, mean_s = float(np.nanmean(vals[:,0])), float(np.nanmean(vals[:,1]))
    lo_c, hi_c = np.nanpercentile(vals[:,0], [2.5, 97.5]).astype(float)
    lo_s, hi_s = np.nanpercentile(vals[:,1], [2.5, 97.5]).astype(float)
    return mean_c, lo_c, hi_c, mean_s, lo_s, hi_s

# ---- 集計ブロック ----------------------------------------------------------
def compute_block_with_ci_df(df: pd.DataFrame, n_boot: int) -> Dict[str, float]:
    y, P = to_yP(df)
    # Main
    auroc_m, auprc_m, brier_m = (
        macro_auc_from_yP(y, P, "roc"),
        macro_auc_from_yP(y, P, "pr"),
        multiclass_brier_from_yP(y, P),
    )
    auroc_m, auroc_lo, auroc_hi = boot_stat_yP(y, P, lambda yy, PP: macro_auc_from_yP(yy, PP, "roc"), n_boot)
    auprc_m, auprc_lo, auprc_hi = boot_stat_yP(y, P, lambda yy, PP: macro_auc_from_yP(yy, PP, "pr"),  n_boot)
    brier_m, brier_lo, brier_hi = boot_stat_yP(y, P, multiclass_brier_from_yP, n_boot)

    # CITL / slope（macro）
    citl_m, citl_lo, citl_hi, slope_m, slope_lo, slope_hi = boot_citl_slope_yP(y, P, n_boot)

    # Supplement（維持）
    acc_m, acc_lo, acc_hi             = boot_stat_yP(y, P, accuracy_from_yP,  n_boot)
    ece_m, ece_lo, ece_hi             = boot_stat_yP(y, P, ece_10bin_from_yP, n_boot)
    logloss_m, logloss_lo, logloss_hi = boot_stat_yP(y, P, logloss_from_yP,   n_boot)
    f1m_m, f1m_lo, f1m_hi             = boot_stat_yP(y, P, f1_macro_from_yP,  n_boot)

    return dict(
        # Main
        AUROC_macro=auroc_m, AUROC_low=auroc_lo, AUROC_high=auroc_hi,
        AUPRC_macro=auprc_m, AUPRC_low=auprc_lo, AUPRC_high=auprc_hi,
        Brier=brier_m, Brier_low=brier_lo, Brier_high=brier_hi,
        CITL_macro=citl_m, CITL_low=citl_lo, CITL_high=citl_hi,
        Slope_macro=slope_m, Slope_low=slope_lo, Slope_high=slope_hi,
        # Supplement
        Accuracy=acc_m, Accuracy_low=acc_lo, Accuracy_high=acc_hi,
        ECE=ece_m, ECE_low=ece_lo, ECE_high=ece_hi,
        LogLoss=logloss_m, LogLoss_low=logloss_lo, LogLoss_high=logloss_hi,
        F1_macro=f1m_m, F1_macro_low=f1m_lo, F1_macro_high=f1m_hi,
    )

# ==== 表作成（S2 / S3 / S4 のみ） ==========================================
def build_tables_excel():
    """
    出力:
      - TableS2_facility_level
      - TableS3_temporal
      - TableS4_department_level
    ※ Overall pooled は出力しない
    """
    oof: Dict[Tuple[str, str], pd.DataFrame] = {}
    for lm in LANDMARKS:
        for mdl in MODELS:
            df = load_oof_merged(lm, mdl)
            if df is not None and len(df):
                oof[(lm, mdl)] = df

    # 施設別
    fac_rows = []
    for (lm, mdl), df in oof.items():
        if "Facility" not in df.columns:
            continue
        for fac, g in df.groupby("Facility", dropna=False):
            if len(g) < 5:
                continue
            met = compute_block_with_ci_df(g, BOOT_FACILITY)
            fac_rows.append(dict(Landmark=lm, Model=mdl, Facility=fac, **met))
    df_fac = pd.DataFrame(fac_rows)

    # Temporal（Base_Date >= cutoff）
    temp_rows = []
    for (lm, mdl), df in oof.items():
        if "Base_Date" not in df.columns:
            _log(f"[warn] Temporal skipped: {lm}-{mdl} (Base_Date missing)"); continue
        tmp = df.copy()
        tmp["Base_Date"] = pd.to_datetime(tmp["Base_Date"], errors="coerce")
        tmp = tmp[tmp["Base_Date"] >= TEMPORAL_CUTOFF]
        if len(tmp) < 5:
            _log(f"[warn] Temporal small: {lm}-{mdl} (>= {TEMPORAL_CUTOFF.date()} rows < 5)"); continue
        met = compute_block_with_ci_df(tmp, BOOT_TEMPORAL)
        temp_rows.append(dict(Landmark=f"{lm}_Temporal", Model=mdl, **met))
    df_temp = pd.DataFrame(temp_rows)

    # 診療科別（Department or Department_en）
    dept_rows = []
    for (lm, mdl), df in oof.items():
        dep_col = "Department" if "Department" in df.columns else ("Department_en" if "Department_en" in df.columns else None)
        if not dep_col:
            _log(f"[warn] Department-level skipped: {lm}-{mdl} (Department column missing)"); continue
        tmp = df.copy()
        tmp[dep_col] = tmp[dep_col].astype(str).replace({"": "Other"}).fillna("Other")
        for dep, g in tmp.groupby(dep_col, dropna=False):
            if len(g) < 10:
                continue
            met = compute_block_with_ci_df(g, BOOT_DEPT)
            dept_rows.append(dict(Landmark=lm, Model=mdl, Department=str(dep), **met))
    df_dept = pd.DataFrame(dept_rows)

    # Excel出力（openpyxl）
    xlsx_path = TABLE_DIR / "metrics_summary_no_overall.xlsx"
    with pd.ExcelWriter(xlsx_path, engine="openpyxl") as xw:
        # 共通カラム順（Main＋CITL/Slope＋Supplement）
        cols_sup_tail = [
            "Accuracy","Accuracy_low","Accuracy_high",
            "ECE","ECE_low","ECE_high",
            "LogLoss","LogLoss_low","LogLoss_high",
            "F1_macro","F1_macro_low","F1_macro_high",
        ]
        cols_main_cali = [
            "AUROC_macro","AUROC_low","AUROC_high",
            "AUPRC_macro","AUPRC_low","AUPRC_high",
            "Brier","Brier_low","Brier_high",
            "CITL_macro","CITL_low","CITL_high",
            "Slope_macro","Slope_low","Slope_high",
        ]
        # Facility
        if len(df_fac):
            cols_fac = ["Landmark","Model","Facility"] + cols_main_cali + cols_sup_tail
            df_fac[[c for c in cols_fac if c in df_fac.columns]].to_excel(
                xw, sheet_name="TableS2_facility_level", index=False
            )
        # Temporal
        if len(df_temp):
            cols_temp = ["Landmark","Model"] + cols_main_cali + cols_sup_tail
            df_temp[[c for c in cols_temp if c in df_temp.columns]].to_excel(
                xw, sheet_name="TableS3_temporal", index=False
            )
        # Department
        if len(df_dept):
            cols_dept = ["Landmark","Model","Department"] + cols_main_cali + cols_sup_tail
            df_dept[[c for c in cols_dept if c in df_dept.columns]].to_excel(
                xw, sheet_name="TableS4_department_level", index=False
            )
    _log(f"[save] Excel tables -> {xlsx_path}")

# ==== エントリポイント ======================================================
def main():
    build_tables_excel()
    _log(f"[done] Tables  -> {TABLE_DIR}")

if __name__ == "__main__":
    main()


[info] RESULTS_DIR -> C:\Users\tears\OneDrive\2025\2511\01_MV\results_temporal_main
[save] Excel tables -> results_temporal_main\tables\metrics_summary_no_overall.xlsx
[done] Tables  -> results_temporal_main\tables


##Table-20251205

In [1]:
# -*- coding: utf-8 -*-
# tables_only_with_citl_slope.py
# 目的:
#   - Overall pooled は出力しない
#   - Temporal / Facility / Department のみをExcelに出力
#   - 指標: AUROC_macro, AUPRC_macro, Brier に加えて CITL_macro / Slope_macro（いずれも95% CI）
# 前提:
#   - results_* 配下に OOF CSV（y_true, p_0,p_1,p_2, Landmark, Model, Base_Date など）が存在

from __future__ import annotations

# ---- スレッド抑制：カーネル安定化 ----
import os as _os
_os.environ.setdefault("OMP_NUM_THREADS", "1")
_os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
_os.environ.setdefault("MKL_NUM_THREADS", "1")
_os.environ.setdefault("VECLIB_MAXIMUM_THREADS", "1")
_os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

from pathlib import Path
from typing import Dict, Tuple, List, Callable
import numpy as np
import pandas as pd

from sklearn.metrics import (
    precision_recall_curve, average_precision_score,
    roc_curve, auc, accuracy_score, log_loss, f1_score
)
from sklearn.linear_model import LogisticRegression

# ==== パス・定数 ============================================================
def _log(s: str): print(s, flush=True)

def _auto_results_dir() -> Path:
    cand = [
        Path("./results_temporal_main"),
        Path("./results_nested_logo"),
        Path("./results_light"),
        Path("./results"),
    ]
    for p in cand:
        if p.exists():
            _log(f"[info] RESULTS_DIR -> {p.resolve()}")
            return p
    p = Path("./results_temporal_main"); p.mkdir(parents=True, exist_ok=True)
    _log(f"[info] RESULTS_DIR created -> {p.resolve()}")
    return p

RESULTS_DIR = _auto_results_dir()
TABLE_DIR   = RESULTS_DIR / "tables";  TABLE_DIR.mkdir(parents=True, exist_ok=True)

LANDMARKS = ["LM0", "LM3"]
MODELS    = ["Logistic", "XGBoost", "LightGBM"]
CLASS_LABELS = [0, 1, 2]
K = len(CLASS_LABELS)
EXPECTED_LABELS = np.array(CLASS_LABELS)

# ==== ブート回数（増量） =====================================================
BOOT_TEMPORAL  = 1000   # Temporal（本文用）
BOOT_FACILITY  = 1000   # 施設別
BOOT_DEPT      = 1000   # 診療科別

# ==== Temporalのカットオフ ===================================================
TEMPORAL_CUTOFF = pd.Timestamp("2024-01-01")

RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

# ==== OOFローディング =======================================================
def _validate_labels(df: pd.DataFrame):
    u = np.sort(df["y_true"].dropna().unique())
    if not np.all(np.isin(u, EXPECTED_LABELS)):
        raise ValueError(f"Unexpected labels in y_true: {u}. Expected subset of {EXPECTED_LABELS}.")

def _read_oof_file(path: Path) -> pd.DataFrame | None:
    try:
        df = pd.read_csv(path)
    except Exception as e:
        _log(f"[warn] read fail: {path.name} ({e})"); return None
    if "y_true" not in df.columns:
        _log(f"[warn] y_true missing: {path.name}"); return None

    # Department列の補完（Department_enがあれば優先利用）
    if ("Department" not in df.columns) and ("Department_en" in df.columns):
        df = df.copy()
        df["Department"] = df["Department_en"]

    keep = [c for c in ["INDEX","Facility","Landmark","Model","y_true","y_pred","Base_Date","Department","Department_en"] if c in df.columns]
    prob_cols = [c for c in df.columns if c.startswith("p_")]
    if not prob_cols:
        _log(f"[warn] probability columns missing: {path.name}"); return None
    keep += prob_cols
    df = df[keep].dropna(subset=["y_true"] + prob_cols)

    if "Landmark" not in df.columns:
        name = path.name
        lm = "LM3" if "LM3" in name else ("LM0" if "LM0" in name else None)
        if lm: df["Landmark"] = lm
    if "Model" not in df.columns:
        for m in MODELS:
            if m in path.name:
                df["Model"] = m; break
    return df

def load_oof_merged(landmark: str, model: str) -> pd.DataFrame | None:
    patterns = [
        f"oof_temporal_{landmark}_{model}_iso.csv",
        f"oof_logo_{landmark}_{model}_*_iso.csv",
        f"oof_{landmark}_{model}_nestedLOGO.csv",
        f"oof_{landmark}_{model}.csv",
    ]
    files: List[Path] = []
    for pat in patterns:
        files.extend(RESULTS_DIR.rglob(pat))
    files = sorted(set(files))
    if not files:
        _log(f"[warn] OOF not found: {landmark}-{model}")
        return None

    dfs = []
    for f in files:
        d = _read_oof_file(f)
        if d is not None and len(d):
            dfs.append(d)
    if not dfs: return None
    df_all = pd.concat(dfs, axis=0, ignore_index=True)
    _validate_labels(df_all)
    return df_all

# ==== y, P 変換と基本ユーティリティ ========================================
def to_yP(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    y = df["y_true"].astype(np.int32).values
    P = np.zeros((len(df), K), dtype=np.float32)
    for j, k in enumerate(CLASS_LABELS):
        c = f"p_{k}"
        if c in df.columns:
            P[:, j] = df[c].astype(np.float32).values
    # 正規化・クリップ
    np.clip(P, 1e-7, 1.0, out=P)
    denom = P.sum(axis=1, keepdims=True)
    denom[denom == 0.0] = 1.0
    P /= denom
    return y, P

def _has_pos_neg(y_bin: np.ndarray) -> bool:
    if y_bin.size == 0: return False
    s = int(y_bin.sum())
    return (s > 0) and (s < y_bin.size)

# ==== メトリクス ============================================================
def macro_auc_from_yP(y: np.ndarray, P: np.ndarray, kind: str = "roc") -> float:
    vals = []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): continue
        s = P[:, j]
        if kind == "roc":
            fpr, tpr, _ = roc_curve(y_bin, s)
            if fpr.size > 1: vals.append(auc(fpr, tpr))
        else:
            try:
                vals.append(average_precision_score(y_bin, s))
            except Exception:
                pass
    return float(np.mean(vals)) if vals else np.nan

def multiclass_brier_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    Y = np.zeros_like(P, dtype=np.float32)
    for i, yi in enumerate(y):
        idx = int(yi)
        if 0 <= idx < K: Y[i, idx] = 1.0
    diff = Y - P
    return float(np.mean(np.sum(diff * diff, axis=1)))

def ece_10bin_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    conf = P.max(axis=1)
    y_pred = P.argmax(axis=1)
    acc = (y == y_pred).astype(np.float32)
    bins = np.linspace(0.0, 1.0, 11, dtype=np.float32)
    ece = 0.0
    for i in range(10):
        lo, hi = bins[i], bins[i+1]
        m = (conf >= lo) & (conf < hi if i < 9 else conf <= hi)
        if m.any():
            ece += float(m.mean() * abs(acc[m].mean() - conf[m].mean()))
    return ece

def accuracy_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    return float((y == P.argmax(axis=1)).mean())

def logloss_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    P_clip = np.clip(P, 1e-15, 1 - 1e-15)
    row_sum = P_clip.sum(axis=1, keepdims=True)
    row_sum[row_sum == 0.0] = 1.0
    P_clip = P_clip / row_sum
    return float(log_loss(y, P_clip, labels=np.array(CLASS_LABELS)))

def f1_macro_from_yP(y: np.ndarray, P: np.ndarray) -> float:
    y_pred = P.argmax(axis=1)
    return float(f1_score(y, y_pred, average="macro"))

# ---- CITL / Calibration slope（OVR→macro） --------------------------------
def _logit(p: np.ndarray) -> np.ndarray:
    p = np.clip(p, 1e-12, 1 - 1e-12)
    return np.log(p / (1.0 - p))

def _fit_cali_no_penalty(x: np.ndarray, y_bin: np.ndarray) -> tuple[float, float]:
    """
    scikit-learn 1.6.1 互換: 正則化なしを優先
      - primary: solver='lbfgs', penalty='none'
      - fallback: penalty='l2', C=1e9
    戻り値: (intercept, slope)
    """
    # primary（正則化なし）
    try:
        lr = LogisticRegression(
            solver="lbfgs",
            penalty="none",
            fit_intercept=True,
            max_iter=2000,
        )
        lr.fit(x, y_bin)
        return float(lr.intercept_[0]), float(lr.coef_[0, 0])
    except Exception:
        # fallback（L2極小正則化）
        lr = LogisticRegression(
            solver="lbfgs",
            penalty="l2",
            C=1e9,
            fit_intercept=True,
            max_iter=2000,
        )
        lr.fit(x, y_bin)
        return float(lr.intercept_[0]), float(lr.coef_[0, 0])

def citl_slope_from_yP(y: np.ndarray, P: np.ndarray) -> tuple[float, float]:
    """
    2パラメータ再較正:  y_bin ~ a + b * logit(p_j)
    クラス j=0,1,2 で a,b を推定し macro 平均を返す。
    """
    citls, slopes = [], []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): 
            continue
        x = _logit(P[:, j]).reshape(-1, 1)
        a, b = _fit_cali_no_penalty(x, y_bin)
        slopes.append(b); citls.append(a)
    if not slopes:
        return np.nan, np.nan
    return float(np.mean(citls)), float(np.mean(slopes))

# ---- ブートストラップ ------------------------------------------------------
def boot_stat_yP(y: np.ndarray, P: np.ndarray, func: Callable[[np.ndarray, np.ndarray], float],
                 n_boot: int) -> tuple[float, float, float]:
    n = y.size
    if n < 2: return np.nan, np.nan, np.nan
    vals = np.empty(n_boot, dtype=np.float32)
    for b in range(n_boot):
        idx = rng.integers(0, n, n, dtype=np.int32)
        vals[b] = func(y[idx], P[idx])
    mean = float(np.nanmean(vals))
    lo, hi = np.nanpercentile(vals, [2.5, 97.5]).astype(float)
    return mean, lo, hi

def boot_citl_slope_yP(y: np.ndarray, P: np.ndarray, n_boot: int) -> tuple[float,float,float,float,float,float]:
    n = y.size
    if n < 2: return (np.nan,)*6
    vals = np.empty((n_boot, 2), dtype=np.float32)
    for b in range(n_boot):
        idx = rng.integers(0, n, n, dtype=np.int32)
        vals[b, :] = np.array(citl_slope_from_yP(y[idx], P[idx]), dtype=np.float32)
    mean_c, mean_s = float(np.nanmean(vals[:,0])), float(np.nanmean(vals[:,1]))
    lo_c, hi_c = np.nanpercentile(vals[:,0], [2.5, 97.5]).astype(float)
    lo_s, hi_s = np.nanpercentile(vals[:,1], [2.5, 97.5]).astype(float)
    return mean_c, lo_c, hi_c, mean_s, lo_s, hi_s

# ---- 集計ブロック ----------------------------------------------------------
def compute_block_with_ci_df(df: pd.DataFrame, n_boot: int) -> Dict[str, float]:
    y, P = to_yP(df)
    # Main
    auroc_m, auprc_m, brier_m = (
        macro_auc_from_yP(y, P, "roc"),
        macro_auc_from_yP(y, P, "pr"),
        multiclass_brier_from_yP(y, P),
    )
    auroc_m, auroc_lo, auroc_hi = boot_stat_yP(y, P, lambda yy, PP: macro_auc_from_yP(yy, PP, "roc"), n_boot)
    auprc_m, auprc_lo, auprc_hi = boot_stat_yP(y, P, lambda yy, PP: macro_auc_from_yP(yy, PP, "pr"),  n_boot)
    brier_m, brier_lo, brier_hi = boot_stat_yP(y, P, multiclass_brier_from_yP, n_boot)

    # CITL / slope（macro）
    citl_m, citl_lo, citl_hi, slope_m, slope_lo, slope_hi = boot_citl_slope_yP(y, P, n_boot)

    # Supplement（維持）
    acc_m, acc_lo, acc_hi             = boot_stat_yP(y, P, accuracy_from_yP,  n_boot)
    ece_m, ece_lo, ece_hi             = boot_stat_yP(y, P, ece_10bin_from_yP, n_boot)
    logloss_m, logloss_lo, logloss_hi = boot_stat_yP(y, P, logloss_from_yP,   n_boot)
    f1m_m, f1m_lo, f1m_hi             = boot_stat_yP(y, P, f1_macro_from_yP,  n_boot)

    return dict(
        # Main
        AUROC_macro=auroc_m, AUROC_low=auroc_lo, AUROC_high=auroc_hi,
        AUPRC_macro=auprc_m, AUPRC_low=auprc_lo, AUPRC_high=auprc_hi,
        Brier=brier_m, Brier_low=brier_lo, Brier_high=brier_hi,
        CITL_macro=citl_m, CITL_low=citl_lo, CITL_high=citl_hi,
        Slope_macro=slope_m, Slope_low=slope_lo, Slope_high=slope_hi,
        # Supplement
        Accuracy=acc_m, Accuracy_low=acc_lo, Accuracy_high=acc_hi,
        ECE=ece_m, ECE_low=ece_lo, ECE_high=ece_hi,
        LogLoss=logloss_m, LogLoss_low=logloss_lo, LogLoss_high=logloss_hi,
        F1_macro=f1m_m, F1_macro_low=f1m_lo, F1_macro_high=f1m_hi,
    )

# ==== 表作成（S2 / S3 / S4 のみ） ==========================================
def build_tables_excel():
    """
    出力:
      - TableS2_facility_level
      - TableS3_temporal
      - TableS4_department_level
    ※ Overall pooled は出力しない
    """
    oof: Dict[Tuple[str, str], pd.DataFrame] = {}
    for lm in LANDMARKS:
        for mdl in MODELS:
            df = load_oof_merged(lm, mdl)
            if df is not None and len(df):
                oof[(lm, mdl)] = df

    # 施設別
    fac_rows = []
    for (lm, mdl), df in oof.items():
        if "Facility" not in df.columns:
            continue
        for fac, g in df.groupby("Facility", dropna=False):
            if len(g) < 5:
                continue
            met = compute_block_with_ci_df(g, BOOT_FACILITY)
            fac_rows.append(dict(Landmark=lm, Model=mdl, Facility=fac, **met))
    df_fac = pd.DataFrame(fac_rows)

    # Temporal（Base_Date >= cutoff）
    temp_rows = []
    for (lm, mdl), df in oof.items():
        if "Base_Date" not in df.columns:
            _log(f"[warn] Temporal skipped: {lm}-{mdl} (Base_Date missing)"); continue
        tmp = df.copy()
        tmp["Base_Date"] = pd.to_datetime(tmp["Base_Date"], errors="coerce")
        tmp = tmp[tmp["Base_Date"] >= TEMPORAL_CUTOFF]
        if len(tmp) < 5:
            _log(f"[warn] Temporal small: {lm}-{mdl} (>= {TEMPORAL_CUTOFF.date()} rows < 5)"); continue
        met = compute_block_with_ci_df(tmp, BOOT_TEMPORAL)
        temp_rows.append(dict(Landmark=f"{lm}_Temporal", Model=mdl, **met))
    df_temp = pd.DataFrame(temp_rows)

    # 診療科別（Department or Department_en）
    dept_rows = []
    for (lm, mdl), df in oof.items():
        dep_col = "Department" if "Department" in df.columns else ("Department_en" if "Department_en" in df.columns else None)
        if not dep_col:
            _log(f"[warn] Department-level skipped: {lm}-{mdl} (Department column missing)"); continue
        tmp = df.copy()
        tmp[dep_col] = tmp[dep_col].astype(str).replace({"": "Other"}).fillna("Other")
        for dep, g in tmp.groupby(dep_col, dropna=False):
            if len(g) < 10:
                continue
            met = compute_block_with_ci_df(g, BOOT_DEPT)
            dept_rows.append(dict(Landmark=lm, Model=mdl, Department=str(dep), **met))
    df_dept = pd.DataFrame(dept_rows)

    # Excel出力（openpyxl）
    xlsx_path = TABLE_DIR / "metrics_summary_no_overall.xlsx"
    with pd.ExcelWriter(xlsx_path, engine="openpyxl") as xw:
        # 共通カラム順（Main＋CITL/Slope＋Supplement）
        cols_sup_tail = [
            "Accuracy","Accuracy_low","Accuracy_high",
            "ECE","ECE_low","ECE_high",
            "LogLoss","LogLoss_low","LogLoss_high",
            "F1_macro","F1_macro_low","F1_macro_high",
        ]
        cols_main_cali = [
            "AUROC_macro","AUROC_low","AUROC_high",
            "AUPRC_macro","AUPRC_low","AUPRC_high",
            "Brier","Brier_low","Brier_high",
            "CITL_macro","CITL_low","CITL_high",
            "Slope_macro","Slope_low","Slope_high",
        ]
        # Facility
        if len(df_fac):
            cols_fac = ["Landmark","Model","Facility"] + cols_main_cali + cols_sup_tail
            df_fac[[c for c in cols_fac if c in df_fac.columns]].to_excel(
                xw, sheet_name="TableS2_facility_level", index=False
            )
        # Temporal
        if len(df_temp):
            cols_temp = ["Landmark","Model"] + cols_main_cali + cols_sup_tail
            df_temp[[c for c in cols_temp if c in df_temp.columns]].to_excel(
                xw, sheet_name="TableS3_temporal", index=False
            )
        # Department
        if len(df_dept):
            cols_dept = ["Landmark","Model","Department"] + cols_main_cali + cols_sup_tail
            df_dept[[c for c in cols_dept if c in df_dept.columns]].to_excel(
                xw, sheet_name="TableS4_department_level", index=False
            )
    _log(f"[save] Excel tables -> {xlsx_path}")

# ==== エントリポイント ======================================================
def main():
    build_tables_excel()
    _log(f"[done] Tables  -> {TABLE_DIR}")

if __name__ == "__main__":
    main()


[info] RESULTS_DIR -> C:\Users\tears\OneDrive\2025\2511\01_MV\results_temporal_main
[save] Excel tables -> results_temporal_main\tables\metrics_summary_no_overall.xlsx
[done] Tables  -> results_temporal_main\tables


##Figure##

In [1]:
# -*- coding: utf-8 -*-
# figures_temporal_IJMI.py
# ■対象：図のみ（Table出力なし）
# ■要件：
#   - 時間検証（Base_Date >= 2024-01-01）のOOFのみ使用
#   - Figure2：上段=PR（A:LM0, B:LM3）、下段=Calibration（C:LM0, D:LM3）
#       * PRの凡例=右上、罫線OFF、パネルラベル大（やや左寄せ）
#   - Figure3：DCA（A:LM0, B:LM3） 罫線OFF・凡例右上（小さめ）
#   - FigureS1：ROC（A:LM0, B:LM3） 罫線OFF
#   - 指標はAUROC/AUPRCとも macro（平均）を採用
#   - 凡例値はTemporalデータのブート平均（n_boot=800）を使用（既定のまま）
#   - IJMI向け：二段幅≈165mm（≒6.5 inch）, dpi=600
#   - 出力：JPEG/PNG両方（TIFFは出力しない）

from __future__ import annotations

# ---- スレッド抑制（安定化）----
import os as _os
_os.environ.setdefault("OMP_NUM_THREADS", "1")
_os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
_os.environ.setdefault("MKL_NUM_THREADS", "1")
_os.environ.setdefault("VECLIB_MAXIMUM_THREADS", "1")
_os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

# ---- 非GUIバックエンド ----
import matplotlib
matplotlib.use("Agg")

import os
from pathlib import Path
from typing import Dict, List, Tuple, Callable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    precision_recall_curve, average_precision_score,
    roc_curve, auc, accuracy_score, log_loss, f1_score
)

# ====== 体裁（IJMI向け） =====================================================
# 二段幅 ≈ 165 mm → 6.5 inch
INCH_W = 6.5
FIGSIZE_2x2 = (INCH_W, 6.4)   # Figure2（2×2）
FIGSIZE_1x2 = (INCH_W, 4.4)   # Figure3 / FigureS1（横2面）

plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 600,      # IJMI要件クリア
    "font.size": 10,
    "axes.labelsize": 10,
    "axes.titlesize": 11,
    "legend.fontsize": 8,    # ← 凡例を少し小さく
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "axes.linewidth": 0.8,
})

# 色（色覚配慮）
COLOR_MAP = {
    "Logistic":  "#1b9e77",  # green-teal
    "XGBoost":   "#7570b3",  # purple
    "LightGBM":  "#d95f02",  # orange
}
LINESTYLE_MAP = {"Logistic": "-", "XGBoost": "-", "LightGBM": "-"}
LINEWIDTH = 2.0
# Calibration点をやや大きく
MARKER_STYLE = dict(marker="o", markersize=5, linewidth=1.4)

# ---- 出力（JPEG & PNG） ----------------------------------------------------
SAVE_KW = dict(bbox_inches="tight")

def _log(s: str):
    print(s, flush=True)

def _savefig_all(fig: plt.Figure, path_wo_ext: Path):
    out_jpeg = path_wo_ext.with_suffix(".jpeg")
    out_png  = path_wo_ext.with_suffix(".png")
    fig.savefig(out_jpeg, format="jpeg", pil_kwargs={"quality": 95}, **SAVE_KW)
    fig.savefig(out_png,  format="png", **SAVE_KW)
    plt.close(fig)
    _log(f"[save] {out_jpeg}")
    _log(f"[save] {out_png}")

# ====== パス =================================================================
def _auto_results_dir() -> Path:
    cand = [Path("./results_temporal_main"), Path("./results_nested_logo"),
            Path("./results_light"), Path("./results")]
    for p in cand:
        if p.exists():
            _log(f"[info] RESULTS_DIR -> {p.resolve()}")
            return p
    p = Path("./results_temporal_main")
    p.mkdir(parents=True, exist_ok=True)
    _log(f"[info] RESULTS_DIR created -> {p.resolve()}")
    return p

RESULTS_DIR = _auto_results_dir()
FIG_DIR = RESULTS_DIR / "figures"; FIG_DIR.mkdir(parents=True, exist_ok=True)

LANDMARKS = ["LM0", "LM3"]
MODELS = ["Logistic", "XGBoost", "LightGBM"]
CLASS_LABELS = [0, 1, 2]
K = len(CLASS_LABELS)
EXPECTED_LABELS = np.array(CLASS_LABELS)

# ==== 時間検証・ブートなど ===================================================
TEMPORAL_CUTOFF = pd.Timestamp("2024-01-01")
USE_BOOT_FOR_LEGEND = True
BOOT_TEMPORAL = 2000   # 変更しない
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

# ==== OOFローディング（Temporalのみ抽出）====================================
def _validate_labels(df: pd.DataFrame):
    u = np.sort(df["y_true"].dropna().unique())
    if not np.all(np.isin(u, EXPECTED_LABELS)):
        raise ValueError(f"Unexpected labels in y_true: {u}. Expected subset of {EXPECTED_LABELS}.")

def _read_oof_file(path: Path) -> pd.DataFrame | None:
    try:
        df = pd.read_csv(path)
    except Exception as e:
        _log(f"[warn] read fail: {path.name} ({e})"); return None
    if "y_true" not in df.columns:
        _log(f"[warn] y_true missing: {path.name}"); return None
    if ("Department" not in df.columns) and ("Department_en" in df.columns):
        df = df.copy(); df["Department"] = df["Department_en"]
    keep = [c for c in ["INDEX","Facility","Landmark","Model","y_true","y_pred",
                        "Base_Date","Department","Department_en"] if c in df.columns]
    prob_cols = [c for c in df.columns if c.startswith("p_")]
    if not prob_cols:
        _log(f"[warn] probability columns missing: {path.name}"); return None
    keep += prob_cols
    df = df[keep].dropna(subset=["y_true"] + prob_cols)

    # 欠損時はファイル名から推定
    if "Landmark" not in df.columns:
        name = path.name
        lm = "LM3" if "LM3" in name else ("LM0" if "LM0" in name else None)
        if lm: df["Landmark"] = lm
    if "Model" not in df.columns:
        for m in MODELS:
            if m in path.name:
                df["Model"] = m
                break
    return df

def load_oof_temporal(landmark: str, model: str) -> pd.DataFrame | None:
    patterns = [
        f"oof_temporal_{landmark}_{model}_iso.csv",
        f"oof_logo_{landmark}_{model}_*_iso.csv",
        f"oof_{landmark}_{model}_nestedLOGO.csv",
        f"oof_{landmark}_{model}.csv",
    ]
    files: List[Path] = []
    for pat in patterns:
        files.extend(RESULTS_DIR.rglob(pat))
    files = sorted(set(files))
    if not files:
        _log(f"[warn] OOF not found: {landmark}-{model}"); return None

    dfs = []
    for f in files:
        d = _read_oof_file(f)
        if d is not None and len(d):
            dfs.append(d)
    if not dfs:
        return None
    df_all = pd.concat(dfs, axis=0, ignore_index=True)
    _validate_labels(df_all)

    # ---- Temporalフィルタ（本質）----
    if "Base_Date" in df_all.columns:
        df_all["Base_Date"] = pd.to_datetime(df_all["Base_Date"], errors="coerce")
        df_all = df_all[df_all["Base_Date"] >= TEMPORAL_CUTOFF]
    return df_all if len(df_all) else None

# ==== y, P 変換・メトリクス・ブート =========================================
def to_yP(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    y = df["y_true"].astype(np.int32).values
    P = np.zeros((len(df), K), dtype=np.float32)
    for j, k in enumerate(CLASS_LABELS):
        c = f"p_{k}"
        if c in df.columns:
            P[:, j] = df[c].astype(np.float32).values
    np.clip(P, 1e-7, 1.0, out=P)
    denom = P.sum(axis=1, keepdims=True); denom[denom == 0.0] = 1.0
    P /= denom
    return y, P

def _has_pos_neg(y_bin: np.ndarray) -> bool:
    if y_bin.size == 0: return False
    s = int(y_bin.sum())
    return (s > 0) and (s < y_bin.size)

def macro_auc_from_yP(y: np.ndarray, P: np.ndarray, kind: str = "roc") -> float:
    vals = []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): continue
        s = P[:, j]
        if kind == "roc":
            fpr, tpr, _ = roc_curve(y_bin, s)
            if fpr.size > 1: vals.append(auc(fpr, tpr))
        else:
            try:
                vals.append(average_precision_score(y_bin, s))
            except Exception:
                pass
    return float(np.mean(vals)) if vals else np.nan

def boot_stat_yP(y: np.ndarray, P: np.ndarray, func: Callable[[np.ndarray, np.ndarray], float],
                 n_boot: int) -> tuple[float, float, float]:
    n = y.size
    if n < 2: return np.nan, np.nan, np.nan
    vals = np.empty(n_boot, dtype=np.float32)
    for b in range(n_boot):
        idx = rng.integers(0, n, n, dtype=np.int32)
        vals[b] = func(y[idx], P[idx])
    mean = float(np.nanmean(vals))
    lo, hi = np.nanpercentile(vals, [2.5, 97.5]).astype(float)
    return mean, lo, hi

# ==== 曲線生成 ===============================================================
def pr_curve_macro(df: pd.DataFrame):
    y, P = to_yP(df)
    rec_grid = np.linspace(0, 1, 501, dtype=np.float32)
    precs, aps = [], []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): 
            continue
        prec, rec, _ = precision_recall_curve(y_bin, P[:, j])
        order = np.argsort(rec); rec, prec = rec[order], prec[order]
        prec_interp = np.interp(rec_grid, rec, prec, left=prec[0], right=prec[-1])
        precs.append(prec_interp.astype(np.float32))
        try:
            aps.append(average_precision_score(y_bin, P[:, j]))
        except Exception:
            pass
    if not precs:
        return rec_grid, np.full_like(rec_grid, np.nan), np.nan
    return rec_grid, np.nanmean(np.vstack(precs), axis=0), float(np.nanmean(aps)) if aps else np.nan

def roc_curve_macro(df: pd.DataFrame):
    y, P = to_yP(df)
    fpr_grid = np.linspace(0, 1, 501, dtype=np.float32)
    tprs, aucs = [], []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): continue
        fpr, tpr, _ = roc_curve(y_bin, P[:, j])
        order = np.argsort(fpr); fpr, tpr = fpr[order], tpr[order]
        tpr_interp = np.interp(fpr_grid, fpr, tpr, left=tpr[0], right=tpr[-1])
        tprs.append(tpr_interp.astype(np.float32))
        if fpr.size > 1: aucs.append(auc(fpr, tpr))
    if not tprs:
        return fpr_grid, np.full_like(fpr_grid, np.nan), np.nan
    return fpr_grid, np.nanmean(np.vstack(tprs), axis=0), float(np.nanmean(aucs)) if aucs else np.nan

# ==== 可視化ユーティリティ ===================================================
PANEL_TAG_SIZE = 16            # A–Dを大きめに
PANEL_TAG_X = 0.006            # ← 左寄せ微調整
PANEL_TAG_Y = 0.99

def _legend(ax, loc="best"):
    # 凡例の密度を上げて小さめ表示
    return ax.legend(loc=loc, frameon=False, borderaxespad=0.0,
                     handlelength=2.2, labelspacing=0.3)

def _safe_plot_exists(d: Dict[str, pd.DataFrame]) -> bool:
    return bool(d) and all(isinstance(v, pd.DataFrame) and len(v) for v in d.values())

# ==== Figure2：PR（上段）＋ Calibration（下段） ===============================
def plot_figure2(df_lm0: Dict[str, pd.DataFrame], df_lm3: Dict[str, pd.DataFrame]):
    if not (_safe_plot_exists(df_lm0) and _safe_plot_exists(df_lm3)):
        _log("[warn] Figure2 skipped (Temporal OOF不足)"); return
    fig, axes = plt.subplots(2, 2, figsize=FIGSIZE_2x2, dpi=120)

    # --- 上段：PR（凡例=右上、罫線OFF） ---
    ax = axes[0, 0]  # A: LM0 PR
    for mdl, df in df_lm0.items():
        x, y, ap_pt = pr_curve_macro(df)
        if USE_BOOT_FOR_LEGEND:
            yy, PP = to_yP(df)
            ap_boot, _, _ = boot_stat_yP(yy, PP, lambda Y, P: macro_auc_from_yP(Y, P, "pr"), BOOT_TEMPORAL)
            label = f"{mdl} (AUPRC={ap_boot:.3f})"
        else:
            label = f"{mdl} (AUPRC={ap_pt:.3f})"
        ax.plot(x, y, label=label,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "A", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="upper right")

    ax = axes[0, 1]  # B: LM3 PR
    for mdl, df in df_lm3.items():
        x, y, ap_pt = pr_curve_macro(df)
        if USE_BOOT_FOR_LEGEND:
            yy, PP = to_yP(df)
            ap_boot, _, _ = boot_stat_yP(yy, PP, lambda Y, P: macro_auc_from_yP(Y, P, "pr"), BOOT_TEMPORAL)
            label = f"{mdl} (AUPRC={ap_boot:.3f})"
        else:
            label = f"{mdl} (AUPRC={ap_pt:.3f})"
        ax.plot(x, y, label=label,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "B", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="upper right")

    # --- 下段：Calibration（10-bin、Macro-ECEを凡例に） ---
    def _plot_calib(ax, df_dict, tag):
        for mdl, df in df_dict.items():
            y, P = to_yP(df)
            conf = P.max(axis=1); y_pred = P.argmax(axis=1)
            acc = (y == y_pred).astype(np.float32)
            bins = np.linspace(0, 1, 11, dtype=np.float32)
            xs, ys = [], []
            for i in range(10):
                lo, hi = bins[i], bins[i+1]
                m = (conf >= lo) & (conf < hi if i < 9 else conf <= hi)
                if m.any():
                    xs.append(float(conf[m].mean()))
                    ys.append(float(acc[m].mean()))
            if not xs:  # 空ビン対策
                xs, ys = [0.0, 1.0], [0.0, 1.0]
            # Macro-ECE
            ece = 0.0
            for i in range(10):
                lo, hi = bins[i], bins[i+1]
                m = (conf >= lo) & (conf < hi if i < 9 else conf <= hi)
                if m.any():
                    ece += float(m.mean() * abs(acc[m].mean() - conf[m].mean()))
            ax.plot(xs, ys, **MARKER_STYLE,
                    label=f"{mdl} (Macro-ECE={ece:.3f})",
                    color=COLOR_MAP.get(mdl, None))
        ax.plot([0, 1], [0, 1], "--", color="gray", linewidth=1.2)
        ax.set_xlabel("Predicted confidence"); ax.set_ylabel("Observed accuracy")
        ax.grid(False)
        ax.text(PANEL_TAG_X, PANEL_TAG_Y, tag, transform=ax.transAxes, ha="left", va="top",
                fontsize=PANEL_TAG_SIZE, fontweight="bold")
        _legend(ax, loc="lower right")

    _plot_calib(axes[1, 0], df_lm0, "C")  # LM0
    _plot_calib(axes[1, 1], df_lm3, "D")  # LM3

    plt.tight_layout()
    _savefig_all(fig, FIG_DIR / "Figure2_temporal")

# ==== Figure3：DCA ===========================================================
CLINICAL_POSITIVE_CLASSES = [1, 2]
THRESH_GRID = np.linspace(0.01, 0.99, 99)

def decision_curve_nb(df: pd.DataFrame, pos_classes: List[int], thresholds: np.ndarray) -> np.ndarray:
    y, P = to_yP(df)
    pos_mask = np.isin(y, np.array(pos_classes, dtype=np.int32)).astype(np.uint8)
    score = P[:, [CLASS_LABELS.index(c) for c in pos_classes]].max(axis=1)
    nb = np.empty_like(thresholds, dtype=np.float32)
    N = float(y.size)
    for i, t in enumerate(thresholds):
        pred_pos = (score >= t).astype(np.uint8)
        TP = int(((pred_pos == 1) & (pos_mask == 1)).sum())
        FP = int(((pred_pos == 1) & (pos_mask == 0)).sum())
        w = t / (1 - t)
        nb[i] = (TP / N) - (FP / N) * w
    return nb

def plot_figure3(df_lm0: Dict[str, pd.DataFrame], df_lm3: Dict[str, pd.DataFrame]):
    if not (_safe_plot_exists(df_lm0) and _safe_plot_exists(df_lm3)):
        _log("[warn] Figure3 skipped (Temporal OOF不足)"); return
    fig, axes = plt.subplots(1, 2, figsize=FIGSIZE_1x2, dpi=120)

    # A: LM0
    ax = axes[0]
    for mdl, df in df_lm0.items():
        nb = decision_curve_nb(df, CLINICAL_POSITIVE_CLASSES, THRESH_GRID)
        ax.plot(THRESH_GRID, nb, label=mdl,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.plot(THRESH_GRID, np.zeros_like(THRESH_GRID), "--", color="0.5", linewidth=1.2, label="Treat-none")
    any_df = next(iter(df_lm0.values()))
    y_any, _ = to_yP(any_df)
    prev = np.isin(y_any, np.array(CLINICAL_POSITIVE_CLASSES, dtype=np.int32)).mean()
    nb_all = prev - (1 - prev) * (THRESH_GRID / (1 - THRESH_GRID))
    ax.plot(THRESH_GRID, nb_all, ":", color="0.0", linewidth=1.4, label="Treat-all")
    ax.set_xlabel("Threshold probability"); ax.set_ylabel("Net benefit")
    ax.set_ylim(-0.75, 0.75)
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "A", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="upper right")   # ← 右上・小さめ

    # B: LM3
    ax = axes[1]
    for mdl, df in df_lm3.items():
        nb = decision_curve_nb(df, CLINICAL_POSITIVE_CLASSES, THRESH_GRID)
        ax.plot(THRESH_GRID, nb, label=mdl,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.plot(THRESH_GRID, np.zeros_like(THRESH_GRID), "--", color="0.5", linewidth=1.2, label="Treat-none")
    any_df = next(iter(df_lm3.values()))
    y_any, _ = to_yP(any_df)
    prev = np.isin(y_any, np.array(CLINICAL_POSITIVE_CLASSES, dtype=np.int32)).mean()
    nb_all = prev - (1 - prev) * (THRESH_GRID / (1 - THRESH_GRID))
    ax.plot(THRESH_GRID, nb_all, ":", color="0.0", linewidth=1.4, label="Treat-all")
    ax.set_xlabel("Threshold probability"); ax.set_ylabel("Net benefit")
    ax.set_ylim(-0.75, 0.75)
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "B", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="upper right")   # ← 右上・小さめ

    plt.tight_layout()
    _savefig_all(fig, FIG_DIR / "Figure3_temporal")

# ==== FigureS1：ROC ==========================================================
def plot_figureS1(df_lm0: Dict[str, pd.DataFrame], df_lm3: Dict[str, pd.DataFrame]):
    if not (_safe_plot_exists(df_lm0) and _safe_plot_exists(df_lm3)):
        _log("[warn] FigureS1 skipped (Temporal OOF不足)"); return
    fig, axes = plt.subplots(1, 2, figsize=FIGSIZE_1x2, dpi=120)

    # A: LM0
    ax = axes[0]
    for mdl, df in df_lm0.items():
        fpr, tpr, _ = roc_curve_macro(df)
        if USE_BOOT_FOR_LEGEND:
            yy, PP = to_yP(df)
            au_boot, _, _ = boot_stat_yP(yy, PP, lambda Y, P: macro_auc_from_yP(Y, P, "roc"), BOOT_TEMPORAL)
            label = f"{mdl} (AUROC={au_boot:.3f})"
        else:
            _, _, au_pt = roc_curve_macro(df)
            label = f"{mdl} (AUROC={au_pt:.3f})"
        ax.plot(fpr, tpr, label=label,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.plot([0,1],[0,1],"--", color="0.5", linewidth=1.2)
    ax.set_xlabel("FPR"); ax.set_ylabel("TPR")
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "A", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="lower right")

    # B: LM3
    ax = axes[1]
    for mdl, df in df_lm3.items():
        fpr, tpr, _ = roc_curve_macro(df)
        if USE_BOOT_FOR_LEGEND:
            yy, PP = to_yP(df)
            au_boot, _, _ = boot_stat_yP(yy, PP, lambda Y, P: macro_auc_from_yP(Y, P, "roc"), BOOT_TEMPORAL)
            label = f"{mdl} (AUROC={au_boot:.3f})"
        else:
            _, _, au_pt = roc_curve_macro(df)
            label = f"{mdl} (AUROC={au_pt:.3f})"
        ax.plot(fpr, tpr, label=label,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.plot([0,1],[0,1],"--", color="0.5", linewidth=1.2)
    ax.set_xlabel("FPR"); ax.set_ylabel("TPR")
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "B", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="lower right")

    plt.tight_layout()
    _savefig_all(fig, FIG_DIR / "FigureS1_temporal")

# ==== メイン =================================================================
def main():
    _log("[info] Loading Temporal OOFs only ...")
    df_lm0 = {m: load_oof_temporal("LM0", m) for m in MODELS}
    df_lm0 = {k: v for k, v in df_lm0.items() if v is not None and len(v)}
    df_lm3 = {m: load_oof_temporal("LM3", m) for m in MODELS}
    df_lm3 = {k: v for k, v in df_lm3.items() if v is not None and len(v)}

    if not df_lm0 or not df_lm3:
        _log("[FATAL] Temporal OOF not found/empty for LM0 or LM3."); return

    plot_figure2(df_lm0, df_lm3)   # PR + Calibration（2×2）
    plot_figure3(df_lm0, df_lm3)   # DCA（1×2）
    plot_figureS1(df_lm0, df_lm3)  # ROC（1×2）

    _log(f"[done] Figures -> {FIG_DIR.resolve()}")

if __name__ == "__main__":
    main()


[info] RESULTS_DIR -> C:\Users\tears\OneDrive\2025\2511\01_MV\results_temporal_main
[info] Loading Temporal OOFs only ...
[save] results_temporal_main\figures\Figure2_temporal.jpeg
[save] results_temporal_main\figures\Figure2_temporal.png
[save] results_temporal_main\figures\Figure3_temporal.jpeg
[save] results_temporal_main\figures\Figure3_temporal.png
[save] results_temporal_main\figures\FigureS1_temporal.jpeg
[save] results_temporal_main\figures\FigureS1_temporal.png
[done] Figures -> C:\Users\tears\OneDrive\2025\2511\01_MV\results_temporal_main\figures


###Fig-20251205

In [2]:
# -*- coding: utf-8 -*-
# figures_temporal_IJMI.py
# ■対象：図のみ（Table出力なし）
# ■要件：
#   - 時間検証（Base_Date >= 2024-01-01）のOOFのみ使用
#   - Figure2：上段=PR（A:LM0, B:LM3）、下段=Calibration（C:LM0, D:LM3）
#       * PRの凡例=右上、罫線OFF、パネルラベル大（やや左寄せ）
#   - Figure3：DCA（A:LM0, B:LM3） 罫線OFF・凡例右上（小さめ）
#   - FigureS1：ROC（A:LM0, B:LM3） 罫線OFF
#   - 指標はAUROC/AUPRCとも macro（平均）を採用
#   - 凡例値はTemporalデータのブート平均（n_boot=800）を使用（既定のまま）
#   - IJMI向け：二段幅≈165mm（≒6.5 inch）, dpi=600
#   - 出力：JPEG/PNG両方（TIFFは出力しない）

from __future__ import annotations

# ---- スレッド抑制（安定化）----
import os as _os
_os.environ.setdefault("OMP_NUM_THREADS", "1")
_os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
_os.environ.setdefault("MKL_NUM_THREADS", "1")
_os.environ.setdefault("VECLIB_MAXIMUM_THREADS", "1")
_os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

# ---- 非GUIバックエンド ----
import matplotlib
matplotlib.use("Agg")

import os
from pathlib import Path
from typing import Dict, List, Tuple, Callable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    precision_recall_curve, average_precision_score,
    roc_curve, auc, accuracy_score, log_loss, f1_score
)

# ====== 体裁（IJMI向け） =====================================================
# 二段幅 ≈ 165 mm → 6.5 inch
INCH_W = 6.5
FIGSIZE_2x2 = (INCH_W, 6.4)   # Figure2（2×2）
FIGSIZE_1x2 = (INCH_W, 4.4)   # Figure3 / FigureS1（横2面）

plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 600,      # IJMI要件クリア
    "font.size": 10,
    "axes.labelsize": 10,
    "axes.titlesize": 11,
    "legend.fontsize": 8,    # ← 凡例を少し小さく
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "axes.linewidth": 0.8,
})

# 色（色覚配慮）
COLOR_MAP = {
    "Logistic":  "#1b9e77",  # green-teal
    "XGBoost":   "#7570b3",  # purple
    "LightGBM":  "#d95f02",  # orange
}
LINESTYLE_MAP = {"Logistic": "-", "XGBoost": "-", "LightGBM": "-"}
LINEWIDTH = 2.0
# Calibration点をやや大きく
MARKER_STYLE = dict(marker="o", markersize=5, linewidth=1.4)

# ---- 出力（JPEG & PNG） ----------------------------------------------------
SAVE_KW = dict(bbox_inches="tight")

def _log(s: str):
    print(s, flush=True)

def _savefig_all(fig: plt.Figure, path_wo_ext: Path):
    out_jpeg = path_wo_ext.with_suffix(".jpeg")
    out_png  = path_wo_ext.with_suffix(".png")
    fig.savefig(out_jpeg, format="jpeg", pil_kwargs={"quality": 95}, **SAVE_KW)
    fig.savefig(out_png,  format="png", **SAVE_KW)
    plt.close(fig)
    _log(f"[save] {out_jpeg}")
    _log(f"[save] {out_png}")

# ====== パス =================================================================
def _auto_results_dir() -> Path:
    cand = [Path("./results_temporal_main"), Path("./results_nested_logo"),
            Path("./results_light"), Path("./results")]
    for p in cand:
        if p.exists():
            _log(f"[info] RESULTS_DIR -> {p.resolve()}")
            return p
    p = Path("./results_temporal_main")
    p.mkdir(parents=True, exist_ok=True)
    _log(f"[info] RESULTS_DIR created -> {p.resolve()}")
    return p

RESULTS_DIR = _auto_results_dir()
FIG_DIR = RESULTS_DIR / "figures"; FIG_DIR.mkdir(parents=True, exist_ok=True)

LANDMARKS = ["LM0", "LM3"]
MODELS = ["Logistic", "XGBoost", "LightGBM"]
CLASS_LABELS = [0, 1, 2]
K = len(CLASS_LABELS)
EXPECTED_LABELS = np.array(CLASS_LABELS)

# ==== 時間検証・ブートなど ===================================================
TEMPORAL_CUTOFF = pd.Timestamp("2024-01-01")
USE_BOOT_FOR_LEGEND = True
BOOT_TEMPORAL = 800   # コメントの n_boot=800 に合わせる
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

# ==== OOFローディング（Temporalのみ抽出）====================================
def _validate_labels(df: pd.DataFrame):
    u = np.sort(df["y_true"].dropna().unique())
    if not np.all(np.isin(u, EXPECTED_LABELS)):
        raise ValueError(f"Unexpected labels in y_true: {u}. Expected subset of {EXPECTED_LABELS}.")

def _read_oof_file(path: Path) -> pd.DataFrame | None:
    try:
        df = pd.read_csv(path)
    except Exception as e:
        _log(f"[warn] read fail: {path.name} ({e})"); return None
    if "y_true" not in df.columns:
        _log(f"[warn] y_true missing: {path.name}"); return None
    if ("Department" not in df.columns) and ("Department_en" in df.columns):
        df = df.copy(); df["Department"] = df["Department_en"]
    keep = [c for c in ["INDEX","Facility","Landmark","Model","y_true","y_pred",
                        "Base_Date","Department","Department_en"] if c in df.columns]
    prob_cols = [c for c in df.columns if c.startswith("p_")]
    if not prob_cols:
        _log(f"[warn] probability columns missing: {path.name}"); return None
    keep += prob_cols
    df = df[keep].dropna(subset=["y_true"] + prob_cols)

    # 欠損時はファイル名から推定
    if "Landmark" not in df.columns:
        name = path.name
        lm = "LM3" if "LM3" in name else ("LM0" if "LM0" in name else None)
        if lm: df["Landmark"] = lm
    if "Model" not in df.columns:
        for m in MODELS:
            if m in path.name:
                df["Model"] = m
                break
    return df

def load_oof_temporal(landmark: str, model: str) -> pd.DataFrame | None:
    patterns = [
        f"oof_temporal_{landmark}_{model}_iso.csv",
        f"oof_logo_{landmark}_{model}_*_iso.csv",
        f"oof_{landmark}_{model}_nestedLOGO.csv",
        f"oof_{landmark}_{model}.csv",
    ]
    files: List[Path] = []
    for pat in patterns:
        files.extend(RESULTS_DIR.rglob(pat))
    files = sorted(set(files))
    if not files:
        _log(f"[warn] OOF not found: {landmark}-{model}"); return None

    dfs = []
    for f in files:
        d = _read_oof_file(f)
        if d is not None and len(d):
            dfs.append(d)
    if not dfs:
        return None
    df_all = pd.concat(dfs, axis=0, ignore_index=True)
    _validate_labels(df_all)

    # ---- Temporalフィルタ（本質）----
    if "Base_Date" in df_all.columns:
        df_all["Base_Date"] = pd.to_datetime(df_all["Base_Date"], errors="coerce")
        df_all = df_all[df_all["Base_Date"] >= TEMPORAL_CUTOFF]
    return df_all if len(df_all) else None

# ==== y, P 変換・メトリクス・ブート =========================================
def to_yP(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    y = df["y_true"].astype(np.int32).values
    P = np.zeros((len(df), K), dtype=np.float32)
    for j, k in enumerate(CLASS_LABELS):
        c = f"p_{k}"
        if c in df.columns:
            P[:, j] = df[c].astype(np.float32).values
    np.clip(P, 1e-7, 1.0, out=P)
    denom = P.sum(axis=1, keepdims=True); denom[denom == 0.0] = 1.0
    P /= denom
    return y, P

def _has_pos_neg(y_bin: np.ndarray) -> bool:
    if y_bin.size == 0: return False
    s = int(y_bin.sum())
    return (s > 0) and (s < y_bin.size)

def macro_auc_from_yP(y: np.ndarray, P: np.ndarray, kind: str = "roc") -> float:
    vals = []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): continue
        s = P[:, j]
        if kind == "roc":
            fpr, tpr, _ = roc_curve(y_bin, s)
            if fpr.size > 1: vals.append(auc(fpr, tpr))
        else:
            try:
                vals.append(average_precision_score(y_bin, s))
            except Exception:
                pass
    return float(np.mean(vals)) if vals else np.nan

def boot_stat_yP(y: np.ndarray, P: np.ndarray, func: Callable[[np.ndarray, np.ndarray], float],
                 n_boot: int) -> tuple[float, float, float]:
    n = y.size
    if n < 2: return np.nan, np.nan, np.nan
    vals = np.empty(n_boot, dtype=np.float32)
    for b in range(n_boot):
        idx = rng.integers(0, n, n, dtype=np.int32)
        vals[b] = func(y[idx], P[idx])
    mean = float(np.nanmean(vals))
    lo, hi = np.nanpercentile(vals, [2.5, 97.5]).astype(float)
    return mean, lo, hi

# ==== 曲線生成 ===============================================================
def pr_curve_macro(df: pd.DataFrame):
    y, P = to_yP(df)
    rec_grid = np.linspace(0, 1, 501, dtype=np.float32)
    precs, aps = [], []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): 
            continue
        prec, rec, _ = precision_recall_curve(y_bin, P[:, j])
        order = np.argsort(rec); rec, prec = rec[order], prec[order]
        prec_interp = np.interp(rec_grid, rec, prec, left=prec[0], right=prec[-1])
        precs.append(prec_interp.astype(np.float32))
        try:
            aps.append(average_precision_score(y_bin, P[:, j]))
        except Exception:
            pass
    if not precs:
        return rec_grid, np.full_like(rec_grid, np.nan), np.nan
    return rec_grid, np.nanmean(np.vstack(precs), axis=0), float(np.nanmean(aps)) if aps else np.nan

def roc_curve_macro(df: pd.DataFrame):
    y, P = to_yP(df)
    fpr_grid = np.linspace(0, 1, 501, dtype=np.float32)
    tprs, aucs = [], []
    for j in range(K):
        y_bin = (y == CLASS_LABELS[j]).astype(np.uint8)
        if not _has_pos_neg(y_bin): continue
        fpr, tpr, _ = roc_curve(y_bin, P[:, j])
        order = np.argsort(fpr); fpr, tpr = fpr[order], tpr[order]
        tpr_interp = np.interp(fpr_grid, fpr, tpr, left=tpr[0], right=tpr[-1])
        tprs.append(tpr_interp.astype(np.float32))
        if fpr.size > 1: aucs.append(auc(fpr, tpr))
    if not tprs:
        return fpr_grid, np.full_like(fpr_grid, np.nan), np.nan
    return fpr_grid, np.nanmean(np.vstack(tprs), axis=0), float(np.nanmean(aucs)) if aucs else np.nan

# ==== 可視化ユーティリティ ===================================================
PANEL_TAG_SIZE = 16            # A–Dを大きめに
PANEL_TAG_X = 0.006            # ← 左寄せ微調整
PANEL_TAG_Y = 0.99

def _legend(ax, loc="best"):
    # 凡例の密度を上げて小さめ表示
    return ax.legend(loc=loc, frameon=False, borderaxespad=0.0,
                     handlelength=2.2, labelspacing=0.3)

def _safe_plot_exists(d: Dict[str, pd.DataFrame]) -> bool:
    return bool(d) and all(isinstance(v, pd.DataFrame) and len(v) for v in d.values())

# ==== Figure2：PR（上段）＋ Calibration（下段） ===============================
def plot_figure2(df_lm0: Dict[str, pd.DataFrame], df_lm3: Dict[str, pd.DataFrame]):
    if not (_safe_plot_exists(df_lm0) and _safe_plot_exists(df_lm3)):
        _log("[warn] Figure2 skipped (Temporal OOF不足)"); return
    fig, axes = plt.subplots(2, 2, figsize=FIGSIZE_2x2, dpi=120)

    # --- 上段：PR（凡例=右上、罫線OFF） ---
    ax = axes[0, 0]  # A: LM0 PR
    for mdl, df in df_lm0.items():
        x, y, ap_pt = pr_curve_macro(df)
        if USE_BOOT_FOR_LEGEND:
            yy, PP = to_yP(df)
            ap_boot, _, _ = boot_stat_yP(yy, PP, lambda Y, P: macro_auc_from_yP(Y, P, "pr"), BOOT_TEMPORAL)
            label = f"{mdl} (AUPRC={ap_boot:.3f})"
        else:
            label = f"{mdl} (AUPRC={ap_pt:.3f})"
        ax.plot(x, y, label=label,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "A", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="upper right")

    ax = axes[0, 1]  # B: LM3 PR
    for mdl, df in df_lm3.items():
        x, y, ap_pt = pr_curve_macro(df)
        if USE_BOOT_FOR_LEGEND:
            yy, PP = to_yP(df)
            ap_boot, _, _ = boot_stat_yP(yy, PP, lambda Y, P: macro_auc_from_yP(Y, P, "pr"), BOOT_TEMPORAL)
            label = f"{mdl} (AUPRC={ap_boot:.3f})"
        else:
            label = f"{mdl} (AUPRC={ap_pt:.3f})"
        ax.plot(x, y, label=label,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "B", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="upper right")

    # --- 下段：Calibration（10-bin、ECEを凡例に） ---
    def _plot_calib(ax, df_dict, tag):
        for mdl, df in df_dict.items():
            y, P = to_yP(df)
            conf = P.max(axis=1); y_pred = P.argmax(axis=1)
            acc = (y == y_pred).astype(np.float32)
            bins = np.linspace(0, 1, 11, dtype=np.float32)
            xs, ys = [], []
            for i in range(10):
                lo, hi = bins[i], bins[i+1]
                m = (conf >= lo) & (conf < hi if i < 9 else conf <= hi)
                if m.any():
                    xs.append(float(conf[m].mean()))
                    ys.append(float(acc[m].mean()))
            if not xs:  # 空ビン対策
                xs, ys = [0.0, 1.0], [0.0, 1.0]
            # ECE
            ece = 0.0
            for i in range(10):
                lo, hi = bins[i], bins[i+1]
                m = (conf >= lo) & (conf < hi if i < 9 else conf <= hi)
                if m.any():
                    ece += float(m.mean() * abs(acc[m].mean() - conf[m].mean()))
            ax.plot(xs, ys, **MARKER_STYLE,
                    label=f"{mdl} (ECE={ece:.3f})",
                    color=COLOR_MAP.get(mdl, None))
        ax.plot([0, 1], [0, 1], "--", color="gray", linewidth=1.2)
        ax.set_xlabel("Predicted confidence"); ax.set_ylabel("Observed accuracy")
        ax.grid(False)
        ax.text(PANEL_TAG_X, PANEL_TAG_Y, tag, transform=ax.transAxes, ha="left", va="top",
                fontsize=PANEL_TAG_SIZE, fontweight="bold")
        _legend(ax, loc="lower right")

    _plot_calib(axes[1, 0], df_lm0, "C")  # LM0
    _plot_calib(axes[1, 1], df_lm3, "D")  # LM3

    plt.tight_layout()
    _savefig_all(fig, FIG_DIR / "Figure2_temporal")

# ==== Figure3：DCA ===========================================================
CLINICAL_POSITIVE_CLASSES = [1, 2]
THRESH_GRID = np.linspace(0.01, 0.99, 99)

def decision_curve_nb(df: pd.DataFrame, pos_classes: List[int], thresholds: np.ndarray) -> np.ndarray:
    y, P = to_yP(df)
    pos_mask = np.isin(y, np.array(pos_classes, dtype=np.int32)).astype(np.uint8)
    score = P[:, [CLASS_LABELS.index(c) for c in pos_classes]].max(axis=1)
    nb = np.empty_like(thresholds, dtype=np.float32)
    N = float(y.size)
    for i, t in enumerate(thresholds):
        pred_pos = (score >= t).astype(np.uint8)
        TP = int(((pred_pos == 1) & (pos_mask == 1)).sum())
        FP = int(((pred_pos == 1) & (pos_mask == 0)).sum())
        w = t / (1 - t)
        nb[i] = (TP / N) - (FP / N) * w
    return nb

def plot_figure3(df_lm0: Dict[str, pd.DataFrame], df_lm3: Dict[str, pd.DataFrame]):
    if not (_safe_plot_exists(df_lm0) and _safe_plot_exists(df_lm3)):
        _log("[warn] Figure3 skipped (Temporal OOF不足)"); return
    fig, axes = plt.subplots(1, 2, figsize=FIGSIZE_1x2, dpi=120)

    # A: LM0
    ax = axes[0]
    for mdl, df in df_lm0.items():
        nb = decision_curve_nb(df, CLINICAL_POSITIVE_CLASSES, THRESH_GRID)
        ax.plot(THRESH_GRID, nb, label=mdl,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.plot(THRESH_GRID, np.zeros_like(THRESH_GRID), "--", color="0.5", linewidth=1.2, label="Treat-none")
    any_df = next(iter(df_lm0.values()))
    y_any, _ = to_yP(any_df)
    prev = np.isin(y_any, np.array(CLINICAL_POSITIVE_CLASSES, dtype=np.int32)).mean()
    nb_all = prev - (1 - prev) * (THRESH_GRID / (1 - THRESH_GRID))
    ax.plot(THRESH_GRID, nb_all, ":", color="0.0", linewidth=1.4, label="Treat-all")
    ax.set_xlabel("Threshold probability"); ax.set_ylabel("Net benefit")
    ax.set_ylim(-0.75, 0.75)
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "A", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="upper right")   # ← 右上・小さめ

    # B: LM3
    ax = axes[1]
    for mdl, df in df_lm3.items():
        nb = decision_curve_nb(df, CLINICAL_POSITIVE_CLASSES, THRESH_GRID)
        ax.plot(THRESH_GRID, nb, label=mdl,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.plot(THRESH_GRID, np.zeros_like(THRESH_GRID), "--", color="0.5", linewidth=1.2, label="Treat-none")
    any_df = next(iter(df_lm3.values()))
    y_any, _ = to_yP(any_df)
    prev = np.isin(y_any, np.array(CLINICAL_POSITIVE_CLASSES, dtype=np.int32)).mean()
    nb_all = prev - (1 - prev) * (THRESH_GRID / (1 - THRESH_GRID))
    ax.plot(THRESH_GRID, nb_all, ":", color="0.0", linewidth=1.4, label="Treat-all")
    ax.set_xlabel("Threshold probability"); ax.set_ylabel("Net benefit")
    ax.set_ylim(-0.75, 0.75)
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "B", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="upper right")   # ← 右上・小さめ

    plt.tight_layout()
    _savefig_all(fig, FIG_DIR / "Figure3_temporal")

# ==== FigureS1：ROC ==========================================================
def plot_figureS1(df_lm0: Dict[str, pd.DataFrame], df_lm3: Dict[str, pd.DataFrame]):
    if not (_safe_plot_exists(df_lm0) and _safe_plot_exists(df_lm3)):
        _log("[warn] FigureS1 skipped (Temporal OOF不足)"); return
    fig, axes = plt.subplots(1, 2, figsize=FIGSIZE_1x2, dpi=120)

    # A: LM0
    ax = axes[0]
    for mdl, df in df_lm0.items():
        fpr, tpr, _ = roc_curve_macro(df)
        if USE_BOOT_FOR_LEGEND:
            yy, PP = to_yP(df)
            au_boot, _, _ = boot_stat_yP(yy, PP, lambda Y, P: macro_auc_from_yP(Y, P, "roc"), BOOT_TEMPORAL)
            label = f"{mdl} (AUROC={au_boot:.3f})"
        else:
            _, _, au_pt = roc_curve_macro(df)
            label = f"{mdl} (AUROC={au_pt:.3f})"
        ax.plot(fpr, tpr, label=label,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.plot([0,1],[0,1],"--", color="0.5", linewidth=1.2)
    ax.set_xlabel("FPR"); ax.set_ylabel("TPR")
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "A", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="lower right")

    # B: LM3
    ax = axes[1]
    for mdl, df in df_lm3.items():
        fpr, tpr, _ = roc_curve_macro(df)
        if USE_BOOT_FOR_LEGEND:
            yy, PP = to_yP(df)
            au_boot, _, _ = boot_stat_yP(yy, PP, lambda Y, P: macro_auc_from_yP(Y, P, "roc"), BOOT_TEMPORAL)
            label = f"{mdl} (AUROC={au_boot:.3f})"
        else:
            _, _, au_pt = roc_curve_macro(df)
            label = f"{mdl} (AUROC={au_pt:.3f})"
        ax.plot(fpr, tpr, label=label,
                color=COLOR_MAP.get(mdl, None),
                linestyle=LINESTYLE_MAP.get(mdl,"-"),
                linewidth=LINEWIDTH)
    ax.plot([0,1],[0,1],"--", color="0.5", linewidth=1.2)
    ax.set_xlabel("FPR"); ax.set_ylabel("TPR")
    ax.grid(False)
    ax.text(PANEL_TAG_X, PANEL_TAG_Y, "B", transform=ax.transAxes, ha="left", va="top",
            fontsize=PANEL_TAG_SIZE, fontweight="bold")
    _legend(ax, loc="lower right")

    plt.tight_layout()
    _savefig_all(fig, FIG_DIR / "FigureS1_temporal")

# ==== メイン =================================================================
def main():
    _log("[info] Loading Temporal OOFs only ...")
    df_lm0 = {m: load_oof_temporal("LM0", m) for m in MODELS}
    df_lm0 = {k: v for k, v in df_lm0.items() if v is not None and len(v)}
    df_lm3 = {m: load_oof_temporal("LM3", m) for m in MODELS}
    df_lm3 = {k: v for k, v in df_lm3.items() if v is not None and len(v)}

    if not df_lm0 or not df_lm3:
        _log("[FATAL] Temporal OOF not found/empty for LM0 or LM3."); return

    plot_figure2(df_lm0, df_lm3)   # PR + Calibration（2×2）
    plot_figure3(df_lm0, df_lm3)   # DCA（1×2）
    plot_figureS1(df_lm0, df_lm3)  # ROC（1×2）

    _log(f"[done] Figures -> {FIG_DIR.resolve()}")

if __name__ == "__main__":
    main()


[info] RESULTS_DIR -> C:\Users\tears\OneDrive\2025\2511\01_MV\results_temporal_main
[info] Loading Temporal OOFs only ...
[save] results_temporal_main\figures\Figure2_temporal.jpeg
[save] results_temporal_main\figures\Figure2_temporal.png
[save] results_temporal_main\figures\Figure3_temporal.jpeg
[save] results_temporal_main\figures\Figure3_temporal.png
[save] results_temporal_main\figures\FigureS1_temporal.jpeg
[save] results_temporal_main\figures\FigureS1_temporal.png
[done] Figures -> C:\Users\tears\OneDrive\2025\2511\01_MV\results_temporal_main\figures


SHAP

In [2]:
# -*- coding: utf-8 -*-
# ============================================================
# SHAP: LR (Fig S2) / LightGBM (Fig S3) / XGBoost (Fig 3)
# - LM0/LM3 × (Tracheostomy=1, Death=2) 2×2パネル
# - Departmentを英語4カテゴリに正規化 + Dept_detailも英語化
# - ラベルはASCII英語に統一（文字化け回避）、**は除去
# - JPEG/PNG 600dpi、Top20 CSV、A–Dラベル（白縁）
# ============================================================

from __future__ import annotations
from pathlib import Path
from typing import List, Dict, Tuple, Optional
import re, unicodedata

import numpy as np
import pandas as pd
import shap
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import xgboost as xgb
from PIL import Image, ImageDraw, ImageFont

# ===== パス =====
RESULTS_DIR = Path("./results_temporal_main")
IN_CSV     = RESULTS_DIR / "analysis_features.csv"
OUT_DIR    = RESULTS_DIR / "shap_plots"
OUT_DIR.mkdir(parents=True, exist_ok=True)

FIG_PREFIX = {"LR": "FigureS2_", "LGBM": "FigureS3_", "XGB": "Figure3_"}
CMAP = plt.get_cmap("coolwarm")

# ===== フォント（CJKフォールバック）=====
import matplotlib as mpl
mpl.rcParams.update({
    "font.sans-serif": [
        "Noto Sans CJK JP", "IPAexGothic", "Yu Gothic",
        "MS Gothic", "Hiragino Sans", "Arial", "DejaVu Sans"
    ],
    "font.family": "sans-serif",
    "pdf.fonttype": 42,
    "ps.fonttype": 42,
    "savefig.facecolor": "white",
})

# ============================================================
# 部門の英語正規化（あなたの関数を採用）
# ============================================================
def prepare_department_en(df: pd.DataFrame) -> pd.DataFrame:
    """
    Department/診療科名 を英語4カテゴリに正規化:
      - Cardio, Neuro, Emergency, Other
    併せて診療科の詳細を頻度で集約し Dept_detail に格納。
    """
    def _contains_any(text: str, keywords: list[str]) -> bool:
        if not isinstance(text, str):
            return False
        t_low = text.lower()
        for kw in keywords:
            if any(ord(ch) > 127 for ch in kw):  # 日本語キーワード
                if kw in text:
                    return True
            else:
                if kw in t_low:
                    return True
        return False

    if "Department" in df.columns:
        src = df["Department"].astype(str).fillna("Other")
    elif "診療科名" in df.columns:
        src = df["診療科名"].astype(str).fillna("Other")
    else:
        src = pd.Series(["Other"] * len(df), index=df.index, dtype="object")

    cardio_kw = ["心臓", "循環", "循環器", "心臓血管", "cardio", "cardiac", "cardiovascular", "cv"]
    neuro_kw  = ["脳", "神経", "脳神経", "neuro"]
    emerg_kw  = ["救", "救急", "er", "emerg", "emergency"]

    def map_dep(x: str) -> str:
        if _contains_any(x, cardio_kw):
            return "Cardio"
        if _contains_any(x, neuro_kw):
            return "Neuro"
        if _contains_any(x, emerg_kw):
            return "Emergency"
        return "Other"

    df = df.copy()
    df["Department_en"] = src.map(map_dep)

    if "診療科名" in df.columns:
        s = df["診療科名"].astype(str).fillna("Unknown")
    elif "Department" in df.columns:
        s = df["Department"].astype(str).fillna("Unknown")
    else:
        s = pd.Series(["Unknown"] * len(df), index=df.index)

    # 頻度20件未満はまとめる（閾値は適宜調整）
    freq = s.value_counts()
    keep = set(freq[freq >= 20].index)
    df["Dept_detail"] = s.where(s.isin(keep), other="OtherMinor")
    return df

# Dept_detail を英語に寄せる簡易マップ（足りない分はASCII化で落とす）
JP2EN_DEPT = {
    "心臓内科": "Cardiology", "循環器内科": "Cardiology", "心臓血管外科": "Cardiovascular Surgery",
    "消化器外科": "Gastrointestinal Surgery", "救急科": "Emergency Medicine",
    "脳神経外科": "Neurosurgery", "脳神経内科": "Neurology",
    "麻酔科": "Anesthesiology", "集中治療": "Critical Care",
}

# ===== 文字化け防止のサニタイズ =====
_non_ascii = re.compile(r"[^\x00-\x7F]+")

def ascii_safe(s: str) -> str:
    t = unicodedata.normalize("NFKD", s)
    t = _non_ascii.sub(" ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def sanitize_feature_names(raw_names: List[str]) -> List[str]:
    """
    - '**' を除去
    - 'col_val' → 'col: val'（カテゴリOneHot名を読みやすく）
    - 非ASCIIは落として英語のみ
    """
    out = []
    for n in raw_names:
        n = n.strip()
        if n.startswith("**") and n.endswith("**") and len(n) >= 4:
            n = n[2:-2].strip()

        # OneHot 名の体裁（先頭側の列名はそのまま、右側だけ1回だけ区切る）
        if "_" in n:
            col, val = n.split("_", 1)
            n = f"{col}: {val}"

        n = ascii_safe(n)
        out.append(n)
    return out

# ===== データ入出力 =====
def load_df(path: Path) -> Tuple[pd.DataFrame, str]:
    df = pd.read_csv(path, encoding="utf-8-sig")
    y_col = next((c for c in ["y_true", "Cox_Event", "target"] if c in df.columns), None)
    if y_col is None:
        raise KeyError("analysis_features.csv に y_true も Cox_Event も見つかりません。")
    if "Landmark" not in df.columns:
        raise KeyError("analysis_features.csv に Landmark 列がありません。")

    drop_cols = ["INDEX", "All_Day", "Base_Date", "施設名", "Facility"]
    df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")
    df[y_col] = pd.to_numeric(df[y_col], errors="coerce").astype("Int64")

    # ★ 部門の英語正規化 + Dept_detail英語化
    df = prepare_department_en(df)
    if "Dept_detail" in df.columns:
        df["Dept_detail"] = (
            df["Dept_detail"]
              .map(lambda x: JP2EN_DEPT.get(str(x), str(x)))  # 辞書で英語化
              .map(ascii_safe)                                # 非ASCII除去
        )
    return df, y_col

def dense(X):
    return X.toarray() if isinstance(X, csr_matrix) else np.asarray(X)

def build_xy(df: pd.DataFrame, lm: str, y_col: str) -> Tuple[csr_matrix, np.ndarray, List[str]]:
    d = df[df["Landmark"] == lm].dropna(subset=[y_col]).copy()
    if d.empty:
        raise ValueError(f"{lm}: 該当データがありません。")
    y = d[y_col].astype(int).values
    X = d.drop(columns=[y_col, "Landmark"], errors="ignore")

    # 以降のカテゴリ列に Dept_detail/Department_en を含める
    cat_cols = [c for c in X.columns if X[c].dtype == "object"]
    num_cols = [c for c in X.columns if c not in cat_cols]

    enc  = OneHotEncoder(handle_unknown="ignore", sparse_output=True, dtype=np.float32)
    simp = SimpleImputer(strategy="median")
    scl  = StandardScaler(with_mean=True, with_std=True)

    X_cat = enc.fit_transform(X[cat_cols]) if cat_cols else csr_matrix((len(X), 0), dtype=np.float32)
    cat_names = list(enc.get_feature_names_out(cat_cols)) if cat_cols else []

    X_num = scl.fit_transform(simp.fit_transform(X[num_cols])) if num_cols else np.zeros((len(X), 0))
    X_csr = hstack([csr_matrix(X_num, dtype=np.float32), X_cat], format="csr")

    # ★ 太字をやめ、英語・ASCII名へ統一
    raw_feat = list(num_cols) + cat_names
    feat = sanitize_feature_names(raw_feat)
    return X_csr, y, feat

def class_pos_map(y: np.ndarray) -> Dict[int, int]:
    labels = np.unique(y)
    return {int(lbl): int(i) for i, lbl in enumerate(labels)}

def align_shap_matrix(vals, n_features: int):
    if isinstance(vals, list):
        arr = []
        for v in vals:
            f = v.shape[1]
            if f == n_features + 1:
                v = v[:, :n_features]
            elif f != n_features:
                v = v[:, :min(f, n_features)]
            arr.append(v)
        return np.stack(arr, axis=0)
    else:
        arr = vals
        if arr.ndim == 3 and arr.shape[-1] not in (n_features, n_features + 1):
            arr = np.transpose(arr, (2, 0, 1))
        f = arr.shape[-1]
        if f == n_features + 1:
            arr = arr[..., :n_features]
        elif f != n_features:
            arr = arr[..., :min(f, n_features)]
        return arr

def save_top20(shap_arr: np.ndarray, feat: List[str], model_key: str, lm: str):
    K, n, F = shap_arr.shape
    feat_use = feat[:F]
    absmean = np.mean(np.abs(shap_arr), axis=1)
    for k in range(K):
        order = np.argsort(absmean[k])[::-1][:20]
        pd.DataFrame({
            "rank": np.arange(1, len(order)+1),
            "feature": [feat_use[i] for i in order],
            "mean_abs_shap": absmean[k][order],
        }).to_csv(
            OUT_DIR / f"top20_{model_key}_{lm}_cls{k}.csv",
            index=False, encoding="utf-8-sig"
        )

def save_summary_image(shap_1cls, X_dense, feat, out_path, max_display=20):
    Fm = shap_1cls.shape[1]
    feat_use = feat[:Fm]
    X_use = X_dense[:, :Fm]
    plt.figure(figsize=(6.85, 5.0))
    shap.summary_plot(
        shap_1cls, X_use, feature_names=feat_use,
        plot_type="dot", show=False, cmap=CMAP,
        max_display=max_display, color_bar=True
    )
    plt.tight_layout()
    plt.savefig(out_path.with_suffix(".jpg"), dpi=600, bbox_inches="tight")
    plt.savefig(out_path.with_suffix(".png"), dpi=600, bbox_inches="tight")
    plt.close()

def _load_font(img: Image.Image, rel_size: float = 0.05) -> Optional[ImageFont.FreeTypeFont]:
    font_size = max(12, int(min(img.size) * rel_size))
    for name in [
        "NotoSansCJK-Regular.ttc", "NotoSansCJKjp-Regular.otf",
        "ipaexg.ttf", "YuGothM.ttc", "msgothic.ttc",
        "arial.ttf", "DejaVuSans.ttf"
    ]:
        try:
            return ImageFont.truetype(name, font_size)
        except Exception:
            continue
    return None

def draw_panel_label(img: Image.Image, label: str, pad: int = 16) -> Image.Image:
    draw = ImageDraw.Draw(img)
    font = _load_font(img, rel_size=0.06)
    x, y = pad, pad
    for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
        draw.text((x+dx, y+dy), label, fill=(255,255,255), font=font)
    draw.text((x, y), label, fill=(0,0,0), font=font)
    return img

def assemble_2x2_panel(model_key: str, prefix: str):
    files = [
        OUT_DIR / f"{prefix}{model_key}_LM0_Tracheostomy.jpg",
        OUT_DIR / f"{prefix}{model_key}_LM0_Death.jpg",
        OUT_DIR / f"{prefix}{model_key}_LM3_Tracheostomy.jpg",
        OUT_DIR / f"{prefix}{model_key}_LM3_Death.jpg",
    ]
    if not all(p.exists() for p in files):
        missing = [p.name for p in files if not p.exists()]
        print(f"[warn] panel skip ({model_key}) missing: {missing}")
        return

    imgs = [Image.open(p).convert("RGB") for p in files]
    w, h = max(im.size[0] for im in imgs), max(im.size[1] for im in imgs)
    imgs = [im.resize((w, h)) for im in imgs]
    imgs = [draw_panel_label(im, lab) for im, lab in zip(imgs, ["A", "B", "C", "D"])]

    panel = Image.new("RGB", (w*2, h*2), (255, 255, 255))
    for i, im in enumerate(imgs):
        panel.paste(im, ((i % 2)*w, (i // 2)*h))

    out_jpg = OUT_DIR / f"{prefix}{model_key}_PANEL_2x2.jpg"
    out_png = OUT_DIR / f"{prefix}{model_key}_PANEL_2x2.png"
    panel.save(out_jpg, quality=95)
    panel.save(out_png)
    print(f"[panel] {out_jpg}\n[panel] {out_png}")

# ===== 実行 =====
if __name__ == "__main__":
    df, y_col = load_df(IN_CSV)

    # ---------- Logistic Regression (Figure S2) ----------
    for lm in ["LM0", "LM3"]:
        X_csr, y, feat = build_xy(df, lm, y_col)
        Xd = dense(X_csr)
        pos = class_pos_map(y)
        lr = LogisticRegression(
            max_iter=2000, penalty="l2", solver="saga",
            multi_class="multinomial", random_state=42, n_jobs=1, tol=1e-4
        )
        lr.fit(Xd, y)
        expl = shap.LinearExplainer(lr, Xd, feature_perturbation="interventional")
        vals = expl.shap_values(Xd)
        shap_arr = align_shap_matrix(vals, Xd.shape[1])
        save_top20(shap_arr, feat, "LR", lm)
        for label, tag in [(1, "Tracheostomy"), (2, "Death")]:
            if label in pos:
                k = pos[label]
                save_summary_image(shap_arr[k], Xd, feat, OUT_DIR / f"{FIG_PREFIX['LR']}LR_{lm}_{tag}")
    assemble_2x2_panel("LR", FIG_PREFIX["LR"])

    # ---------- LightGBM (Figure S3) ----------
    LGB_PARAMS = dict(
        objective="multiclass", n_estimators=400,
        learning_rate=0.05, subsample=0.85, colsample_bytree=0.85,
        reg_lambda=1.0, random_state=42, n_jobs=1, verbosity=-1,
    )
    for lm in ["LM0", "LM3"]:
        X_csr, y, feat = build_xy(df, lm, y_col)
        Xd = dense(X_csr)
        pos = class_pos_map(y)
        lgbm = LGBMClassifier(**{**LGB_PARAMS, "num_class": len(np.unique(y))})
        lgbm.fit(Xd, y)
        expl = shap.TreeExplainer(lgbm)
        vals = expl.shap_values(Xd)
        shap_arr = align_shap_matrix(vals, Xd.shape[1])
        save_top20(shap_arr, feat, "LGBM", lm)
        for label, tag in [(1, "Tracheostomy"), (2, "Death")]:
            if label in pos:
                k = pos[label]
                save_summary_image(shap_arr[k], Xd, feat, OUT_DIR / f"{FIG_PREFIX['LGBM']}LGBM_{lm}_{tag}")
    assemble_2x2_panel("LGBM", FIG_PREFIX["LGBM"])

    # ---------- XGBoost (Figure 3) ----------
    XGB_PARAMS = dict(
        objective="multi:softprob", tree_method="hist",
        eval_metric="mlogloss", n_estimators=400, max_depth=6,
        learning_rate=0.05, subsample=0.9, colsample_bytree=0.9,
        reg_lambda=1.0, random_state=42, n_jobs=1, verbosity=0,
    )
    for lm in ["LM0", "LM3"]:
        X_csr, y, feat = build_xy(df, lm, y_col)
        Xd = dense(X_csr)
        pos = class_pos_map(y)
        xgbm = xgb.XGBClassifier(**{**XGB_PARAMS, "num_class": len(np.unique(y))})
        xgbm.fit(Xd, y)
        expl = shap.TreeExplainer(xgbm)
        vals = expl.shap_values(Xd)
        shap_arr = align_shap_matrix(vals, Xd.shape[1])
        save_top20(shap_arr, feat, "XGB", lm)
        for label, tag in [(1, "Tracheostomy"), (2, "Death")]:
            if label in pos:
                k = pos[label]
                save_summary_image(shap_arr[k], Xd, feat, OUT_DIR / f"{FIG_PREFIX['XGB']}XGB_{lm}_{tag}")
    assemble_2x2_panel("XGB", FIG_PREFIX["XGB"])

    print("[OK] SHAP panels created with English ASCII feature names (no tofu, no **).")


c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Day3_ALT' 'Day3_AST' 'Day3_Alb' 'Day3_APTT' 'Day3_BUN' 'Day3_CK'
 'Day3_CRP' 'Day3_Cl' 'Day3_Cre' 'Day3_D-dimer' 'Day3_Hb' 'Day3_K'
 'Day3_Na' 'Day3_PLT' 'Day3_PT-INR' 'Day3_T-Bil' 'Day3_TP' 'Day3_WBC'
 'Day3_CoreSed' 'Day3_Opioid' 'Day3_NAD' 'Day3_Adrenaline' 'Day3_DOA'
 'Day3_DOB' 'Day3_Insulin' 'Day3_Steroid' 'Day3_Vasopressin'
 'Day3_Diuretic' 'Day3_AntiMRSA' 'Day3_Antibiotic' 'Day3_Carbapenem'
 'Day3_Antifungal']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\sklearn\linear_model\_

[panel] results_temporal_main\shap_plots\FigureS2_LR_PANEL_2x2.jpg
[panel] results_temporal_main\shap_plots\FigureS2_LR_PANEL_2x2.png


c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Day3_ALT' 'Day3_AST' 'Day3_Alb' 'Day3_APTT' 'Day3_BUN' 'Day3_CK'
 'Day3_CRP' 'Day3_Cl' 'Day3_Cre' 'Day3_D-dimer' 'Day3_Hb' 'Day3_K'
 'Day3_Na' 'Day3_PLT' 'Day3_PT-INR' 'Day3_T-Bil' 'Day3_TP' 'Day3_WBC'
 'Day3_CoreSed' 'Day3_Opioid' 'Day3_NAD' 'Day3_Adrenaline' 'Day3_DOA'
 'Day3_DOB' 'Day3_Insulin' 'Day3_Steroid' 'Day3_Vasopressin'
 'Day3_Diuretic' 'Day3_AntiMRSA' 'Day3_Antibiotic' 'Day3_Carbapenem'
 'Day3_Antifungal']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


[panel] results_temporal_main\shap_plots\FigureS3_LGBM_PANEL_2x2.jpg
[panel] results_temporal_main\shap_plots\FigureS3_LGBM_PANEL_2x2.png


c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Day3_ALT' 'Day3_AST' 'Day3_Alb' 'Day3_APTT' 'Day3_BUN' 'Day3_CK'
 'Day3_CRP' 'Day3_Cl' 'Day3_Cre' 'Day3_D-dimer' 'Day3_Hb' 'Day3_K'
 'Day3_Na' 'Day3_PLT' 'Day3_PT-INR' 'Day3_T-Bil' 'Day3_TP' 'Day3_WBC'
 'Day3_CoreSed' 'Day3_Opioid' 'Day3_NAD' 'Day3_Adrenaline' 'Day3_DOA'
 'Day3_DOB' 'Day3_Insulin' 'Day3_Steroid' 'Day3_Vasopressin'
 'Day3_Diuretic' 'Day3_AntiMRSA' 'Day3_Antibiotic' 'Day3_Carbapenem'
 'Day3_Antifungal']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


[panel] results_temporal_main\shap_plots\Figure3_XGB_PANEL_2x2.jpg
[panel] results_temporal_main\shap_plots\Figure3_XGB_PANEL_2x2.png
[OK] SHAP panels created with English ASCII feature names (no tofu, no **).


In [2]:
# -*- coding: utf-8 -*-
# ============================================================
# SHAP: LR (Fig S3) / LightGBM (Fig S4) / XGBoost (Fig 3)
# - LM0/LM3 × (Tracheostomy=1, Death=2) 2×2パネル
# - Departmentを英語4カテゴリに正規化 + Dept_detailも英語化
# - ラベルはASCII英語に統一（文字化け回避）、**は除去
# - JPEG/PNG 600dpi、Top20 CSV、A–Dラベル（白縁）
# ============================================================

from __future__ import annotations
from pathlib import Path
from typing import List, Dict, Tuple, Optional
import re
import unicodedata

import numpy as np
import pandas as pd
import shap
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import xgboost as xgb
from PIL import Image, ImageDraw, ImageFont

# ===== パス =====
RESULTS_DIR = Path("./results_temporal_main")
IN_CSV = RESULTS_DIR / "analysis_features.csv"
OUT_DIR = RESULTS_DIR / "shap_plots"
OUT_DIR.mkdir(parents=True, exist_ok=True)

FIG_PREFIX = {"LR": "FigureS3_", "LGBM": "FigureS4_", "XGB": "Figure3_"}
CMAP = plt.get_cmap("coolwarm")

# ===== フォント設定（CJKフォールバック + 太字）=====
import matplotlib as mpl
mpl.rcParams.update(
    {
        "font.sans-serif": [
            "Noto Sans CJK JP",
            "IPAexGothic",
            "Yu Gothic",
            "MS Gothic",
            "Hiragino Sans",
            "Arial",
            "DejaVu Sans",
        ],
        "font.family": "sans-serif",
        "pdf.fonttype": 42,
        "ps.fonttype": 42,
        "savefig.facecolor": "white",
        # 見やすさ向上のための太字・サイズ
        "font.weight": "bold",
        "axes.labelweight": "bold",
        "axes.titleweight": "bold",
    }
)

# ============================================================
# 部門の英語正規化
# ============================================================
def prepare_department_en(df: pd.DataFrame) -> pd.DataFrame:
    """
    Department/診療科名 を英語4カテゴリに正規化:
      - Cardio, Neuro, Emergency, Other
    併せて診療科の詳細を頻度で集約し Dept_detail に格納。
    """

    def _contains_any(text: str, keywords: list[str]) -> bool:
        if not isinstance(text, str):
            return False
        t_low = text.lower()
        for kw in keywords:
            # 日本語キーワードはそのまま一致を見に行く
            if any(ord(ch) > 127 for ch in kw):
                if kw in text:
                    return True
            else:
                if kw in t_low:
                    return True
        return False

    if "Department" in df.columns:
        src = df["Department"].astype(str).fillna("Other")
    elif "診療科名" in df.columns:
        src = df["診療科名"].astype(str).fillna("Other")
    else:
        src = pd.Series(["Other"] * len(df), index=df.index, dtype="object")

    cardio_kw = [
        "心臓",
        "循環",
        "循環器",
        "心臓血管",
        "cardio",
        "cardiac",
        "cardiovascular",
        "cv",
    ]
    neuro_kw = ["脳", "神経", "脳神経", "neuro"]
    emerg_kw = ["救", "救急", "er", "emerg", "emergency"]

    def map_dep(x: str) -> str:
        if _contains_any(x, cardio_kw):
            return "Cardio"
        if _contains_any(x, neuro_kw):
            return "Neuro"
        if _contains_any(x, emerg_kw):
            return "Emergency"
        return "Other"

    df = df.copy()
    df["Department_en"] = src.map(map_dep)

    if "診療科名" in df.columns:
        s = df["診療科名"].astype(str).fillna("Unknown")
    elif "Department" in df.columns:
        s = df["Department"].astype(str).fillna("Unknown")
    else:
        s = pd.Series(["Unknown"] * len(df), index=df.index)

    # 頻度20件未満は OtherMinor にまとめる
    freq = s.value_counts()
    keep = set(freq[freq >= 20].index)
    df["Dept_detail"] = s.where(s.isin(keep), other="OtherMinor")
    return df


# Dept_detail を英語に寄せる簡易マップ
JP2EN_DEPT = {
    "心臓内科": "Cardiology",
    "循環器内科": "Cardiology",
    "心臓血管外科": "Cardiovascular Surgery",
    "消化器外科": "Gastrointestinal Surgery",
    "救急科": "Emergency Medicine",
    "脳神経外科": "Neurosurgery",
    "脳神経内科": "Neurology",
    "麻酔科": "Anesthesiology",
    "集中治療": "Critical Care",
}

# ===== 文字化け防止のサニタイズ =====
_non_ascii = re.compile(r"[^\x00-\x7F]+")


def ascii_safe(s: str) -> str:
    t = unicodedata.normalize("NFKD", s)
    t = _non_ascii.sub(" ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t


def sanitize_feature_names(raw_names: List[str]) -> List[str]:
    """
    - '**' を除去
    - 'col_val' → 'col: val'（カテゴリOneHot名を読みやすく）
    - 特定略語のリネーム（CoreSed → Sedatives）
    - 非ASCIIは落として英語のみ
    """
    out = []
    for n in raw_names:
        n = n.strip()

        # 以前の太字表記 **xxx** を除去
        if n.startswith("**") and n.endswith("**") and len(n) >= 4:
            n = n[2:-2].strip()

        # OneHot 名の体裁（先頭側の列名はそのまま、右側だけ1回だけ区切る）
        if "_" in n:
            col, val = n.split("_", 1)
            n = f"{col}: {val}"

        # CoreSed → Sedatives へ統一
        n = n.replace("CoreSed", "Sedatives")

        # ASCII 化
        n = ascii_safe(n)
        out.append(n)
    return out


# ===== データ入出力 =====
def load_df(path: Path) -> Tuple[pd.DataFrame, str]:
    df = pd.read_csv(path, encoding="utf-8-sig")

    y_col = next(
        (c for c in ["y_true", "Cox_Event", "target"] if c in df.columns), None
    )
    if y_col is None:
        raise KeyError("analysis_features.csv に y_true も Cox_Event も見つかりません。")
    if "Landmark" not in df.columns:
        raise KeyError("analysis_features.csv に Landmark 列がありません。")

    drop_cols = ["INDEX", "All_Day", "Base_Date", "施設名", "Facility"]
    df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")
    df[y_col] = pd.to_numeric(df[y_col], errors="coerce").astype("Int64")

    # 部門の英語正規化 + Dept_detail英語化
    df = prepare_department_en(df)
    if "Dept_detail" in df.columns:
        df["Dept_detail"] = (
            df["Dept_detail"]
            .map(lambda x: JP2EN_DEPT.get(str(x), str(x)))  # 辞書で英語化
            .map(ascii_safe)  # 非ASCII除去
        )
    return df, y_col


def dense(X):
    return X.toarray() if isinstance(X, csr_matrix) else np.asarray(X)


def build_xy(df: pd.DataFrame, lm: str, y_col: str) -> Tuple[csr_matrix, np.ndarray, List[str]]:
    d = df[df["Landmark"] == lm].dropna(subset=[y_col]).copy()
    if d.empty:
        raise ValueError(f"{lm}: 該当データがありません。")

    y = d[y_col].astype(int).values
    X = d.drop(columns=[y_col, "Landmark"], errors="ignore")

    # カテゴリ列に Dept_detail/Department_en を含める
    cat_cols = [c for c in X.columns if X[c].dtype == "object"]
    num_cols = [c for c in X.columns if c not in cat_cols]

    enc = OneHotEncoder(handle_unknown="ignore", sparse_output=True, dtype=np.float32)
    simp = SimpleImputer(strategy="median")
    scl = StandardScaler(with_mean=True, with_std=True)

    if cat_cols:
        X_cat = enc.fit_transform(X[cat_cols])
        cat_names = list(enc.get_feature_names_out(cat_cols))
    else:
        X_cat = csr_matrix((len(X), 0), dtype=np.float32)
        cat_names = []

    if num_cols:
        X_num = scl.fit_transform(simp.fit_transform(X[num_cols]))
    else:
        X_num = np.zeros((len(X), 0))

    X_csr = hstack([csr_matrix(X_num, dtype=np.float32), X_cat], format="csr")

    raw_feat = list(num_cols) + cat_names
    feat = sanitize_feature_names(raw_feat)
    return X_csr, y, feat


def class_pos_map(y: np.ndarray) -> Dict[int, int]:
    labels = np.unique(y)
    return {int(lbl): int(i) for i, lbl in enumerate(labels)}


def align_shap_matrix(vals, n_features: int):
    """
    shap_values の形を (K, n, F) にそろえる。
    """
    if isinstance(vals, list):
        arr = []
        for v in vals:
            f = v.shape[1]
            if f == n_features + 1:
                v = v[:, :n_features]
            elif f != n_features:
                v = v[:, : min(f, n_features)]
            arr.append(v)
        return np.stack(arr, axis=0)
    else:
        arr = vals
        if arr.ndim == 3 and arr.shape[-1] not in (n_features, n_features + 1):
            arr = np.transpose(arr, (2, 0, 1))
        f = arr.shape[-1]
        if f == n_features + 1:
            arr = arr[..., :n_features]
        elif f != n_features:
            arr = arr[..., : min(f, n_features)]
        return arr


def save_top20(shap_arr: np.ndarray, feat: List[str], model_key: str, lm: str):
    K, n, F = shap_arr.shape
    feat_use = feat[:F]
    absmean = np.mean(np.abs(shap_arr), axis=1)
    for k in range(K):
        order = np.argsort(absmean[k])[::-1][:20]
        pd.DataFrame(
            {
                "rank": np.arange(1, len(order) + 1),
                "feature": [feat_use[i] for i in order],
                "mean_abs_shap": absmean[k][order],
            }
        ).to_csv(
            OUT_DIR / f"top20_{model_key}_{lm}_cls{k}.csv",
            index=False,
            encoding="utf-8-sig",
        )


def save_summary_image(
    shap_1cls: np.ndarray,
    X_dense: np.ndarray,
    feat: List[str],
    out_path: Path,
    max_display: int = 20,
):
    """
    単一クラス分の SHAP 行列から summary plot を作成して保存。
    フォント・ラベルを太字＆適度なサイズに調整。
    """
    Fm = shap_1cls.shape[1]
    feat_use = feat[:Fm]
    X_use = X_dense[:, :Fm]

    plt.figure(figsize=(6.85, 5.0))
    shap.summary_plot(
        shap_1cls,
        X_use,
        feature_names=feat_use,
        plot_type="dot",
        show=False,
        cmap=CMAP,
        max_display=max_display,
        color_bar=True,
    )

    # ▼ 可読性向上：フォント太字＆サイズ調整
    ax = plt.gca()
    for lbl in ax.get_yticklabels():
        lbl.set_fontweight("bold")
        lbl.set_fontsize(11)
    ax.xaxis.label.set_fontweight("bold")
    ax.xaxis.label.set_fontsize(12)

    plt.tight_layout()
    plt.savefig(out_path.with_suffix(".jpg"), dpi=600, bbox_inches="tight")
    plt.savefig(out_path.with_suffix(".png"), dpi=600, bbox_inches="tight")
    plt.close()


def _load_font(img: Image.Image, rel_size: float = 0.05) -> Optional[ImageFont.FreeTypeFont]:
    font_size = max(12, int(min(img.size) * rel_size))
    for name in [
        "NotoSansCJK-Regular.ttc",
        "NotoSansCJKjp-Regular.otf",
        "ipaexg.ttf",
        "YuGothM.ttc",
        "msgothic.ttc",
        "arial.ttf",
        "DejaVuSans.ttf",
    ]:
        try:
            return ImageFont.truetype(name, font_size)
        except Exception:
            continue
    return None


def draw_panel_label(img: Image.Image, label: str, pad: int = 16) -> Image.Image:
    draw = ImageDraw.Draw(img)
    font = _load_font(img, rel_size=0.06)
    x, y = pad, pad
    # 白縁 + 黒文字
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        draw.text((x + dx, y + dy), label, fill=(255, 255, 255), font=font)
    draw.text((x, y), label, fill=(0, 0, 0), font=font)
    return img


def assemble_2x2_panel(model_key: str, prefix: str):
    files = [
        OUT_DIR / f"{prefix}{model_key}_LM0_Tracheostomy.jpg",
        OUT_DIR / f"{prefix}{model_key}_LM0_Death.jpg",
        OUT_DIR / f"{prefix}{model_key}_LM3_Tracheostomy.jpg",
        OUT_DIR / f"{prefix}{model_key}_LM3_Death.jpg",
    ]
    if not all(p.exists() for p in files):
        missing = [p.name for p in files if not p.exists()]
        print(f"[warn] panel skip ({model_key}) missing: {missing}")
        return

    imgs = [Image.open(p).convert("RGB") for p in files]
    w = max(im.size[0] for im in imgs)
    h = max(im.size[1] for im in imgs)
    imgs = [im.resize((w, h)) for im in imgs]
    imgs = [draw_panel_label(im, lab) for im, lab in zip(imgs, ["A", "B", "C", "D"])]

    panel = Image.new("RGB", (w * 2, h * 2), (255, 255, 255))
    for i, im in enumerate(imgs):
        panel.paste(im, ((i % 2) * w, (i // 2) * h))

    out_jpg = OUT_DIR / f"{prefix}{model_key}_PANEL_2x2.jpg"
    out_png = OUT_DIR / f"{prefix}{model_key}_PANEL_2x2.png"
    panel.save(out_jpg, quality=95)
    panel.save(out_png)
    print(f"[panel] {out_jpg}\n[panel] {out_png}")


# ===== メイン実行 =====
if __name__ == "__main__":
    df, y_col = load_df(IN_CSV)

    # ---------- Logistic Regression (Figure S3) ----------
    for lm in ["LM0", "LM3"]:
        X_csr, y, feat = build_xy(df, lm, y_col)
        Xd = dense(X_csr)
        pos = class_pos_map(y)

        lr = LogisticRegression(
            max_iter=2000,
            penalty="l2",
            solver="saga",
            multi_class="multinomial",
            random_state=42,
            n_jobs=1,
            tol=1e-4,
        )
        lr.fit(Xd, y)

        expl = shap.LinearExplainer(lr, Xd, feature_perturbation="interventional")
        vals = expl.shap_values(Xd)
        shap_arr = align_shap_matrix(vals, Xd.shape[1])

        save_top20(shap_arr, feat, "LR", lm)

        for label, tag in [(1, "Tracheostomy"), (2, "Death")]:
            if label in pos:
                k = pos[label]
                save_summary_image(
                    shap_arr[k],
                    Xd,
                    feat,
                    OUT_DIR / f"{FIG_PREFIX['LR']}LR_{lm}_{tag}",
                )

    assemble_2x2_panel("LR", FIG_PREFIX["LR"])

    # ---------- LightGBM (Figure S4) ----------
    LGB_PARAMS = dict(
        objective="multiclass",
        n_estimators=400,
        learning_rate=0.05,
        subsample=0.85,
        colsample_bytree=0.85,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=1,
        verbosity=-1,
    )

    for lm in ["LM0", "LM3"]:
        X_csr, y, feat = build_xy(df, lm, y_col)
        Xd = dense(X_csr)
        pos = class_pos_map(y)

        lgbm = LGBMClassifier(**{**LGB_PARAMS, "num_class": len(np.unique(y))})
        lgbm.fit(Xd, y)

        expl = shap.TreeExplainer(lgbm)
        vals = expl.shap_values(Xd)
        shap_arr = align_shap_matrix(vals, Xd.shape[1])

        save_top20(shap_arr, feat, "LGBM", lm)

        for label, tag in [(1, "Tracheostomy"), (2, "Death")]:
            if label in pos:
                k = pos[label]
                save_summary_image(
                    shap_arr[k],
                    Xd,
                    feat,
                    OUT_DIR / f"{FIG_PREFIX['LGBM']}LGBM_{lm}_{tag}",
                )

    assemble_2x2_panel("LGBM", FIG_PREFIX["LGBM"])

    # ---------- XGBoost (Main Figure 3) ----------
    XGB_PARAMS = dict(
        objective="multi:softprob",
        tree_method="hist",
        eval_metric="mlogloss",
        n_estimators=400,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=1,
        verbosity=0,
    )

    for lm in ["LM0", "LM3"]:
        X_csr, y, feat = build_xy(df, lm, y_col)
        Xd = dense(X_csr)
        pos = class_pos_map(y)

        xgbm = xgb.XGBClassifier(**{**XGB_PARAMS, "num_class": len(np.unique(y))})
        xgbm.fit(Xd, y)

        expl = shap.TreeExplainer(xgbm)
        vals = expl.shap_values(Xd)
        shap_arr = align_shap_matrix(vals, Xd.shape[1])

        save_top20(shap_arr, feat, "XGB", lm)

        for label, tag in [(1, "Tracheostomy"), (2, "Death")]:
            if label in pos:
                k = pos[label]
                save_summary_image(
                    shap_arr[k],
                    Xd,
                    feat,
                    OUT_DIR / f"{FIG_PREFIX['XGB']}XGB_{lm}_{tag}",
                )

    assemble_2x2_panel("XGB", FIG_PREFIX["XGB"])

    print(
        "[OK] SHAP panels created with bold ASCII feature names "
        "(CoreSed → Sedatives, 600 dpi JPEG/PNG)."
    )


c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Day3_ALT' 'Day3_AST' 'Day3_Alb' 'Day3_APTT' 'Day3_BUN' 'Day3_CK'
 'Day3_CRP' 'Day3_Cl' 'Day3_Cre' 'Day3_D-dimer' 'Day3_Hb' 'Day3_K'
 'Day3_Na' 'Day3_PLT' 'Day3_PT-INR' 'Day3_T-Bil' 'Day3_TP' 'Day3_WBC'
 'Day3_CoreSed' 'Day3_Opioid' 'Day3_NAD' 'Day3_Adrenaline' 'Day3_DOA'
 'Day3_DOB' 'Day3_Insulin' 'Day3_Steroid' 'Day3_Vasopressin'
 'Day3_Diuretic' 'Day3_AntiMRSA' 'Day3_Antibiotic' 'Day3_Carbapenem'
 'Day3_Antifungal']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\sklearn\linear_model\_

[panel] results_temporal_main\shap_plots\FigureS3_LR_PANEL_2x2.jpg
[panel] results_temporal_main\shap_plots\FigureS3_LR_PANEL_2x2.png


c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Day3_ALT' 'Day3_AST' 'Day3_Alb' 'Day3_APTT' 'Day3_BUN' 'Day3_CK'
 'Day3_CRP' 'Day3_Cl' 'Day3_Cre' 'Day3_D-dimer' 'Day3_Hb' 'Day3_K'
 'Day3_Na' 'Day3_PLT' 'Day3_PT-INR' 'Day3_T-Bil' 'Day3_TP' 'Day3_WBC'
 'Day3_CoreSed' 'Day3_Opioid' 'Day3_NAD' 'Day3_Adrenaline' 'Day3_DOA'
 'Day3_DOB' 'Day3_Insulin' 'Day3_Steroid' 'Day3_Vasopressin'
 'Day3_Diuretic' 'Day3_AntiMRSA' 'Day3_Antibiotic' 'Day3_Carbapenem'
 'Day3_Antifungal']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


[panel] results_temporal_main\shap_plots\FigureS4_LGBM_PANEL_2x2.jpg
[panel] results_temporal_main\shap_plots\FigureS4_LGBM_PANEL_2x2.png


c:\Users\tears\anaconda3\envs\py311\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Day3_ALT' 'Day3_AST' 'Day3_Alb' 'Day3_APTT' 'Day3_BUN' 'Day3_CK'
 'Day3_CRP' 'Day3_Cl' 'Day3_Cre' 'Day3_D-dimer' 'Day3_Hb' 'Day3_K'
 'Day3_Na' 'Day3_PLT' 'Day3_PT-INR' 'Day3_T-Bil' 'Day3_TP' 'Day3_WBC'
 'Day3_CoreSed' 'Day3_Opioid' 'Day3_NAD' 'Day3_Adrenaline' 'Day3_DOA'
 'Day3_DOB' 'Day3_Insulin' 'Day3_Steroid' 'Day3_Vasopressin'
 'Day3_Diuretic' 'Day3_AntiMRSA' 'Day3_Antibiotic' 'Day3_Carbapenem'
 'Day3_Antifungal']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


[panel] results_temporal_main\shap_plots\Figure3_XGB_PANEL_2x2.jpg
[panel] results_temporal_main\shap_plots\Figure3_XGB_PANEL_2x2.png
[OK] SHAP panels created with bold ASCII feature names (CoreSed → Sedatives, 600 dpi JPEG/PNG).
